In [1]:
import pandas as pd
import gc
import pickle

In [89]:
Alias=pd.read_csv('data/source/Alias.csv')
Alias

,VC_fullname,Alias
0,10T Holdings,10T
1,123RF LIMITED,123RF
2,1535创投,杭州壹伍叁伍创业投资管理有限公司
3,1982 Ventures Pte.Ltd.,1982 Ventures
4,20/20 HealthCare Partners,20/20 HealthCare
...,...,...
43423,龙芯中科技术股份有限公司,龙芯中科技术
43424,龙芯中科技术股份有限公司,龙芯中科技术有限公司
43425,龙赢富泽资产管理(北京)有限公司,龙赢富泽资产
43426,龙赢富泽资产管理(北京)有限公司,龙赢富泽资产管理（北京）有限公司


In [90]:
ChinaVCtype=pd.read_csv('data/source/ChinaVCtype.csv')
ChinaVCtype

,VC_fullname,Unified_social_credit_ID,GVCagency_ne,GVCagency_equity,GVCsoe_ne,GVCsoe_equity,IVC_ne,CVC_ne,BVC_ne,FVC_ne,...,UVC_excl,PenVC_excl,Unknown,Direct_judge,Corporate_parent,Parent_unified_social_credit_ID,Parent_industry_qcc,Parent_listed,Parent_listedmarket,Parent_stockcode
0,1535创投,91330102341958567N,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
1,3M New Ventures,NaN,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,...,0,0,0,No,明尼苏达矿业制造（中国）投资有限公司,913100006073666908,租赁和商务服务业,NaN,NaN,NaN
2,"Amino Capital Management Company,LLC",NaN,0.0,0.0,1.0,99.989998,1.0,0.0,0.0,1.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
3,AppWorks Ventures之初创投,NaN,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
4,Ares Management LLC,NaN,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49182,龙旗控股,91460000MA5U1HC28G,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49183,龙洲股份,91350000753125975Q,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49184,龙马新三板进取基金1号（管理）,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49185,龚洁伟,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ChinaVCtype.columns

Index(['VC_fullname', 'Unified_social_credit_ID', 'GVCagency_ne',
       'GVCagency_equity', 'GVCsoe_ne', 'GVCsoe_equity', 'IVC_ne', 'CVC_ne',
       'BVC_ne', 'FVC_ne', 'UVC_ne', 'PenVC_ne', 'GVC_excl', 'GVC_equity',
       'CVC_excl', 'IVC_excl', 'BVC_excl', 'FVC_excl', 'UVC_excl',
       'PenVC_excl', 'Unknown', 'Direct_judge', 'Corporate_parent',
       'Parent_unified_social_credit_ID', 'Parent_industry_qcc',
       'Parent_listed', 'Parent_listedmarket', 'Parent_stockcode'],
      dtype='object')

现在这部分数据里面有两个文件，一个是Alias，一个是ChinaVCtype，两个文件是通过vcfullname连接的，现在要先把这两个文件整合，整合完之后和invest_all的文件里面的拆分后投资方名称匹配（on=alias），匹配到的字段包括'VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl'。也就是现在根据投资方名称先把投资事件拆分成不同投资方，然后再根据不同的投资方的名称和GVC type里面的alias匹配，新加入'VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl'这几个字段。之后再把他们整合到投资事件层面，整合的规则是：1）GVC_DUMMY: 对于每一个投资事件，只要有一个投资方里面的GVC_excl是1，则这个投资事件的GVC就是1，2）GVC_Continuous：对于每一个投资事件，计算投资方中GVC_excl是1的比例

In [91]:
ChinaVCtype.VC_fullname.nunique()

49187

In [8]:
Alias.groupby('Alias').VC_fullname.nunique().sort_values()

Alias
(Ad)Ventures    1
108度集团          1
10T             1
11.2 Capital    1
123RF           1
               ..
通和资本            2
源渡创投            2
微光创投            2
日本SBI控股株式会社     2
溪山天使会           2
Name: VC_fullname, Length: 43305, dtype: int64

In [9]:
Alias[Alias.Alias=='溪山天使会']

,VC_fullname,Alias
36418,溪山股权投资基金管理(深圳)有限公司,溪山天使会
36419,溪山股权投资基金管理（深圳）有限公司,溪山天使会


In [10]:
Alias[Alias.Alias=='日本SBI控股株式会社']

,VC_fullname,Alias
1785,SBI Holdings,日本SBI控股株式会社
26403,日本SBI控股株式会社北京代表处,日本SBI控股株式会社


In [16]:
Alias.groupby('VC_fullname').Alias.nunique().sort_values()

VC_fullname
4BIO Partners LLP            1
123RF LIMITED                1
1535创投                       1
1982 Ventures Pte.Ltd.       1
20/20 HealthCare Partners    1
                            ..
安吉兴平股权投资合伙企业（有限合伙）           3
广东德和信企业科技有限公司                3
简道众创投资股份有限公司                 3
宁波创达创业投资合伙企业（有限合伙）           3
贵州省科技风险投资有限公司                3
Name: Alias, Length: 41283, dtype: int64

In [17]:
Alias[Alias.VC_fullname=='贵州省科技风险投资有限公司']

,VC_fullname,Alias
40721,贵州省科技风险投资有限公司,贵州省科技风险投资公司
40722,贵州省科技风险投资有限公司,贵州风投
40723,贵州省科技风险投资有限公司,贵阳市科技风险投资有限公司


In [18]:
Alias[Alias.VC_fullname=='宁波创达创业投资合伙企业（有限合伙）']

,VC_fullname,Alias
20274,宁波创达创业投资合伙企业（有限合伙）,创达创业投资
20275,宁波创达创业投资合伙企业（有限合伙）,苏州创达创业投资企业(有限合伙)
20276,宁波创达创业投资合伙企业（有限合伙）,衢州创达投资合伙企业（有限合伙）


In [22]:
Alias[Alias.Alias=='创达创业投资']

,VC_fullname,Alias
20274,宁波创达创业投资合伙企业（有限合伙）,创达创业投资


In [95]:
map_fullname = Alias.groupby('VC_fullname')['Alias'].unique().map(lambda x:';'.join(i.strip() for i  in x))
map_fullname

VC_fullname
10T Holdings                                     10T
123RF LIMITED                                  123RF
1535创投                              杭州壹伍叁伍创业投资管理有限公司
1982 Ventures Pte.Ltd.                 1982 Ventures
20/20 HealthCare Partners           20/20 HealthCare
                                      ...           
龙腾荣盛投资管理有限公司                                  龙腾荣盛投资
龙腾资本有限公司                                        龙腾资本
龙芯中科技术股份有限公司                       龙芯中科技术;龙芯中科技术有限公司
龙赢富泽资产管理(北京)有限公司             龙赢富泽资产;龙赢富泽资产管理（北京）有限公司
龙金（北京）投资管理有限公司                            龙金（北京）投资管理
Name: Alias, Length: 41283, dtype: object

In [9]:
map_fullname.loc['宁波创达创业投资合伙企业（有限合伙）']

'创达创业投资;苏州创达创业投资企业(有限合伙);衢州创达投资合伙企业（有限合伙）'

In [96]:
ChinaVCtype['Alias']=ChinaVCtype['VC_fullname'].map(map_fullname)
ChinaVCtype['Alias'].count()

np.int64(40754)

In [14]:
ChinaVCtype.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49187 entries, 0 to 49186
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   VC_fullname                      49187 non-null  object 
 1   Unified_social_credit_ID         31871 non-null  object 
 2   GVCagency_ne                     43681 non-null  float64
 3   GVCagency_equity                 34731 non-null  float64
 4   GVCsoe_ne                        43681 non-null  float64
 5   GVCsoe_equity                    34731 non-null  float64
 6   IVC_ne                           36458 non-null  float64
 7   CVC_ne                           36458 non-null  float64
 8   BVC_ne                           36458 non-null  float64
 9   FVC_ne                           36458 non-null  float64
 10  UVC_ne                           36458 non-null  float64
 11  PenVC_ne                         36458 non-null  float64
 12  GVC_excl          

In [21]:
ChinaVCtype['Parent_listed'].unique(),ChinaVCtype['Corporate_parent'].nunique()

(array([nan, 'Listed', 'Delisted'], dtype=object), 3146)

In [119]:
ChinaVCtype.to_excel('data/ChinaVCtype.xlsx')

In [469]:
ChinaVCtype=pd.read_excel('data/ChinaVCtype.xlsx',index_col=0)
ChinaVCtype.columns

Index(['VC_fullname', 'Unified_social_credit_ID', 'GVCagency_ne',
       'GVCagency_equity', 'GVCsoe_ne', 'GVCsoe_equity', 'IVC_ne', 'CVC_ne',
       'BVC_ne', 'FVC_ne', 'UVC_ne', 'PenVC_ne', 'GVC_excl', 'GVC_equity',
       'CVC_excl', 'IVC_excl', 'BVC_excl', 'FVC_excl', 'UVC_excl',
       'PenVC_excl', 'Unknown', 'Direct_judge', 'Corporate_parent',
       'Parent_unified_social_credit_ID', 'Parent_industry_qcc',
       'Parent_listed', 'Parent_listedmarket', 'Parent_stockcode', 'Alias'],
      dtype='object')

In [2]:
invest_all=pd.read_excel('data/source/invest_all.xlsx',index_col=0)
invest_all.columns

Index(['企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话', '企业网址',
       '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址', '上市状态',
       '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1', '交易时间',
       '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式', '是否关联交易',
       '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型', '投资金额',
       '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称', '退出方基金名称',
       '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称', '中介类型',
       '投资方名称_A', '投资方基金名称_A', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业',
       '子行业'],
      dtype='object')

In [4]:
invest_all.shape

(155309, 62)

In [30]:
invest_all['投资方名称_A']=invest_all['投资方名称'].str.split(';').dropna().map(lambda x:[i.strip() for i in x])
invest_all['投资方名称_A']

0                          [上海领汇创投]
1         [祥峰投资中国基金, 火山石投资, 西藏高榕资本]
2                   [金顺东投资, 奥都百升网络]
3                            [真格基金]
4                            [时代伯乐]
                    ...            
155304                   [富华睿银基金管理]
155305                        [深创投]
155306                   [CVC Asia]
155307                       [四维图新]
155308                      [及时雨创投]
Name: 投资方名称_A, Length: 155309, dtype: object

In [106]:
map_alias_dict = Alias.groupby('Alias')['VC_fullname'].apply(lambda x:list(set(x))).to_dict()
alias_set = set(map_alias_dict.keys())

In [38]:
invest_all['ChinaVC_alias']=invest_all['投资方名称_A'].dropna().apply(lambda x:';'.join(set(x)&alias_set)).replace('',pd.NA)
invest_all['ChinaVC_alias'].str.split(';').explode().nunique(),invest_all['ChinaVC_alias'].isna().sum()

(36057, np.int64(36884))

In [75]:
invest_all['VC_fullname'] =invest_all['ChinaVC_alias'].dropna().map(lambda x: ';'.join({fullname for alias in x.split(';') for fullname in map_alias_dict[alias]}))
invest_all['VC_fullname'] 

0                          上海领汇创业投资有限公司
1                          西藏高榕资本管理有限公司
2          上海金顺东投资管理有限公司;北京奥都百升网络科技有限公司
3         北京真格天成投资管理有限公司;天津真格天地投资管理有限公司
4                     深圳市时代伯乐创业投资管理有限公司
                      ...              
155304                              NaN
155305                    深圳市创新投资集团有限公司
155306            CVC Asia Pacific,Ltd.
155307                   北京四维图新科技股份有限公司
155308                    深圳市汤乔电子科技有限公司
Name: VC_fullname, Length: 155309, dtype: object

In [49]:
vc2gvc = ChinaVCtype.set_index('VC_fullname')['GVC_excl'].to_dict()
invest_all['GVC'] = invest_all['VC_fullname'].dropna().str.split(';').map( lambda vc_list: [vc2gvc.get(vc, 0) for vc in vc_list])# if vc in vc2gvc
invest_all['GVC'] 

0            [0]
1            [0]
2         [0, 0]
3         [0, 0]
4            [0]
           ...  
155304       NaN
155305       [1]
155306       [0]
155307       [0]
155308       [0]
Name: GVC, Length: 155309, dtype: object

In [51]:
invest_all['GVC'].explode().sum()

28186

In [57]:
invest_all['GVC_DUMMY']=invest_all['GVC'].dropna().map(any).astype(int)#.map(lambda x: 1 if 1 in x else 0)
invest_all['GVC_DUMMY'].sum()

np.float64(22923.0)

In [62]:
import numpy as np
invest_all['GVC_Continuous']= invest_all['GVC'].dropna().map(np.mean)#.map(lambda x: sum(x) / len(x))
invest_all['GVC_Continuous'].mean()

np.float64(0.13929885744424478)

In [ ]:
invest_all['GVC_Continuous'].describe()

In [67]:
invest_all['GVC']=invest_all['GVC'].dropna().map(lambda x: ';'.join(str(i).strip() for i in x) )

In [69]:
invest_all.to_excel('data/invest_all_gvc.xlsx')

In [2]:
invest_all=pd.read_excel('data/invest_all_gvc.xlsx',index_col=0)
invest_all.shape

(155309, 67)

In [76]:
invest_all['VC_fullname']=invest_all['VC_fullname'].dropna().str.split(';')
invest_vc=invest_all.dropna(subset=['VC_fullname']).explode('VC_fullname')
invest_vc['VC_fullname'].isna().sum(),invest_vc.shape

(np.int64(0), (218426, 67))

In [88]:
(invest_vc['VC_fullname']=='').sum()

np.int64(0)

In [83]:
ChinaVCtype[['VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl']].duplicated().sum()

np.int64(0)

In [85]:
invest_vc=invest_vc.merge(ChinaVCtype[['VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl']],on='VC_fullname',how='left')
invest_vc.shape

(218426, 70)

In [90]:
invest_vc.to_csv('data/invest_all_vctype.csv')

In [164]:
import gc 
del patent_all
gc.collect()

2657

In [165]:
%%time
patent_all=pd.read_csv('data/source/patent_all_2408.csv')
patent_all.shape,patent_all.columns

<timed exec>:1: DtypeWarning: Columns (8,9,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 5min 14s
Wall time: 5min 21s


((13896974, 17),
 Index(['申请号', '申请日', '申请人', '公开（公告）号', '公开（公告）日', '首次公开日', '标题 (中文)', '专利类型',
        'IPC主分类', '引证申请人', '被引证申请人', '引证专利', '当前法律状态', '当前权利人', '工商统一社会信用代码',
        '工商上市代码', 'table_name'],
       dtype='object'))

In [9]:
%%time
patent_1=pd.read_csv('data/source/patent_data_2019_202410.csv',usecols=patent_all.columns[:-1])
patent_1.shape

CPU times: total: 23.1 s
Wall time: 23.6 s


<timed exec>:1: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.


(688260, 16)

In [10]:
patent_1['table_name']='patent202410'

In [11]:
import pickle
with open("data/patent_data_types_2024.pkl", "rb") as f:
    pdtypes=pickle.load(f)
pdtypes

{'序号': dtype('int64'),
 '标题 (中文)': dtype('O'),
 '标题 (英文)': dtype('O'),
 '摘要 (中文)': dtype('O'),
 '摘要 (英文)': dtype('O'),
 '申请人': dtype('O'),
 '公开（公告）号': dtype('O'),
 '公开（公告）日': dtype('<M8[ns]'),
 '申请号': dtype('O'),
 '申请日': dtype('<M8[ns]'),
 '公开类型': dtype('O'),
 '专利类型': dtype('O'),
 '公开国别': dtype('O'),
 '链接到incoPat': dtype('O'),
 '标题（小语种原文）': dtype('float64'),
 '摘要（小语种原文）': dtype('float64'),
 '首项权利要求': dtype('O'),
 '首权翻译': dtype('O'),
 '独立权利要求': dtype('O'),
 '权利要求数量': dtype('float64'),
 '独立权利要求数量': dtype('float64'),
 '从属权利要求数量': dtype('float64'),
 '文献页数': dtype('float64'),
 '首权字数': dtype('float64'),
 '技术功效句': dtype('O'),
 '技术功效短语': dtype('O'),
 '技术功效1级': dtype('O'),
 '技术功效2级': dtype('O'),
 '技术功效3级': dtype('O'),
 '技术功效TRIZ参数': dtype('O'),
 'IPC主分类': dtype('O'),
 'IPC': dtype('O'),
 '洛迦诺分类号': dtype('float64'),
 'EC': dtype('O'),
 'CPC': dtype('O'),
 'UC': dtype('float64'),
 'FI': dtype('float64'),
 'F-term': dtype('float64'),
 '国民经济分类': dtype('O'),
 '国民经济行业(主)': dtype('O'),
 '新兴产业分类': dtyp

In [ ]:
# data_types=patent_filter.dtypes[patent_filter.dtypes!='datetime64[ns]'].to_dict()
# parse_dates=patent_filter.dtypes[patent_filter.dtypes=='datetime64[ns]'].index.to_list()
# parse_dates

In [12]:
patent_2=pd.read_csv('data/source/patent_data_202411.tsv',sep='\t',usecols=patent_all.columns[:-1])#chunksize=100000,,dtype=data_types,parse_dates=parse_dates
patent_2.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_15528\2741211158.py:1: DtypeWarning: Columns (70) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_2=pd.read_csv('data/source/patent_data_202411.tsv',sep='\t',usecols=patent_all.columns[:-1])#chunksize=100000,,dtype=data_types,parse_dates=parse_dates


(7187608, 16)

In [114]:
patent_all.columns[:-2]

Index(['申请号', '申请日', '申请人', '公开（公告）号', '公开（公告）日', '首次公开日', '标题 (中文)', '专利类型',
       'IPC主分类', '引证申请人', '被引证申请人', '引证专利', '当前法律状态', '当前权利人', '工商统一社会信用代码',
       '工商上市代码', 'table_name', '企业全称'],
      dtype='object')

In [13]:
patent_2['table_name']='patent_incopat_2016b_202411'

In [116]:
import os,glob
from pathlib import Path
path=u'E:/Downloads/BaiduNetdiskDownload'
os.chdir(path)
file_path=os.listdir(path)
file_path

['2017', '2018', '2019', '2020', 'sd_gxkj_deflator_1114.csv']

In [117]:
matching_files=[]
for fi in file_path:
    matching_files+=glob.glob(os.path.join(fi,f"*.xlsx"))
len(matching_files)

107

In [120]:
data_columns=pd.read_excel(matching_files[-1],nrows=10).columns.tolist()
matching_files[-1],len(data_columns)

D:\Program Files\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


('2020\\2024-11-20.xlsx', 213)

In [129]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [126]:
filter_df.columns,columns+['企业全称','path']

(Index(['序号', '标题 (中文)', '摘要 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日',
        '专利类型', '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量',
        '简单同族', '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人',
        '被引证申请人', '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人',
        '工商统一社会信用代码', 'table_name', '企业全称'],
       dtype='object'),
 ['序号',
  '标题 (中文)',
  '摘要 (中文)',
  '申请人',
  '申请日',
  '申请号',
  '公开（公告）号',
  '公开（公告）日',
  '专利类型',
  '专利有效性',
  '公开国别',
  'IPC主分类',
  'IPC',
  '申请人类型',
  '发明人',
  '发明人数量',
  '简单同族',
  '扩展同族',
  '简单同族个数',
  '引证专利',
  '引证科技文献',
  '引证次数',
  '被引证次数',
  '引证申请人',
  '被引证申请人',
  '申请人数量',
  '许可次数',
  '转让次数',
  '权利要求数量',
  '当前法律状态',
  '当前权利人',
  '工商统一社会信用代码',
  '企业全称',
  'path'])

In [127]:
filter_df_invest=filter_df.copy()

In [141]:
%%time
sizes=0
patent_filter = pd.DataFrame()
# patent_df = pd.concat([pd.read_excel(file) for file in matching_files], ignore_index=True)
for i,f_patent in enumerate(matching_files):
    df_chunk=pd.read_excel(f_patent)#,dtype=data_types, error_bad_lines=True
    
    print(i,sizes,f_patent,df_chunk.shape,df_chunk['序号'].max())#,patent_df.shape
    
    # patent_df = pd.concat([patent_df,df_chunk], ignore_index=True)
    df_chunk['申请日']=pd.to_datetime(df_chunk['申请日'])
    df_chunk['公开（公告）日']=pd.to_datetime(df_chunk['公开（公告）日'])
    df_chunk.sort_values(['申请日','公开（公告）日'],inplace=True)
    # df_chunk.drop_duplicates(['申请号','公开（公告）号','专利类型'],keep='last',inplace=True)
    df_chunk.reset_index(drop=True,inplace=True)
    
    df_chunk['序号'] = df_chunk.index+sizes+1

    # try:
    #     df_chunk.to_sql('patent_incopat_2016b_202411', con=engine,chunksize=10000, if_exists='append', index=False)#,dtype=dtypes
    # except Exception as e:
    #     print(f"Error writing to SQL: {e}")

    sizes+=df_chunk.shape[0]
    
    #applicants=df_chunk['申请人'].str.split(';').explode().str.strip()
    #holders=df_chunk['当前权利人'].str.split(';').explode().str.strip()
    #filter_applicants=applicants[applicants.isin(gx_companies)]
    df_chunk['path'] = f_patent
    df_chunk['企业全称']=df_chunk['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_co] if isinstance(x, list) else [])
    filter_df=df_chunk[df_chunk['企业全称'].map(len)>0]
    if not filter_df.empty:#filter_applicants.empty
        #filter_df=df_chunk.iloc[filter_applicants.index.unique()]
        #filter_df=df_chunk.loc[filter_applicants.index]
        #filter_df['Co']=filter_applicants
        patent_filter = pd.concat([patent_filter,filter_df[columns+['企业全称','path']]], ignore_index=True)
        print('filter:',filter_df['企业全称'].explode().nunique(),filter_df.shape[0],patent_filter.shape[0],patent_filter.序号.max())#filter_applicants.nunique(),
            
        del filter_df

    # if i<=44:
    #     continue
    with open('E:/Documents/Jupyter/data/source/patent_data_202411_2.tsv','a',encoding="utf_8_sig") as f: 
        # df_chunk.rename(
        #      columns= {'提出实审时长':'进入实质审查时长',
        #     '申请人国别代码':'申请人国家/地区',
        #     '发明人国别':'发明人国家/地区',
        #     '发明人地址':'发明人地址（原始）',}, inplace=True)
        df_chunk.reindex(columns=data_columns, fill_value=None).to_csv(f,sep='\t', mode="a", index=False,header=f.tell()==0)#使用空值填充缺失的数据columns=data_columns,na_rep='NaN',转csv合并
        
    del df_chunk


0 0 2017\2024-10-31 (1).xlsx (60000, 212) 120000
filter: 1250 4916 4916 59976
1 60000 2017\2024-10-31 (2).xlsx (60000, 212) 180000
filter: 1203 4268 9184 119960
2 120000 2017\2024-10-31 (3).xlsx (60000, 212) 240000
filter: 1162 4704 13888 179958
3 180000 2017\2024-10-31 (4).xlsx (59758, 212) 299758
filter: 1308 5556 19444 239756
4 239758 2017\2024-10-31.xlsx (60000, 212) 60000
filter: 1258 5088 24532 299562
5 299758 2017\2024-11-01 (1).xlsx (60000, 213) 120000
filter: 1160 5392 29924 359731
6 359758 2017\2024-11-01 (2).xlsx (60000, 213) 180000
filter: 1127 4578 34502 419714
7 419758 2017\2024-11-01 (3).xlsx (60000, 213) 240000
filter: 1130 4433 38935 479740
8 479758 2017\2024-11-01 (4).xlsx (56958, 213) 296958
filter: 1200 4581 43516 536713
9 536716 2017\2024-11-01.xlsx (60000, 213) 60000
filter: 1240 4275 47791 596693
10 596716 2017\2024-11-02 (1).xlsx (60000, 213) 120000
filter: 1232 4350 52141 656673
11 656716 2017\2024-11-02 (2).xlsx (60000, 213) 180000
filter: 1133 5112 57253 7167

In [142]:
patent_filter['申请日'].max()

Timestamp('2020-12-31 00:00:00')

In [143]:
os.chdir(u'E:\\Documents\\Jupyter')

In [162]:
['标题 (中文)','申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日',
            '专利类型',  'IPC主分类', '引证专利','引证申请人','被引证申请人', 
            '当前法律状态', '当前权利人', '工商统一社会信用代码', '工商上市代码','table_name']
patent_2.columns

Index(['标题 (中文)', '申请人', '公开（公告）号', '公开（公告）日', '申请号', '申请日', '专利类型', 'IPC主分类',
       '当前权利人', '工商统一社会信用代码', '工商上市代码', '引证专利', '引证申请人', '被引证申请人', '当前法律状态',
       '首次公开日', 'table_name'],
      dtype='object')

In [163]:
patent_3=pd.read_csv('data/source/patent_data_202411_2.tsv',sep='\t',usecols=patent_2.columns[:-1])
patent_3['table_name']='patent_incopat_2020b_202411'
patent_3.shape

(6046257, 17)

In [178]:
patent_all=pd.concat([patent_all.reindex(columns=patent_3.columns),patent_1,patent_2,patent_3])
patent_all['公开（公告）日']=pd.to_datetime(patent_all['公开（公告）日'])
patent_all['申请日']=pd.to_datetime(patent_all['申请日'])
patent_all.shape,patent_all.申请号.nunique()

((27819099, 17), 21247889)

In [179]:
patent_all.table_name.value_counts()

table_name
patent_incopat_202101_202211    8920983
patent_incopat_2016b_202411     7187608
patent_incopat_2020b_202411     6046257
patent202404                    2260240
patent202210                    1772752
patent202410                     688260
patent_incopat_202212_202312     586206
patent202407                     146978
patent202309                     113089
patent202408                      96726
Name: count, dtype: int64

In [180]:
%%time
patent_all.to_csv('data/source/patent_all_2411.csv',index=False)

CPU times: total: 4min 37s
Wall time: 4min 51s


In [181]:
del patent_1,patent_2,patent_3
gc.collect()

557

In [182]:
patent_all=patent_all.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号','专利类型'],keep='first').reset_index(drop=True)
patent_all.shape,patent_all.申请号.nunique()#(16790458, 17),15870285->21247889

((22255063, 17), 21247889)

In [183]:
22255063/27819099

0.7999922283608106

In [16]:
patent_all.to_csv('data/source/patent_all_2024_first.tsv',index=False,sep='\t')
patent_all.专利类型.isna().sum()

np.int64(0)

In [2]:
patent_all=pd.read_csv('data/source/patent_all_2024_first.tsv',sep='\t')
patent_all.shape

(22255063, 17)

In [5]:
patent_all.shape

(22255063, 17)

In [6]:
patent_all.申请日=pd.to_datetime(patent_all.申请日)
patent_all['申请年']= patent_all.申请日.dt.year

In [7]:
patent_all.专利类型.value_counts()#10367641

专利类型
发明申请    15832246
实用新型     4138869
外观设计     1288912
发明授权      995036
Name: count, dtype: int64

In [186]:
patent_all.table_name.value_counts()#6688311

table_name
patent_incopat_2016b_202411     6688311
patent_incopat_202101_202211    5586621
patent_incopat_2020b_202411     5464605
patent202210                    1576086
patent202404                    1492032
patent202410                     688260
patent_incopat_202212_202312     586203
patent202309                      93815
patent202408                      46033
patent202407                      33097
Name: count, dtype: int64

In [11]:
patent_all.groupby(['申请年','专利类型'],as_index=False).申请号.nunique().tail(10)#.sort_values(by='申请号')

,申请年,专利类型,申请号
186,2022.0,外观设计,149643
187,2022.0,实用新型,645116
188,2023.0,发明授权,41050
189,2023.0,发明申请,435220
190,2023.0,外观设计,51816
191,2023.0,实用新型,69510
192,2024.0,发明授权,462
193,2024.0,发明申请,6816
194,2024.0,外观设计,77
195,2024.0,实用新型,101


In [8]:
patent_all[(patent_all['申请年']==2023) & patent_all['专利类型'].isin(['发明授权','发明申请'])].groupby(['table_name','专利类型'],as_index=False).申请号.nunique()

,table_name,专利类型,申请号
0,patent202309,发明授权,190
1,patent202309,发明申请,3819
2,patent202404,发明授权,11204
3,patent202404,发明申请,75000
4,patent202407,发明授权,425
5,patent202407,发明申请,8495
6,patent202408,发明授权,147
7,patent202408,发明申请,2269
8,patent_incopat_202212_202312,发明授权,29084
9,patent_incopat_202212_202312,发明申请,345637


In [32]:
a=set(patent_all.loc[(patent_all['申请年']==2023) & patent_all['专利类型'].isin(['发明授权','发明申请']) & (patent_all.table_name=='patent_incopat_202212_202312'),'申请号'])
b=set(patent_all.loc[(patent_all['申请年']==2023) & patent_all['专利类型'].isin(['发明授权','发明申请']) & (patent_all.table_name!='patent_incopat_202212_202312'),'申请号'])
len(a-b)

339693

In [33]:
len(b-a)

89811

In [10]:
tb_id=patent_all[(patent_all['申请年']==2023) & patent_all['专利类型'].isin(['发明授权','发明申请'])].groupby('table_name')['申请号'].agg(list)
tb_id

table_name
patent202309                    [CN202310001667.2, CN202310003349.X, CN2023100...
patent202404                    [US18149111, CN202310000006.8, CN202310000028....
patent202407                    [CN202310003224.7, CN202310007791.X, CN2023100...
patent202408                    [CN202310003252.9, CN202310003505.2, CN2023100...
patent_incopat_202212_202312    [IN202311000006, IN202311000012, IN20231100002...
Name: 申请号, dtype: object

In [14]:
import pickle
with open("data/source/patent_data_types_2024.pkl", "rb") as f:
    data_dtypes=pickle.load( f)

In [26]:
data_columns=list(data_dtypes.keys())

In [27]:
conn.rollback()

In [30]:
%%time
accumulated_rows = 0
chunk_size = 10000  # 每次处理的行数
batch_size = 10000  # 批处理大小
output_csv = 'data/patent_invention_2023.csv'  # 保存CSV的路径

for table in tb_id.index:
    query = text(f'SELECT * FROM `{table}` WHERE `申请号` in :ids')
    chunks_list = []
    ids = tuple(tb_id[table])
    
    for batch in chunks(ids, batch_size):
        filtered_chunk = pd.read_sql_query(query, conn, params={'ids': batch})
        # filtered_chunk = filtered_chunk.sort_values('公开（公告）日').drop_duplicates(subset='公开（公告）号', keep='first')

        print(f'{len(chunks_list)}:{filtered_chunk.shape[0]}.', end='', flush=True)
        if not filtered_chunk.empty and not filtered_chunk.isna().all().all():
            chunks_list.append(filtered_chunk)
    
    df_filter = pd.concat(chunks_list, ignore_index=True)
    df_filter['table_name'] = table
    df_filter.reindex(columns=data_columns, fill_value=None).to_csv(output_csv, mode='a', header=not pd.io.common.file_exists(output_csv), index=False)
    
    print(f"\nFinished processing table: {table} from {accumulated_rows} with size: {df_filter.shape[0]}")
    accumulated_rows += df_filter.shape[0]
    
    del df_filter
    del chunks_list

0:4118.
Finished processing table: patent202309 from 0 with size: 4118
0:10819.1:10332.2:10837.3:10780.4:10330.5:11207.6:10304.7:10814.8:6867.

<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent202404 from 4118 with size: 92290
0:9676.
Finished processing table: patent202407 from 96408 with size: 9676
0:2511.
Finished processing table: patent202408 from 106084 with size: 2511
0:10014.1:10019.2:10007.3:10030.4:10034.5:10053.6:10051.7:10088.8:10193.9:10145.10:10086.11:10225.12:10284.13:10125.14:10018.15:10298.16:10601.17:10582.18:10676.19:10686.20:10735.21:10800.22:10762.23:10642.24:10667.25:10722.26:10337.27:10299.28:10273.29:10100.30:10245.31:10489.32:10352.33:10077.34:10207.35:10205.36:10062.37:4744.

<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent_incopat_202212_202312 from 108595 with size: 385933
CPU times: total: 1min 51s
Wall time: 9min 6s


In [255]:
invest_companies=set(invest_all.企业全称.str.strip().replace('',pd.NA).dropna().unique())
len(invest_companies)

83793

In [256]:
patent_all['invest_申请人']=patent_all['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_companies] if isinstance(x, list) else [])
(patent_all['invest_申请人'].map(len)>0).sum()#2608946

np.int64(2608946)

In [247]:
patent_all[patent_all['invest_申请人'].map(len)>0].groupby(['table_name','专利类型'],as_index=False).申请号.nunique()

,table_name,专利类型,申请号
0,patent202210,发明授权,214685
1,patent202210,发明申请,239427
2,patent202210,外观设计,87893
3,patent202210,实用新型,284373
4,patent202309,发明授权,7306
5,patent202309,发明申请,21356
6,patent202309,外观设计,1775
7,patent202309,实用新型,817
8,patent202404,发明授权,114829
9,patent202404,发明申请,308024


In [271]:
tb_id=patent_all[(patent_all['invest_申请人'].map(len)>0)&(~patent_all['table_name'].isin(['patent_incopat_2016b_202411','patent_incopat_2020b_202411']))].groupby('table_name')['申请号'].agg(list)
tb_id

table_name
patent202210                    [CN93237739.4, CN94226705.2, CN94226882.2, CN9...
patent202309                    [CN03326189.X, CN03326190.3, CN03326191.1, CN0...
patent202404                    [CN96201248.3, CN96203216.6, CN96203267.0, CN9...
patent202407                    [CN00334820.2, CN00334822.9, CN00334821.0, CN0...
patent202408                    [CN94239416.X, CN00123987.2, CN01239018.6, CN0...
patent202410                    [CN96229998.7, CN96308886.6, CN97244947.7, CN9...
patent_incopat_202101_202211    [CN202110000174.8, CN202110000181.8, CN2021200...
patent_incopat_202212_202312    [CN202211526644.5, CN202211523135.7, CN2022115...
Name: 申请号, dtype: object

In [266]:
data_columns_1=data_columns+['table_name','invest_申请人']
len(data_columns_1)

216

In [277]:
conn.rollback()

In [278]:
%%time
accumulated_rows = 0
batch_size = 10000  # 批处理大小
output_csv = 'data/patent_all_invest_2024.tsv'  # 保存CSV的路径

for table in tb_id.index:
    query = text(f'SELECT * FROM `{table}` WHERE `申请号` in :ids')
    chunks_list = []
    ids = tuple(tb_id[table])
    
    for batch_ids in chunks(ids, batch_size):
        filtered_chunk = pd.read_sql_query(query, conn, params={'ids': batch_ids})
        # filtered_chunk = filtered_chunk.sort_values('公开（公告）日').drop_duplicates(subset='公开（公告）号', keep='first')

        print(f'{len(chunks_list)}:{filtered_chunk.shape[0]}.', end='', flush=True)
        if not filtered_chunk.empty:
            chunks_list.append(filtered_chunk)
    
    if not chunks_list:
        continue
        
    df_filter = pd.concat(chunks_list, ignore_index=True)
    df_filter['table_name'] = table
    df_filter['invest_申请人']=df_filter['申请人'].dropna().apply(lambda x: ';'.join([i.strip() for i in x.split(';') if i.strip() in invest_companies]))
    df_filter.reindex(columns=data_columns_1, fill_value=None).to_csv(output_csv, mode='a',sep='\t',header=not pd.io.common.file_exists(output_csv), index=False)

    print(f"\nFinished processing table: {table} from {accumulated_rows} with size: {df_filter.shape}")
    accumulated_rows += df_filter.shape[0]
    
    del df_filter
    del chunks_list
    
accumulated_rows

0:10003.1:10000.2:10000.3:10000.4:10000.5:10000.6:10000.7:10000.8:10001.9:10002.10:10000.11:10000.12:10000.13:10000.14:10000.15:10000.16:10000.17:10000.18:10000.19:10000.20:10000.21:10000.22:10000.23:10000.24:10000.25:10000.26:10000.27:10000.28:10000.29:10001.30:10000.31:10001.32:10000.33:10000.34:10000.35:10000.36:10000.37:10000.38:10000.39:10000.40:10000.41:10001.42:10000.43:10000.44:10001.45:10000.46:10002.47:10000.48:10000.49:10001.50:10000.51:10000.52:10000.53:10000.54:10000.55:10002.56:10003.57:10001.58:10001.59:10000.60:10000.61:10000.62:10000.63:10000.64:10000.65:10000.66:10000.67:10000.68:10000.69:10000.70:10000.71:10000.72:10000.73:10000.74:10000.75:10000.76:10000.77:10000.78:10000.79:10000.80:10000.81:10000.82:6378.

<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent202210 from 0 with size: (826398, 190)
0:10554.1:10564.2:10898.3:1298.

<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent202309 from 826398 with size: (33314, 190)
0:15843.1:15334.2:14284.3:13276.4:12911.5:12530.6:12675.7:12458.8:12338.9:12091.10:11452.11:11519.12:11633.13:11429.14:11267.15:11449.16:11492.17:11326.18:11375.19:11485.20:11296.21:11266.22:11470.23:11287.24:11243.25:11426.26:11432.27:11246.28:11155.29:11183.30:11066.31:11141.32:11537.33:11362.34:11057.35:11100.36:11169.37:11177.38:11240.39:12010.40:11379.41:11700.42:14966.43:13548.44:12949.45:12514.46:10956.47:10818.48:10687.49:10425.50:10342.51:10216.52:10197.53:10127.54:10326.55:10299.56:10515.57:10143.58:10411.59:10624.60:10115.61:10450.62:10837.63:10239.64:10022.

...


Finished processing table: patent202410 from 1649579 with size: (56029, 196)
0:16991.1:17162.2:17079.3:16490.4:16578.5:16384.6:16258.7:16299.8:16336.9:16197.10:16460.11:16257.12:16399.13:16496.14:16633.15:16663.16:16508.17:16444.18:16417.19:16604.20:16410.21:16539.22:16471.23:16425.24:16194.25:16090.26:15929.27:15876.28:15800.29:15776.30:15724.31:15574.32:15594.33:15063.34:12786.35:12696.36:13059.37:13276.38:13966.39:14528.40:14577.41:14525.42:10123.

<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent_incopat_202101_202211 from 1705608 with size: (673656, 173)
0:10035.1:10044.2:10028.3:10061.4:10074.5:10089.6:8072.

<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent_incopat_202212_202312 from 2379264 with size: (68403, 173)
CPU times: total: 11min 59s
Wall time: 34min 46s


In [279]:
expected_columns=data_columns+['invest_申请人']

In [291]:
import gc
gc.collect()

4386

In [292]:
parse_dates=[]
dtype_spec={}
for k,v in data_dtypes.items():
    if v=='datetime64[ns]':
        parse_dates.append(k)
    else:
        dtype_spec[k]=v
parse_dates

['公开（公告）日',
 '申请日',
 '优先权日',
 '最早优先权日',
 '首次公开日',
 '授权公告日',
 '实质审查生效日',
 '失效日',
 '预估到期日']

In [294]:
%%time
# patent_filter=pd.DataFrame()
chunk_size = 100000 # 每次读取的行数
batch_size = 100000  # 批处理大小
output_csv = 'data/patent_all_invention_invest_2020b.tsv' 
accumulated_rows = 0
batch_count = 0
filtered_chunks = []

for i,df_chunk in enumerate(pd.read_csv('data/source/patent_data_202411.tsv',chunksize=chunk_size,sep='\t', dtype=dtype_spec,parse_dates=parse_dates)):#,low_memory=False
    df_chunk['invest_申请人']=df_chunk['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_companies] if isinstance(x, list) else [])
    mask= (df_chunk['invest_申请人'].map(len)>0)
    if mask.sum()>0:
        filtered_chunks.append(df_chunk[mask]) 
        accumulated_rows+=mask.sum()
        print(f'{i}:{int(mask.sum())},', end='', flush=True)

    if accumulated_rows // batch_size > batch_count:
        patent_filter = pd.concat(filtered_chunks, ignore_index=True)
        print('\n',batch_count,accumulated_rows,patent_filter.申请号.nunique(),patent_filter.shape)
        patent_filter['invest_申请人']= patent_filter['invest_申请人'].dropna().map(';'.join)
        # patent_filter['table_name']='patent_incopat_2016b_202411'
        # patent_filter.reindex(columns=expected_columns, fill_value=None).to_csv(output_csv, mode='a',sep='\t',header=not pd.io.common.file_exists(output_csv), index=False)#,header=f.tell()==0
        with open(output_csv,'a',encoding="utf_8_sig") as f: 
            patent_filter.reindex(columns=expected_columns, fill_value=None).to_csv(f, sep='\t', index=False, header=f.tell() == 0)
            
        batch_count = accumulated_rows // batch_size 
        filtered_chunks.clear()
        del patent_filter
        gc.collect()

    del df_chunk


print('\n',i,batch_count,accumulated_rows)

for i,df_chunk in enumerate(pd.read_csv('data/source/patent_data_202411_2.tsv',chunksize=chunk_size,sep='\t', dtype=dtype_spec,parse_dates=parse_dates)):
    df_chunk['invest_申请人']=df_chunk['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_companies] if isinstance(x, list) else [])
    mask= (df_chunk['invest_申请人'].map(len)>0)
    if mask.sum()>0:
        filtered_chunks.append(df_chunk[mask]) 
        accumulated_rows+=mask.sum()
        print(f'{i}:{int(mask.sum())},', end='', flush=True)

    if accumulated_rows // batch_size > batch_count:
        patent_filter = pd.concat(filtered_chunks, ignore_index=True)
        print('\n',batch_count,accumulated_rows,patent_filter.申请号.nunique(),patent_filter.shape)
        patent_filter['invest_申请人']= patent_filter['invest_申请人'].dropna().map(';'.join)
        # patent_filter['table_name']='patent_incopat_2020b_202411'
        with open(output_csv,'a',encoding="utf_8_sig") as f: 
            patent_filter.reindex(columns=expected_columns, fill_value=None).to_csv(f, sep='\t', index=False, header=f.tell() == 0)
            
        batch_count =  accumulated_rows // batch_size 
        filtered_chunks.clear()
        del patent_filter
        gc.collect()

    del df_chunk


if filtered_chunks:
    patent_filter = pd.concat(filtered_chunks, ignore_index=True)
    print('\n',batch_count,accumulated_rows,patent_filter.申请号.nunique(),patent_filter.shape)
    patent_filter['invest_申请人']= patent_filter['invest_申请人'].dropna().map(';'.join)
    # patent_filter['table_name']='patent_incopat_2020b_202411'
    with open(output_csv , 'a', encoding="utf_8_sig") as f:
        patent_filter.reindex(columns=expected_columns, fill_value=None).to_csv(f, sep='\t', index=False, header=f.tell() == 0)
    # del filtered_chunks
    # gc.collect()

0:45,1:16,2:27,3:647,4:947,5:1395,6:1824,7:1494,8:1793,9:2622,10:2372,11:2922,12:4701,13:5286,14:3821,15:5002,16:4890,17:5948,18:4917,19:5765,20:5022,21:5709,22:5576,23:5902,24:5531,25:5784,26:5706,27:6940,
 0 102604 102604 (102604, 213)
28:5919,29:5929,30:5837,31:6187,32:6758,33:5989,34:5577,35:4931,36:5346,37:5342,38:5211,39:5673,40:6458,41:6606,42:6071,43:6070,44:5894,
 1 202402 99798 (99798, 213)
45:5756,46:6484,47:6714,48:6941,49:7983,50:6589,51:7061,52:7073,53:7899,54:8235,55:7689,56:8674,57:7982,58:7561,
 2 305043 102641 (102641, 213)
59:9252,60:7639,61:7569,62:7725,63:6972,64:7499,65:7750,66:10009,67:8466,68:9066,69:8215,70:7664,
 3 402869 97826 (97826, 213)
71:8113,
 71 4 410982
0:7728,1:8216,2:8607,

<timed exec>:36: DtypeWarning: Columns (188) have mixed types. Specify dtype option on import or set low_memory=False.


3:8463,

<timed exec>:36: DtypeWarning: Columns (188) have mixed types. Specify dtype option on import or set low_memory=False.


4:7608,

...

58:12120,

<timed exec>:36: DtypeWarning: Columns (188) have mixed types. Specify dtype option on import or set low_memory=False.


59:11365,

<timed exec>:36: DtypeWarning: Columns (188) have mixed types. Specify dtype option on import or set low_memory=False.


60:7473,
 9 995160 93854 (93854, 214)
CPU times: total: 1h 1min 30s
Wall time: 1h 2min 39s


In [187]:
patent_all.groupby(['table_name','专利类型'],as_index=False).申请号.nunique().sort_values(by='申请号')

,table_name,专利类型,申请号
15,patent202407,实用新型,639
7,patent202309,实用新型,1236
14,patent202407,外观设计,2148
6,patent202309,外观设计,5933
16,patent202408,发明授权,6351
18,patent202408,外观设计,7138
12,patent202407,发明授权,10744
19,patent202408,实用新型,12262
13,patent202407,发明申请,19566
17,patent202408,发明申请,20058


In [188]:
patent_all.groupby('申请号').申请人.nunique().sort_values()

申请号
CN00114774.9    0
CN00114830.3    0
CN00114789.7    0
CN00114788.9    0
CN00114787.0    0
               ..
EP13852257      3
EP13871817      3
EP10788933      3
EP13852747      3
EP14803859      3
Name: 申请人, Length: 21247889, dtype: int64

In [189]:
patent_all[patent_all.申请号=='EP14803859']

,标题 (中文),申请人,公开（公告）号,公开（公告）日,申请号,申请日,专利类型,IPC主分类,当前权利人,工商统一社会信用代码,工商上市代码,引证专利,引证申请人,被引证申请人,当前法律状态,首次公开日,table_name
5261659,电动手柄用于植入物输送和输送系统,Shanghai Microport Medical (Group) Co Ltd,EP3005983A1,2016-04-13,EP14803859,2014-05-26,发明申请,A61F2/966,"Shanghai Microport Cardioflow Medtech Co., Ltd.",NaN,NaN,NaN,NaN,Boston Scientific Scimed Inc,全部撤销,2016-04-13,patent202404
5261718,电动手柄用于植入物输送和输送系统,Shanghai Microport Medical Group Co Ltd,EP3005983A4,2016-06-29,EP14803859,2014-05-26,发明申请,A61F2/966,"Shanghai Microport Cardioflow Medtech Co., Ltd.",NaN,NaN,US20090099638A1; WO2012116368A2; EP2363099A1; ...,David D Grewe; Koven Technology Canada Inc; Be...,NaN,全部撤销,2016-04-13,patent202404
5261876,用于植入物输送和输送系统的电动手柄,Shanghai Microport Cardioflow Medtech Co Ltd,EP3005983B1,2019-02-06,EP14803859,2014-05-26,发明授权,A61F2/966,"Shanghai Microport Cardioflow Medtech Co., Ltd.",NaN,NaN,US20090099638A1; CN101612436A; US6764494B2; US...,University Hospital Centre Freiburg; Albert Lu...,Neovasc Tiara Inc; Edwards Lifesciences Cardia...,全部撤销,2016-04-13,patent202404


In [43]:
invest_co=set(invest_all['企业全称'].str.strip().unique())
len(invest_co)

83793

In [44]:
%%time
patent_all['企业全称']=patent_all['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_co] if isinstance(x, list) else [])
filter_df=patent_all[patent_all['企业全称'].map(len)>0]
filter_df.shape#2317208, 18

CPU times: total: 20.1 s
Wall time: 20.3 s


(2317208, 18)

In [33]:
filter_df['企业全称'].explode().nunique(),filter_df.申请号.nunique()

(27473, 2015626)

In [37]:
filter_df.table_name.value_counts()

table_name
patent202210                    826378
patent202404                    649220
patent_incopat_202101_202211    427013
patent_incopat_2016b_202411     229760
patent_incopat_202212_202312     68021
patent202410                     55936
patent202309                     31256
patent202407                     22985
patent202408                      6639
Name: count, dtype: int64

In [39]:
filter_df.to_csv('data/patent_co_invest_bridge_202411.csv')

In [42]:
filter_invest=invest_all[invest_all['企业全称'].isin(set(filter_df['企业全称'].explode()))]
filter_invest.shape,invest_all.shape

((61350, 67), (155309, 67))

In [269]:
from sqlalchemy import create_engine,text
from urllib.parse import quote_plus
SQLALCHEMY_DATABASE_URI = f'mysql+pymysql://technet:{quote_plus("***")}@***.aliyuncs.com:3306/technet?charset=utf8mb4'
engine = create_engine(SQLALCHEMY_DATABASE_URI)  
conn = engine.connect()
conn.execute(text("describe patent_all_2408;")).fetchall()

[('申请号', 'varchar(32)', 'YES', 'MUL', None, ''),
 ('申请日', 'datetime', 'YES', '', None, ''),
 ('申请人', 'mediumtext', 'YES', '', None, ''),
 ('公开（公告）号', 'text', 'YES', '', None, ''),
 ('公开（公告）日', 'datetime', 'YES', '', None, ''),
 ('首次公开日', 'datetime', 'YES', '', None, ''),
 ('标题 (中文)', 'mediumtext', 'YES', '', None, ''),
 ('专利类型', 'text', 'YES', '', None, ''),
 ('IPC主分类', 'text', 'YES', '', None, ''),
 ('引证专利', 'mediumtext', 'YES', '', None, ''),
 ('当前法律状态', 'text', 'YES', '', None, ''),
 ('当前权利人', 'mediumtext', 'YES', '', None, ''),
 ('工商统一社会信用代码', 'mediumtext', 'YES', '', None, ''),
 ('工商上市代码', 'mediumtext', 'YES', '', None, ''),
 ('table_name', 'varchar(28)', 'NO', '', '', '')]

In [272]:
del chunks

In [273]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [65]:
columns=['序号', '标题 (中文)', '摘要 (中文)','申请人','申请日','申请号','公开（公告）号','公开（公告）日',
         '专利类型', '专利有效性',
         '公开国别','IPC主分类','IPC','申请人类型','发明人','发明人数量',
         '简单同族','扩展同族','简单同族个数',
         '引证专利','引证科技文献','引证次数','被引证次数','引证申请人','被引证申请人',
         '申请人数量','许可次数','转让次数','权利要求数量','当前法律状态','当前权利人', '工商统一社会信用代码']
column_name = '`'+'`,`'.join(columns)+'`'

In [60]:
tb_id=filter_df.groupby('table_name')['申请号'].agg(list)

In [67]:
%%time
accumulated_rows = 0
chunk_size = 10000  # 每次处理的行数
batch_size = 10000  # 批处理大小
output_csv = 'data/patent_invest_all_2411.csv'  # 保存CSV的路径

for table in tb_id.index:
    if table=='patent_incopat_2016b_202411':
        continue
    query = text(f'SELECT {column_name} FROM `{table}` WHERE `申请号` in :ids')
    chunks_list = []
    ids = tuple(tb_id[table])
    
    for batch in chunks(ids, batch_size):
        filtered_chunk = pd.read_sql_query(query, conn, params={'ids': batch})
        # filtered_chunk = filtered_chunk.sort_values('公开（公告）日').drop_duplicates(subset='公开（公告）号', keep='first')

        print(f'{len(chunks_list)}:{filtered_chunk.shape[0]}.', end='', flush=True)
        if not filtered_chunk.empty and not filtered_chunk.isna().all().all():
            chunks_list.append(filtered_chunk)
    
    df_filter = pd.concat(chunks_list, ignore_index=True)
    df_filter['table_name'] = table
    df_filter.to_csv(output_csv, mode='a', header=not pd.io.common.file_exists(output_csv), index=False)
    
    print(f"\nFinished processing table: {table} from {accumulated_rows} with size: {df_filter.shape[0]}")
    accumulated_rows += df_filter.shape[0]
    
    del df_filter
    del chunks_list

0:10003.1:10000.2:10000.3:10000.4:10000.5:10000.6:10000.7:10000.8:10001.9:10002.10:10000.11:10000.12:10000.13:10000.14:10000.15:10000.16:10000.17:10000.18:10000.19:10000.20:10000.21:10000.22:10000.23:10000.24:10000.25:10000.26:10000.27:10000.28:10000.29:10001.30:10000.31:10001.32:10000.33:10000.34:10000.35:10000.36:10000.37:10000.38:10000.39:10000.40:10000.41:10001.42:10000.43:10000.44:10001.45:10000.46:10002.47:10000.48:10000.49:10001.50:10000.51:10000.52:10000.53:10000.54:10000.55:10002.56:10003.57:10001.58:10001.59:10000.60:10000.61:10000.62:10000.63:10000.64:10000.65:10000.66:10000.67:10000.68:10000.69:10000.70:10000.71:10000.72:10000.73:10000.74:10000.75:10000.76:10000.77:10000.78:10000.79:10000.80:10000.81:10000.82:6378.

<timed exec>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent202210 from 0 with size: 826398
0:10554.1:10564.2:10898.3:1298.

<timed exec>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent202309 from 826398 with size: 33314
0:15843.1:15334.2:14284.3:13276.4:12911.5:12530.6:12675.7:12458.8:12338.9:12091.10:11452.11:11519.12:11633.13:11429.14:11267.15:11449.16:11492.17:11326.18:11375.19:11485.20:11296.21:11266.22:11470.23:11287.24:11243.25:11426.26:11432.27:11246.28:11155.29:11183.30:11066.31:11141.32:11537.33:11362.34:11057.35:11100.36:11169.37:11177.38:11240.39:12010.40:11379.41:11700.42:14966.43:13548.44:12949.45:12514.46:10956.47:10818.48:10687.49:10425.50:10342.51:10216.52:10197.53:10127.54:10326.55:10299.56:10515.57:10143.58:10411.59:10624.60:10115.61:10450.62:10837.63:10239.64:10022.

...


Finished processing table: patent202407 from 1608547 with size: 31495
0:9537.
Finished processing table: patent202408 from 1640042 with size: 9537
0:10023.1:10031.2:10017.3:10007.4:10012.5:5939.
Finished processing table: patent202410 from 1649579 with size: 56029
0:16991.1:17162.2:17079.3:16490.4:16578.5:16384.6:16258.7:16299.8:16336.9:16197.10:16460.11:16257.12:16399.13:16496.14:16633.15:16663.16:16508.17:16444.18:16417.19:16604.20:16410.21:16539.22:16471.23:16425.24:16194.25:16090.26:15929.27:15876.28:15800.29:15776.30:15724.31:15574.32:15594.33:15063.34:12786.35:12696.36:13059.37:13276.38:13966.39:14528.40:14577.41:14525.42:10123.

<timed exec>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent_incopat_202101_202211 from 1705608 with size: 673656
0:10035.1:10044.2:10028.3:10061.4:10074.5:10089.6:8072.

<timed exec>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Finished processing table: patent_incopat_202212_202312 from 2379264 with size: 68403
CPU times: total: 2min 10s
Wall time: 9min 50s


In [68]:
conn.close()

In [69]:
filter_df=pd.read_csv('data/patent_invest_all_2411.csv')
filter_df.table_name.value_counts()

table_name
patent202210                    826398
patent202404                    748835
patent_incopat_202101_202211    673656
patent_incopat_202212_202312     68403
patent202410                     56029
patent202309                     33314
patent202407                     31495
patent202408                      9537
Name: count, dtype: int64

In [77]:
%%time
for i,df_chunk in enumerate(pd.read_csv('data/source/patent_data_202411.tsv',sep='\t',usecols=columns,chunksize=100000)):
    mask= df_chunk.申请号.isin(tb_id['patent_incopat_2016b_202411'])
    if mask.sum()>0:
        filtered_chunk=df_chunk[mask]
        filtered_chunk['table_name'] = 'patent_incopat_2016b_202411'
        filter_df = pd.concat([filter_df,filtered_chunk], ignore_index=True)
        # filtered_chunk.to_csv(output_csv, mode='a', header=False, index=False)
        print(f'{filtered_chunk.shape[0]}.', end='', flush=True)                  

<timed exec>:1: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


33.

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


11.

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

4376.

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4076.

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4603.CPU times: total: 16min 24s
Wall time: 16min 38s


In [78]:
filter_df.table_name.value_counts()#229760

table_name
patent202210                    826398
patent202404                    748835
patent_incopat_202101_202211    673656
patent_incopat_2016b_202411     229760
patent_incopat_202212_202312     68403
patent202410                     56029
patent202309                     33314
patent202407                     31495
patent202408                      9537
Name: count, dtype: int64

In [82]:
filter_df['企业全称']=filter_df['申请人'].str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() in invest_co] if isinstance(x, list) else [])
(filter_df['企业全称'].map(len)==0).sum()

np.int64(13922)

In [148]:
patent_filter['table_name']='patent_incopat_2020b_202411'

In [149]:
filter_df_invest.shape,patent_filter.columns,filter_df_invest.columns

((2677427, 34),
 Index(['序号', '标题 (中文)', '摘要 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日',
        '专利类型', '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量',
        '简单同族', '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人',
        '被引证申请人', '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人',
        '工商统一社会信用代码', '企业全称', 'path', 'table_name'],
       dtype='object'),
 Index(['序号', '标题 (中文)', '摘要 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日',
        '专利类型', '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量',
        '简单同族', '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人',
        '被引证申请人', '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人',
        '工商统一社会信用代码', 'table_name', '企业全称'],
       dtype='object'))

In [150]:
patent_filter['path'].value_counts()

path
2020\2024-11-20.xlsx        9522
2019\2024-11-15.xlsx        8779
2018\2024-11-10.xlsx        8090
2020\2024-11-20 (1).xlsx    7947
2019\2024-11-15 (1).xlsx    7402
                            ... 
2020\2024-11-05 (5).xlsx    3474
2018\2024-11-06 (4).xlsx    3063
2018\2024-11-09 (4).xlsx    2830
2020\2024-11-15 (5).xlsx     876
2018\2024-11-10 (3).xlsx     192
Name: count, Length: 107, dtype: int64

In [154]:
(patent_filter['企业全称'].map(len)==0).sum()

np.int64(0)

In [152]:
filter_df_invest=pd.concat([filter_df_invest,patent_filter[filter_df_invest.columns]])
filter_df_invest.shape,(filter_df_invest['企业全称'].map(len)==0).sum()#(2677427, 34)

((3261605, 34), np.int64(13922))

In [156]:
filter_df_invest.to_csv(output_csv, index=False)
filter_df_invest.shape,output_csv

((3261605, 34), 'data/patent_invest_all_2411.csv')

In [157]:
patent_co=filter_df_invest[filter_df_invest['企业全称'].map(len)>0].explode('企业全称')
patent_co.shape,patent_co.企业全称.nunique(),patent_co.申请号.nunique()#(2706600, 34),27473, 2015626

((3301719, 34), 29792, 2256690)

In [158]:
patent_co['公开（公告）日']=pd.to_datetime(patent_co['公开（公告）日'])
patent_co['申请日']=pd.to_datetime(patent_co['申请日'])
patent_co=patent_co.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号','企业全称','专利类型'],keep='first').reset_index(drop=True)
patent_co.shape,patent_co.企业全称.nunique(),patent_co.申请号.nunique()

((2652202, 34), 29792, 2256690)

In [195]:
29792/len(invest_co),2256690/patent_all.申请号.nunique()

(0.3555428257730359, 0.10620772727116562)

In [196]:
patent_co[patent_co.专利类型=='发明申请'].申请号.nunique()/patent_all[patent_all.专利类型=='发明申请'].申请号.nunique()

0.08908319422392312

In [159]:
patent_co.to_csv('data/patent_co_invest_all_2256_30k_202411.csv')

In [41]:
filter_df_invest=pd.read_csv('data/patent_invest_all_2411.csv')
filter_df_invest['公开（公告）日'].isna().sum()

np.int64(0)

In [42]:
def parse_date(date_str):
    formats = ['%Y-%m-%d', '%Y-%m-%d %H:%M:%S']  # 定义可能的日期格式
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT
# filter_df_invest['公开（公告）日'].map(parse_date)
is_datetime = filter_df_invest['公开（公告）日'].str.contains(":", na=False)
filter_df_invest.loc[is_datetime, '公开（公告）日'] = pd.to_datetime(filter_df_invest.loc[is_datetime, '公开（公告）日'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
filter_df_invest.loc[~is_datetime, '公开（公告）日'] = pd.to_datetime(filter_df_invest.loc[~is_datetime, '公开（公告）日'], format='%Y-%m-%d', errors='coerce')

In [43]:
is_datetime = filter_df_invest['申请日'].str.contains(":", na=False)
filter_df_invest.loc[is_datetime, '申请日'] = pd.to_datetime(filter_df_invest.loc[is_datetime, '申请日'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
filter_df_invest.loc[~is_datetime, '申请日'] = pd.to_datetime(filter_df_invest.loc[~is_datetime, '申请日'], format='%Y-%m-%d', errors='coerce')
filter_df_invest.table_name.value_counts()

table_name
patent202210                    826398
patent202404                    748835
patent_incopat_202101_202211    673656
patent_incopat_2020b_202411     584178
patent_incopat_2016b_202411     229760
patent_incopat_202212_202312     68403
patent202410                     56029
patent202309                     33314
patent202407                     31495
patent202408                      9537
Name: count, dtype: int64

In [49]:
filter_df_invest['申请日'] = pd.to_datetime(filter_df_invest['申请日'])
filter_df_invest['公开（公告）日'] = pd.to_datetime(filter_df_invest['公开（公告）日'])
filter_df_invest[ '申请日'].dtypes

dtype('<M8[ns]')

In [48]:
import ast
filter_df_invest['企业全称']=filter_df_invest['企业全称'].dropna().apply(lambda x: ast.literal_eval(x)).map(';'.join)

In [50]:
filter_df_invest.to_csv('data/patent_invest_all_2411.csv', index=False)

In [2]:
patent_co=pd.read_csv('data/patent_co_invest_all_2256_30k_202411.csv')
patent_co['公开（公告）日']=pd.to_datetime(patent_co['公开（公告）日'])
patent_co['申请日']=pd.to_datetime(patent_co['申请日'])
patent_co.shape

(2652202, 35)

In [160]:
patent_co.专利有效性.value_counts()

专利有效性
有效    1518313
失效     587730
审中     546159
Name: count, dtype: int64

In [161]:
((patent_co.专利有效性=='有效')&(patent_co.专利类型=='发明授权')).sum()#369097

np.int64(369097)

In [5]:
patent_co['技术大类']=patent_co.IPC主分类.str.strip().str[:3]
patent_co['引证专利_A'] = patent_co['引证专利'].fillna('').str.split(';').apply(lambda x: [i.strip() for i in x if i.strip()])
patent_co['申请年']=patent_co['申请日'].dt.year

In [191]:
#年份从2005年到2019的每一个观察年2020年前
(patent_co['申请年']<=2020).sum()#1560553

np.int64(1856680)

In [192]:
patent_co.loc[patent_co['申请年']<=2020,'企业全称'].nunique(),patent_co.loc[patent_co['申请年']<=2020,'申请号'].nunique()#17303, 1301197

(23889, 1542261)

In [198]:
(invest_all['企业全称'].isin(patent_co['企业全称'])&(invest_all['交易时间'].dt.year<=2020)).sum(),(invest_all['企业全称'].isin(patent_co.loc[patent_co['申请年']<=2020,'企业全称'])).sum()#61350

(np.int64(47360), np.int64(53540))

In [256]:
filter_invest=invest_all[invest_all['企业全称'].isin(patent_co.loc[patent_co['申请年']<=2020,'企业全称'])&(invest_all['交易时间'].dt.year<=2020)]
filter_invest.shape

(42218, 67)

In [208]:
filter_patent_co=patent_co.loc[patent_co['企业全称'].isin(filter_invest['企业全称'])&(patent_co['申请年']<=2020)]
filter_patent_co.shape

(1790185, 37)

In [200]:
filter_invest.columns

Index(['企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话', '企业网址',
       '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址', '上市状态',
       '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1', '交易时间',
       '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式', '是否关联交易',
       '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型', '投资金额',
       '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称', '退出方基金名称',
       '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称', '中介类型',
       '投资方名称_A', '投资方基金名称_A', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业',
       '子行业', 'ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY',
       'GVC_Continuous'],
      dtype='object')

In [204]:
filter_invest.交易状态.unique(),filter_invest.交易方式.unique()

(array(['已完成', '进行中', '已失败'], dtype=object),
 array(['协议', nan, '集中竞价', '要约', '公开市场交易', '其他'], dtype=object))

In [257]:
filter_invest=filter_invest[['企业简称', '企业全称', '业务描述', '所在地', '详细地址','企业简介','成立时间', '统一信用代码','经营状态','上市状态',
       '股票代码','交易时间','交易状态','交易轮次', '交易金额', '单位','单位.1','单位.3', '投后估值','投资方名称','投资机构类型','投资方基金名称', '交易地区类型',  '投资方上市状态','投资金额','本次交易股权比例',
        '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业','子行业', 'ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY','GVC_Continuous']]
filter_invest

,企业简称,企业全称,业务描述,所在地,详细地址,企业简介,成立时间,统一信用代码,经营状态,上市状态,...,本次交易股权比例,政府引导基金的管理机构_机构全称,政府引导基金的管理机构_机构简称,行业,子行业,ChinaVC_alias,VC_fullname,GVC,GVC_DUMMY,GVC_Continuous
1,极智嘉,北京极智嘉科技股份有限公司,物流机器人研发商,北京市-朝阳区,北京市朝阳区创远路36号院1号楼1层101室 ;北京市朝阳区北苑路30号北京文化创意大厦D座7层,北京极智嘉科技股份有限公司（曾用名：北京极智嘉科技有限公司）成立于2015年，旗下Geek+...,2015-02-03,91110117330386452K,存续（在营、开业、在册）,未上市,...,; ;,NaN,NaN,先进制造,机器人,西藏高榕资本,西藏高榕资本管理有限公司,0,0.0,0.0
2,你好植物,成都你好植物科技有限公司,NaN,四川省-成都市,成都市高新区西区大道99号伯雅科技园,成都你好植物科技有限公司成立于2013年4月8日，是一家由十年以上快消品行业经验的年青团队创...,2013-04-08,91510106064327459U,存续（在营、开业、在册）,未上市,...,;,NaN,NaN,消费,餐饮,金顺东投资;奥都百升网络,上海金顺东投资管理有限公司;北京奥都百升网络科技有限公司,0;0,0.0,0.0
4,华宿电气,上海华宿电气股份有限公司,电气防火产品研发商,上海市-浦东新区,上海市张江高科技园区达尔文路88号6号楼1楼 ;中国（上海）自由贸易试验区盛荣路88弄1号楼...,上海华宿电气股份有限公司成立于2008年3月24日，是一家专业从事电力及电气安全产品研发、生...,2008-03-24,91310000672703163P,存续（在营、开业、在册）,未上市,...,NaN,NaN,NaN,传统制造,机械设备,时代伯乐,深圳市时代伯乐创业投资管理有限公司,0,0.0,0.0
5,玻思韬,广州玻思韬控释药业有限公司,创新药物研发商,广东省-广州市,广州高新技术产业开发区科学城开源大道11号C1栋第六层（加速器厂区） ;广州科学城开源大道1...,广州玻思韬控释药业有限公司成立于2013年7月15日，是一家创新药物研发商，玻思韬以新型药物...,2013-07-15,91440101072103236N,在营（开业）企业,未上市,...,; ; ; ;,NaN,华泰紫金投资,医疗健康,生物医药,华泰紫金投资;中建投资本,华泰紫金投资有限责任公司;中建投资本管理(天津)有限公司,0;1,1.0,0.5
7,甘来,广州甘来信息科技有限公司,智能零售机器生产商,广东省-广州市,;广州市越秀区环市东路339号广东国际大厦主楼22F,广州甘来信息科技有限公司致力于以SaaS方式提供服务，连接制造商、运营商、品牌商、广告商等角...,2015-04-29,914401153401044476,在营（开业）企业,未上市,...,; ; ;,NaN,NaN,电子信息,信息化服务,正基基金管理;汇能金融,西藏正基基金管理有限公司;深圳前海汇能金融控股集团有限公司,0;0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155299,派尼尔,浙江派尼尔科技股份有限公司,NaN,浙江省-金华市,岭下工业区 ;浙江省金华市婺城区秋滨街道神丽路1217号1#、2#厂房,"公司位于风景秀丽的中国宜居城市金华市,公司成立于2004年9月,公司致力于军民两用高性能舷外...",2004-09-02,91330701766424229P,存续,未上市,...,NaN,NaN,NaN,传统制造,机械设备,NaN,NaN,NaN,NaN,NaN
155300,波士顿电力,波士顿电力公司,NaN,美国,"2200 West Park Drive Westborough, MA 01581-396...",波士顿电力公司汇聚业界资深专家，共组管理团队。波士顿电力立志发展可靠电能，为日常用电产品“电...,NaT,NaN,NaN,未上市,...,; ; ;,NaN,NaN,能源及矿业,电池与储能技术,OAK;Venrock;纪源资本;Gabriel,Venrock Ltd.;纪源投资咨询(上海)有限公司;Oak Investment Par...,0;0;0;0,0.0,0.0
155304,贝因美,贝因美股份有限公司,NaN,浙江省-杭州市,浙江省杭州市滨江区浦沿街道伟业路1号10幢 ;浙江省杭州市滨江区南环路3758号,贝因美股份有限公司（002570.SZ）成立于1999年4月27日，业务涵盖婴幼儿食品、亲子...,1999-04-27,91330000712560866K,存续,A股上市公司,...,2.22%,NaN,NaN,消费,食品饮料,NaN,NaN,NaN,NaN,NaN
155307,导航犬,北京腾瑞万里信息技术有限公司,GPS手机导航服务平台,北京市-大兴区,北京朝阳区建国路89号华贸中心16号楼902室 ;北京市海淀区清华东路16号2005-3室,北京腾瑞万里信息技术有限公司成立于2004年11月，注册资金1000万元，是一家专注于以位置...,2004-11-16,91110108769397617K,存续（在营、开业、在册）,未上市,...,NaN,NaN,NaN,汽车交通,交通出行,四维图新,北京四维图新科技股份有限公司,0,0.0,0.0


In [258]:
filter_invest=filter_invest.merge(filter_patent_co.groupby('企业全称',as_index=False)['申请日'].agg(['min','max']),on='企业全称',how='left')

In [230]:
filter_invest['交易时间'].isna().sum()

np.int64(0)

In [259]:
%%time
co_year=filter_invest.groupby('企业全称').apply(lambda x:np.arange(min(x[['交易时间','成立时间','min']].min().dt.year),
             max(x[['交易时间','max']].max().dt.year)+1,step=1)).explode().reset_index().rename(columns={0:'观察年'})
co_year.shape

CPU times: total: 24.9 s
Wall time: 25.4 s


<timed exec>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


(222489, 2)

In [264]:
filter_invest['观察年']=filter_invest['交易时间'].dt.year
invest_co_year=co_year.merge(filter_invest.drop_duplicates(['企业简称','企业全称','交易时间','交易轮次']),on=['企业全称','观察年'],how='outer')
invest_co_year.info()#227264

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227264 entries, 0 to 227263
Data columns (total 38 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   企业全称              227264 non-null  object        
 1   观察年               227264 non-null  object        
 2   企业简称              42203 non-null   object        
 3   业务描述              34887 non-null   object        
 4   所在地               42153 non-null   object        
 5   详细地址              42092 non-null   object        
 6   企业简介              42199 non-null   object        
 7   成立时间              41999 non-null   datetime64[ns]
 8   统一信用代码            41945 non-null   object        
 9   经营状态              42067 non-null   object        
 10  上市状态              42214 non-null   object        
 11  股票代码              8657 non-null    object        
 12  交易时间              42214 non-null   datetime64[ns]
 13  交易状态              42214 non-null   object        
 14  交易轮次

In [261]:
invest_co_year.columns

Index(['企业全称', '观察年', '企业简称', '业务描述', '所在地', '详细地址', '企业简介', '成立时间', '统一信用代码',
       '经营状态', '上市状态', '股票代码', '交易时间', '交易状态', '交易轮次', '交易金额', '单位', '单位.1',
       '单位.3', '投后估值', '投资方名称', '投资机构类型', '投资方基金名称', '交易地区类型', '投资方上市状态',
       '投资金额', '本次交易股权比例', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业', '子行业',
       'ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY', 'GVC_Continuous',
       'min', 'max'],
      dtype='object')

In [ ]:
[ '企业简称', '业务描述', '所在地', '企业简介', '成立时间', '统一信用代码', '经营状态',
       '上市状态', '股票代码','行业', '子行业' ]

In [265]:
invest_co_year.to_csv('data/invest_year_2020_202411.csv',encoding="utf_8_sig",index=False)

In [349]:
def year_interval(filter_value,cols):
    filter_value_1 = filter_value.dropna(subset = cols) #filter_value.sort_values(by=['apply_date'],ascending=True)
    return (filter_value_1['申请日'].max()-filter_value_1['申请日'].min()).days//365+1

def process_ipc(filter_patent):
    ipcs_patent = filter_patent['IPC'].str.split(';').explode().str.strip().str.extract(r'(\w+)/')[0].str[:3]
    return 1.0 - (ipcs_patent.value_counts(normalize=True)**2).sum()

def process_ipc_2(filter_patent):
    ipcs_patent = filter_patent.loc[(filter_patent['专利类型'] == '发明授权') | (filter_patent['专利类型'] == '发明申请'), 'IPC'].str.split(';').explode().str.strip().str.extract(r'(\w+)/')[0].str[:3]
    return 1.0 - (ipcs_patent.value_counts(normalize=True)**2).sum()

def calculate_metrics(filter_value):
    metrics = {}
    
    # 申请号的唯一数量
    metrics['Application_num'] = filter_value['申请号'].nunique()
    
    # 去重后的 IPC 主分类数量
    metrics['IPC主分类_去重'] = filter_value['IPC主分类'].dropna().map(lambda x: x[:3]).nunique()
    
    # IPC 主分类的平均值
    metrics['IPC主分类_平均'] = metrics['IPC主分类_去重'] / year_interval(filter_value, 'IPC主分类')

    # 计算 IPC 的去重数量和平均值
    ipcs = filter_value['IPC'].str.split(';').explode().str.strip()
    metrics['IPC_个数_去重'] = ipcs.nunique()
    metrics['IPC_个数_平均'] = metrics['IPC_个数_去重'] / year_interval(filter_value, 'IPC')

    # 被引证次数和引证数量
    metrics['被引证次数'] = filter_value['被引证次数'].sum()
    metrics['被引证次数_平均'] = filter_value['被引证次数'].mean()
    metrics['引证数量'] = filter_value['引证次数'].sum()
    
    # 引证专利的去重数量和平均值
    citing = filter_value['引证专利'].str.split(';').explode().str.strip()
    metrics['引证数量_去重'] = citing.nunique()
    metrics['引证数量_平均'] = metrics['引证数量_去重'] / year_interval(filter_value, '引证专利')

    # 简单同族专利相关统计
    metrics['简单同族个数'] = filter_value['简单同族个数'].sum()
    cognate = filter_value['简单同族'].str.split(';').explode().str.strip()
    metrics['简单同族_去重'] = cognate.nunique()
    metrics['简单同族美日欧_比例'] = cognate.str[:2].isin(['US', 'JP', 'EP']).sum() / cognate.shape[0]

    # 发明人数量
    metrics['发明人数量'] = filter_value['发明人数量'].sum()
    metrics['发明人数量_去重'] = filter_value['发明人'].str.split(';').explode().str.strip().nunique()
    metrics['发明人数量_平均'] = metrics['发明人数量_去重'] / year_interval(filter_value, '发明人')

    # 权利要求数量和引证科技文献数量
    metrics['权利要求数量'] = filter_value['权利要求数量'].sum()
    metrics['引证科技文献数量'] = filter_value['引证科技文献'].count()  # 80%空,格式不统一
    
    # 有效专利数量和有效比例
    metrics['有效数量'] = (filter_value['专利有效性'] == '有效').sum()
    metrics['有效_比例'] = metrics['有效数量'] / filter_value.shape[0]

    # 发明授权和发明申请专利数量
    metrics['发明授权'] = (filter_value['专利类型'] == '发明授权').sum()
    metrics['发明申请'] = (filter_value['专利类型'] == '发明申请').sum()
    metrics['发明数量'] = ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请')).sum()

    metrics['实用新型']=(filter_value['专利类型'] == '实用新型').sum()
    metrics['外观设计']=(filter_value['专利类型'] == '外观设计').sum()

    copatent = (filter_value['申请人'].str.count(';') + 1 > 1)
    metrics['invention'] = (~copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['shiyong'] = (~copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['waiguan'] = (~copatent & (filter_value['专利类型'] == '外观设计')).sum()
    
    # 联合申请
    metrics['copatent_invention'] = (copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['copatent_shiyong'] = (copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['copatent_waiguan'] = (copatent & (filter_value['专利类型'] == '外观设计')).sum()

    return metrics

In [353]:
invest_co_year.columns

Index(['企业全称', '观察年', '企业简称', '业务描述', '所在地', '详细地址', '企业简介', '成立时间', '统一信用代码',
       '经营状态', '上市状态', '股票代码', '交易时间', '交易状态', '交易轮次', '交易金额', '单位', '单位.1',
       '单位.3', '投后估值', '投资方名称', '投资机构类型', '投资方基金名称', '交易地区类型', '投资方上市状态',
       '投资金额', '本次交易股权比例', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业', '子行业',
       'ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY', 'GVC_Continuous',
       'min', 'max', '投资方名称_A'],
      dtype='object')

In [356]:
n = [6, 5, 4, 3, 2] 
m1 = [5, 4, 3, 2, 1] 
m2 = [4, 3, 2, 1, 0]
tuple_list =[]
for ni in n:
    tuple_list.append((ni, m1[n.index(ni)])) 
    tuple_list.append((ni, m2[n.index(ni)])) 

tuple_list=tuple_list[:-1]
tuple_list

[(6, 5), (6, 4), (5, 4), (5, 3), (4, 3), (4, 2), (3, 2), (3, 1), (2, 1)]

In [358]:
year

np.int64(2019)

In [359]:
ci

14165

In [361]:
%%time
#patent_companie=invest_co_year.copy()
knowledge_year=5

for year in invest_co_year.观察年.unique():
    # if year<=2019:
    #     continue
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    invest_value=patent_companie[patent_companie.观察年 == year]
    patent_value=patent_co[patent_co['申请年'] <=  year]

    co_frequency = max(1, invest_value['企业全称'].nunique() // 100) 
    for ci,co in enumerate(invest_value['企业全称'].unique()):
        # if year<=2019 and ci <=14160:
        #     continue
        filter_patent= patent_value[patent_value['企业全称']==co]
        filter_patent= filter_patent.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')#'申请日',
        if filter_patent.empty:
            continue

        co_metrics ={}
        
        co_metrics['知识宽度'] = process_ipc(filter_patent) # 累积量
        co_metrics['知识宽度_2'] = process_ipc_2(filter_patent)
        
        mask_years = filter_patent['申请年'].between(year - knowledge_year, year - 1)
        co_metrics[f'前{knowledge_year}年知识宽度'] = process_ipc(filter_patent[mask_years]) 
        co_metrics[f'前{knowledge_year}年知识宽度_2'] = process_ipc_2(filter_patent[mask_years]) 
        tech_category =  filter_patent.loc[mask_years,'技术大类'].dropna().to_list()
        past_citations = filter_patent.loc[mask_years, '引证专利_A'].explode().dropna()#.str.strip()
        past_patents = filter_patent.loc[filter_patent['申请年'] != year, '公开（公告）号'].dropna().str.strip()
        existing_knowledge = set(past_patents) | set(past_citations)

        for n in range(knowledge_year, 0, -1):
            mask_years = filter_patent['申请年'].between(year - n, year - 1)
            co_metrics[f'前{n}年被引证次数']=filter_patent.loc[mask_years, '被引证次数'].sum()

        co_metrics.update(calculate_metrics(filter_patent))

        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            
            filter_value_date=filter_patent[filter_patent.申请日>=max_date-pd.Timedelta(days=N)]
            if filter_value_date.empty:
                continue
                
            mask=filter_value_date.申请日 <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]

            innovation_ipc =set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=filter_value_date.技术大类.isin(innovation_ipc).sum()
            co_metrics[f'突破性创新2_申请日_{suffix}']=innovation_patent_num
            co_metrics[f'突破性创新3_申请日_{suffix}']=len(innovation_ipc)#innovation_ipc_num 
            co_metrics[f'ipc_innovation_申请日_{suffix}']=innovation_patent_num / len(filter_value_date)
            #---
            mask= filter_value_date['公开（公告）日'] <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]

            innovation_ipc =set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=filter_value_date.技术大类.isin(innovation_ipc).sum()
            co_metrics[f'突破性创新2_公告日_{suffix}']=innovation_patent_num
            co_metrics[f'突破性创新3_公告日_{suffix}']=len(innovation_ipc)#innovation_ipc_num  
            co_metrics[f'ipc_innovation_公告日_{suffix}']=innovation_patent_num / len(filter_value_date)
        
        filter_value = filter_patent[filter_patent['申请年'] ==  year]
        if not filter_value.empty:           
            co_metrics.update({f'year_{k}': v for k, v in calculate_metrics(filter_value).items()})   # 计算指标
       
            explorative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) - set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            exploitative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            co_metrics['explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage)
            co_metrics['exploitative'] = (exploitative_percentage > 0.8).sum() / len(exploitative_percentage)
            co_metrics['jianjin_patent_num'] = filter_value['技术大类'].isin(tech_category).sum()
            
        if co_metrics:    
            co_metrics = {'patent_'+ key: value for key, value in co_metrics.items()}
            
            patent_companie.loc[invest_value[invest_value['企业全称']==co].index, co_metrics.keys()] = co_metrics.values() # pd.Series(co_metrics) np.repeat

        if ci % co_frequency == 0:
            print('.', end='', flush=True)

    print(year,patent_value['企业全称'].nunique(),invest_value['企业全称'].nunique(),invest_value['企业简称'].nunique())# 4h,125796,35438

patent_companie.to_csv('data\\融资与专利2020b_202411.csv', encoding="utf_8_sig")

.....................................................................................................2020 23889 12175 3468
CPU times: total: 2h 16min 47s
Wall time: 3h 26min 2s


1. 交易金额中的特殊字符
* 需要处理的内容：
    * 删除尾缀中包含（级）的字符：出现在一些字符型交易金额中，如 数千万（级）。
    * 多余的空格和换行符：在数据清理时需要去除。
* 处理方式：
    * 使用 .replace('（级）', '').strip() 去掉尾缀和清理空格。

2. 单位中的文字
* 需要处理的单位：
    * 带有 "万" 字符的单位：如 万人民币、万欧元、万美元。
    * 可能存在的异常单位：如 USD 或不规则单位，需在换算时忽略。
* 处理方式：
    * 清理单位中的多余文字：如 .replace('万', '').strip() 去掉 "万"。
    * 与汇率表（merged_data 的 CurrencyName 列）进行匹配。

3. 数值型和字符型交易金额的判断
* 需要处理的情况：
    * 数值型交易金额：直接可以转换为浮点数的交易金额（如 1000 万）。
    * 字符型交易金额：无法直接转换为浮点数的金额（如 数千万（级）、未披露 等）。
* 处理方式：
    * 判断是否为数值型：使用 float(amount) 尝试转换。
    * 对字符型金额保留原样，并尝试使用汇率转换。

4. 汇率相关的文字内容
* 需要匹配的文字：
    * CurrencyName 列中的外币单位名称：如 美元、欧元。
    * 日期列 SgnMonth：需格式化为 YYYY-MM 的年月形式。
* 处理方式：
    * 按交易金额中的单位名称与汇率表匹配，提取对应的汇率值 DollarConIntRMB_Ave。
    * 根据 交易时间 格式化为年月（如 2023-01），与汇率表中的 SgnMonth 匹配。

5. 字符型交易金额的换算规则
* 需要处理的字符金额内容：
    * 数千万（级）、未披露、数百万 等描述性金额。
* 处理方式：
    * 对于字符型金额，如果能提取数值信息（如 数千万），可进行估算或忽略。
    * 对于无法解析的金额（如 未披露），换算结果应标记为 None。

In [362]:
patent_companie

,企业全称,观察年,企业简称,业务描述,所在地,详细地址,企业简介,成立时间,统一信用代码,经营状态,...,patent_year_外观设计,patent_year_invention,patent_year_shiyong,patent_year_waiguan,patent_year_copatent_invention,patent_year_copatent_shiyong,patent_year_copatent_waiguan,patent_explorative,patent_exploitative,patent_jianjin_patent_num
0,NEC能源元器件株式会社,2006.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.0,0.0
1,NEC能源元器件株式会社,2007.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,2.0
2,NEC能源元器件株式会社,2008.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NEC能源元器件株式会社,2009.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,0.0,9.0,0.0,0.0,5.0,0.0,0.0,0.928571,0.0,13.0
4,NEC能源元器件株式会社,2010.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,0.0,9.0,0.0,0.0,2.0,0.0,0.0,0.818182,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227259,龟鹿药业集团有限公司,2016,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227260,龟鹿药业集团有限公司,2017,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227261,龟鹿药业集团有限公司,2018,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
227262,龟鹿药业集团有限公司,2019,龟鹿药业,生物阿胶研发商,河南省-驻马店市,栎城乡工贸开发区,龟鹿药业集团有限公司成立于2003年12月2日，是一家“研产销、原料基地、文化旅游”为一体的...,2003-12-02,914117296152920544,存续（在营、开业、在册）,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [298]:
invest_co_year[['投资方名称', '投资方基金名称','投资机构类型', '投资金额', '单位.3', '本次交易股权比例','交易地区类型', '投资方上市状态']]

,投资方名称,投资方基金名称,投资机构类型,投资金额,单位.3,本次交易股权比例,交易地区类型,投资方上市状态
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
227259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227262,中原联创 ;新蔡金财投资,农开豫仁发展基金 ;,PE ;企业,未披露; 未披露,;,;,境内交易 ;境内交易,未上市 ;未上市


In [303]:
invest_co_year['企业全称'].nunique()

20704

In [326]:
invest_invest_year=invest_co_year[['投资方名称', '投资方基金名称','投资机构类型', '投资金额', '单位.3', '本次交易股权比例','交易地区类型', '投资方上市状态']].copy()
for column in invest_invest_year.columns:
    invest_invest_year[column] = (invest_invest_year[column].str.split(';').dropna().apply(lambda x: [i.strip() for i in x ]) )#if i.strip()
invest_invest_year

,投资方名称,投资方基金名称,投资机构类型,投资金额,单位.3,本次交易股权比例,交易地区类型,投资方上市状态
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
227259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227262,"[中原联创, 新蔡金财投资]","[农开豫仁发展基金, ]","[PE, 企业]","[未披露, 未披露]","[, ]","[, ]","[境内交易, 境内交易]","[未上市, 未上市]"


In [327]:
all(invest_invest_year[invest_invest_year.columns].dropna().map(len).nunique(axis=1) == 1)

True

In [323]:
invest_invest_year=invest_invest_year.dropna(axis=0).explode(invest_invest_year.columns.to_list())
invest_invest_year

,投资方名称,投资方基金名称,投资机构类型,投资金额,单位.3,本次交易股权比例,交易地区类型,投资方上市状态
74,百利宏创投,,VC,1500.00,万元,0.99%,境内交易,未上市
74,君联资本,君联睿智基金,VC,10032.86,万元,6.62%,境内交易,未上市
74,天津弘毅投资,弘毅投资产业一期基金,VC、PE,7500.00,万元,4.95%,境内交易,未上市
114,天堂硅谷,浙江天堂硅谷银泽,PE,未披露,,,境内交易,未上市
114,天堂硅谷,天堂硅谷合盈创投,PE,未披露,,,境内交易,未上市
...,...,...,...,...,...,...,...,...
227232,广州碧蓝环保,,企业,数千万,元,,境内交易,未上市
227232,阿基米德金融,,企业,数千万,元,,境内交易,未上市
227232,磐鼎管理咨询,,企业,数千万,元,,境内交易,未上市
227262,中原联创,农开豫仁发展基金,PE,未披露,,,境内交易,未上市


In [329]:
for column in invest_invest_year.columns:
    filter_invest[column] = (filter_invest[column].str.split(';').dropna().apply(lambda x: [i.strip() for i in x ]) )#if i.strip()
all(filter_invest[filter_invest.columns].dropna().map(len).nunique(axis=1) == 1)

True

In [351]:
filter_invest.shape

(42218, 38)

In [348]:
filter_invest_1=filter_invest.dropna(subset=invest_invest_year.columns).explode(invest_invest_year.columns.to_list())
filter_invest_1=pd.concat([filter_invest_1,filter_invest[filter_invest[invest_invest_year.columns].isna().all(axis=1)]])
filter_invest_1

,企业简称,企业全称,业务描述,所在地,详细地址,企业简介,成立时间,统一信用代码,经营状态,上市状态,...,行业,子行业,ChinaVC_alias,VC_fullname,GVC,GVC_DUMMY,GVC_Continuous,min,max,观察年
0,极智嘉,北京极智嘉科技股份有限公司,物流机器人研发商,北京市-朝阳区,北京市朝阳区创远路36号院1号楼1层101室 ;北京市朝阳区北苑路30号北京文化创意大厦D座7层,北京极智嘉科技股份有限公司（曾用名：北京极智嘉科技有限公司）成立于2015年，旗下Geek+...,2015-02-03,91110117330386452K,存续（在营、开业、在册）,未上市,...,先进制造,机器人,西藏高榕资本,西藏高榕资本管理有限公司,0,0.0,0.0,2017-03-29,2020-12-31,2017
0,极智嘉,北京极智嘉科技股份有限公司,物流机器人研发商,北京市-朝阳区,北京市朝阳区创远路36号院1号楼1层101室 ;北京市朝阳区北苑路30号北京文化创意大厦D座7层,北京极智嘉科技股份有限公司（曾用名：北京极智嘉科技有限公司）成立于2015年，旗下Geek+...,2015-02-03,91110117330386452K,存续（在营、开业、在册）,未上市,...,先进制造,机器人,西藏高榕资本,西藏高榕资本管理有限公司,0,0.0,0.0,2017-03-29,2020-12-31,2017
0,极智嘉,北京极智嘉科技股份有限公司,物流机器人研发商,北京市-朝阳区,北京市朝阳区创远路36号院1号楼1层101室 ;北京市朝阳区北苑路30号北京文化创意大厦D座7层,北京极智嘉科技股份有限公司（曾用名：北京极智嘉科技有限公司）成立于2015年，旗下Geek+...,2015-02-03,91110117330386452K,存续（在营、开业、在册）,未上市,...,先进制造,机器人,西藏高榕资本,西藏高榕资本管理有限公司,0,0.0,0.0,2017-03-29,2020-12-31,2017
1,你好植物,成都你好植物科技有限公司,NaN,四川省-成都市,成都市高新区西区大道99号伯雅科技园,成都你好植物科技有限公司成立于2013年4月8日，是一家由十年以上快消品行业经验的年青团队创...,2013-04-08,91510106064327459U,存续（在营、开业、在册）,未上市,...,消费,餐饮,金顺东投资;奥都百升网络,上海金顺东投资管理有限公司;北京奥都百升网络科技有限公司,0;0,0.0,0.0,2018-10-21,2018-10-21,2017
1,你好植物,成都你好植物科技有限公司,NaN,四川省-成都市,成都市高新区西区大道99号伯雅科技园,成都你好植物科技有限公司成立于2013年4月8日，是一家由十年以上快消品行业经验的年青团队创...,2013-04-08,91510106064327459U,存续（在营、开业、在册）,未上市,...,消费,餐饮,金顺东投资;奥都百升网络,上海金顺东投资管理有限公司;北京奥都百升网络科技有限公司,0;0,0.0,0.0,2018-10-21,2018-10-21,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42176,江麓容大,湖南容大智能变速器股份有限公司,汽车无级自动变速器研发生产商,湖南省-长沙市,湖南省长沙市岳麓区杜容路68号,湖南江麓容大车辆传动股份有限公司由原湖南容大汽车电子技术有限公司股东、江麓机电科技有限公司、...,2003-11-20,9143000075582020XW,存续（在营、开业、在册）,未上市,...,汽车交通,汽车零部件,NaN,NaN,NaN,NaN,NaN,2020-04-30,2020-04-30,2008
42186,木林森,木林森股份有限公司,LED封装及应用产品供应商,广东省-中山市,中山市小榄镇木林森大道1号,"木林森股份有限公司（002745.SZ）木林森股份有限公司，成立于1997年。主品牌标识为""...",1997-03-03,914420002821438692,在营（开业）企业,A股上市公司,...,电子信息,半导体,NaN,NaN,NaN,NaN,NaN,2010-09-19,2020-12-28,2008
42191,东软集团,东软集团股份有限公司,NaN,辽宁省-沈阳市,辽宁省沈阳市浑南新区新秀街2号东软软件园 ;沈阳市浑南新区新秀街2号,东软集团股份有限公司（600718.SH）成立于1991年6月17日，以软件技术为核心，业务...,1991-06-17,91210100604608172K,存续（在营、开业、在册）,A股上市公司,...,电子信息,软件,NaN,NaN,NaN,NaN,NaN,2005-05-30,2020-12-31,2008
42212,杰瑞股份,烟台杰瑞石油服务集团股份有限公司,NaN,山东省-烟台市,烟台市莱山区杰瑞路5号 ;山东省烟台市莱山区熬柯玛大街7号,烟台杰瑞石油服务集团股份有限公司（002352.SZ）成立于1999年12月10日，主营油田...,1999-12-10,91370000720717309H,在营（开业）企业,A股上市公司,...,传统制造,机械设备,NaN,NaN,NaN,NaN,NaN,2008-02-13,2019-11-05,2008


In [347]:
filter_invest_1.to_csv('data/invest_2020_202411.csv',encoding="utf_8_sig",index=True)

In [324]:
invest_invest_year.to_csv('data/invest_invest_2020_202411.csv',encoding="utf_8_sig",index=True)

In [101]:
merged_data=pd.read_csv('data/source/合并后的货币汇率数据.csv')
merged_data.columns

Index(['SgnMonth', 'CountryCode', 'Cntrnm', 'CurrencyCode', 'CurrencyName',
       'DollarConIntRMB_End', 'DollarConIntRMB_Ave', 'SDRConIntRMB_End',
       'SDRConIntRMB_Ave'],
      dtype='object')

In [104]:
merged_data.Cntrnm.nunique(),merged_data.CountryCode.nunique(),merged_data.CurrencyCode.nunique(),

(209, 209, 174)

In [107]:
merged_data.SgnMonth.min(),merged_data.SgnMonth.max()

('1940-01', '2024-10')

In [ ]:
df_temp=pd.read_excel(r'/Users/wumian/Desktop/cjy合作论文/data/invest_all.xlsx',index_col=0)
df_temp.replace({'_x000D_\\n':';','_x000D_': ';','_x000D_\n': ';','\r\n': ';'}, regex=True, inplace=True)#
df_temp = df_temp.reset_index().reset_index().rename(columns={'level_0':'invest_id'})

# 假设 df_temp 是原始数据框
# 需要拆分的列
split_columns = ['投资方名称', '投资机构类型', '投资金额', '单位.3', '本次交易股权比例']

# 创建一个空的 DataFrame 来存储拆分后的数据
df_temp_explode = pd.DataFrame()

# Step 1: 对需要拆分的列进行 str.split(';') 并 explode
for col in split_columns:
    # 对每一列进行拆分并添加 invest_id
    df_temp_explode_new = df_temp[col].fillna('').astype(str).str.split(';').explode().reset_index().rename(columns={'index':'invest_id'})
    
    # 给每个拆分后的行添加一个新的列 'invest_id' 来唯一标识每个投资项
    
    # 将拆分后的列添加到 df_temp_explode 中
    df_temp_explode[col] = df_temp_explode_new[col]

# Step 2: 给 df_temp 添加一个新的列 'invest_id'，并在合并时作为基准
df_temp_explode['invest_id'] = df_temp_explode_new['invest_id']

# Step 3: 合并 df_temp_explode 和原始 df_temp（基于 'invest_id'）
df_temp_merged = df_temp_explode.merge(df_temp, left_on='invest_id', right_on='invest_id', how='left')
columns_to_fill = ['企业简称', '企业全称', '所在地','详细地址','统一信用代码','交易金额', '单位', '交易轮次', '交易时间', '交易状态','投资方基金名称', '交易地区类型', 
       '投资方上市状态','投资方名称_x', '投资机构类型_x', '投资金额_x', '单位.3_x', '本次交易股权比例_x', 'invest_id' ]

df_temp_merged_final = df_temp_merged.sort_values(by=['invest_id','企业全称'])[columns_to_fill]

In [ ]:
df_temp[['交易轮次', '交易金额', '单位','投后估值', '单位.1' ]]
# 1. 定义清理和合并交易金额的函数
def clean_and_merge_amount(row):
    """
    清理交易金额的尾缀（级），并合并金额和单位。
    """
    amount = str(row['交易金额']).replace('（级）', '').strip()  # 去掉“（级）”并清理空格
    unit = str(row['单位']).strip()  # 清理单位
    return f"{amount} {unit}"

# 2. 应用清理和合并函数
df_temp['交易金额_单位'] = df_temp.apply(clean_and_merge_amount, axis=1)

# 3. 将交易金额分为数值型和字符型
def is_numeric_amount(row):
    """
    判断交易金额是否为数值型（可直接转换为浮点数）。
    """
    try:
        amount, unit = row['交易金额_单位'].split()
        float(amount)  # 尝试转换金额为浮点数
        return True
    except ValueError:
        return False

# 标记交易金额为数值型或字符型
df_temp['是否数值型'] = df_temp.apply(is_numeric_amount, axis=1)

# 4. 定义换算函数（针对字符型金额）
def convert_currency_to_rmb(row, merged_data):
    """
    根据交易时间和汇率数据将字符型交易金额转换为人民币。
    """
    try:
        amount, unit = row['交易金额_单位'].split()
        if not row['是否数值型']:  # 只处理字符型金额
            unit_cleaned = unit.replace('万', '').strip()  # 去掉“万”并清理单位
            date_str = pd.to_datetime(row['交易时间']).strftime('%Y-%m')  # 格式化为年月字符串

            # 查找对应时间的汇率
            rate_row = merged_data[(merged_data['SgnMonth'] == date_str) & 
                                   (merged_data['CurrencyName'] == unit_cleaned)]
            if not rate_row.empty:
                # 外币转换为人民币
                to_rmb_rate = rate_row['DollarConIntRMB_Ave'].values[0]
                return round(float(amount) * to_rmb_rate, 2)  # 保留两位小数
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

    return None

# 5. 分别处理数值型和字符型金额
df_temp['交易金额_RMB'] = df_temp.apply(
    lambda row: float(row['交易金额']) * 10000 if row['是否数值型'] else convert_currency_to_rmb(row, merged_data),
    axis=1
)

# 6. 删除中间步骤的临时列（可选）
df_temp.drop(columns=['交易金额_单位', '是否数值型'], inplace=True)

In [ ]:
#因为那个invest_all里面还有一个这个数据，我觉得是可以先做为判断标准的：
# 假设 df_multiple_investors 是待处理的数据

# 1. 按 '企业全称' 分组并重设索引
grouped = df_multiple_investors.groupby('企业全称', as_index=False)

# 2. 对股权比例不缺失的部分，按股权比例排序并选择股权比例最大的一行作为领投机构
df_no_missing_equity = grouped.apply(lambda x: x[x['本次交易股权比例_x'].notna()])  # 选择股权比例不缺失的部分
df_no_missing_equity = df_no_missing_equity.sort_values(by=['企业全称', '本次交易股权比例_x'], ascending=[True, False])

# 选择股权比例最大的那一行作为领投机构
df_no_missing_equity['领投机构'] = df_no_missing_equity.groupby('企业全称')['投资方名称_x'].transform('first')

# 3. 对股权比例缺失的部分进行处理
df_missing_equity = grouped.apply(lambda x: x[x['本次交易股权比例_x'].isna()])  # 选择股权比例缺失的部分
df_missing_equity['领投机构'] = None  # 暂时设置为 None，后续可以根据其他条件处理

# 4. 合并处理后的数据
df_final = pd.concat([df_no_missing_equity, df_missing_equity])

# 5. 排序并清理索引
df_final = df_final.sort_values(by=['企业全称', '本次交易股权比例_x'], ascending=[True, False]).reset_index(drop=True)

1—判断领投规则 1、看该投融资事件是不是有多个投资方，如果只有一个投资方，则以该投资方为领投   
2、如果有多个投资方   
1）看本次交易股权比例是否有值，如果有值的话，选择股权交易比例最大的为领投机构2）如果没有值或者交易比例相同的话，首先根据每个投资机构在这家公司之前投过的事件判断，如果之前有投过，并且在前面轮次有股权交易比例，并且是最大的话，则判断这家投资机构为领投 3）最后根据这家投资机构在所有投融资事件中，截止到交易时间为止的投资次数，选择投资次数最大的为领投机构。

2—下面这两个的判断也需要结合领投来判断

In [ ]:

invest_co['政府引导基金_DUMMY']=(~invest_co['政府引导基金的管理机构_机构简称'].isna()).astype(int)
invest_co['政府引导基金_DUMMY'].sum()

invest_co['GVC']=invest_co['GVC'].dropna().map(lambda x: ';'.join(str(i) for i in x))
invest_co['投资方名称_A']=invest_co['投资方名称_A'].dropna().map(lambda x: ';'.join(x))

然后VC行业经验和VC声誉先用领投的去计算，如果计算出来的领投有多个的话，就加权吧

In [4]:
invest_all.columns

Index(['企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话', '企业网址',
       '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址', '上市状态',
       '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1', '交易时间',
       '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式', '是否关联交易',
       '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型', '投资金额',
       '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称', '退出方基金名称',
       '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称', '中介类型',
       '投资方名称_A', '投资方基金名称_A', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业',
       '子行业', 'ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY',
       'GVC_Continuous', '投资方名称_B'],
      dtype='object')

In [44]:
invest_all['投资金额_A']=invest_all['投资金额'].str.split(';').dropna().map(lambda x:[i.strip() if i.strip() else pd.NA for i in x])

In [10]:
invest_all['投资方名称_A']=invest_all['投资方名称'].str.split(';').dropna().map(lambda x:[i.strip() if i.strip() else pd.NA for i in x])
invest_all['投资方名称_B']=invest_all['投资方名称'].str.split(';').map(lambda x: list(set(i.strip() for i in x if i.strip())) if isinstance(x, list) else [])

In [5]:
invest_all['投资方基金名称_B']=invest_all['投资方基金名称'].str.split(';').map(lambda x: list(set(i.strip() for i in x if i.strip())) if isinstance(x, list) else [])
invest_all['投资机构基金数量']=invest_all['投资方基金名称_B'].dropna().map(len)#lambda x:len(set(x)))
invest_all['投资机构数量']=invest_all['投资方名称_B'].dropna().map(len)

In [6]:
invest_all['Age']=(invest_all['交易时间']- invest_all['成立时间']).dt.days//365+1
invest_all['交易年月']=invest_all['交易时间'].astype(str).str[:7]

In [7]:
invest_all[['省','市']]=invest_all.所在地.str.split('-', expand=True).fillna(pd.NA)
invest_all.省=invest_all.省.str.replace('省|市','', regex=True).str.strip()
invest_all.市=invest_all.市.str.replace('市','').str.strip()

In [14]:
invest_all['本次交易股权比例_A']=invest_all['本次交易股权比例'].str.split(';').dropna().map(lambda x: [float(i.strip().strip('%')) if i.strip() else np.nan for i in x])# / 100 
invest_all['本次交易股权比例_B']=invest_all['本次交易股权比例'].str.split(';').map(lambda x: [float(i.strip().strip('%')) for i in x if i.strip()] if isinstance(x, list) else [])

In [86]:
invest_all['交易方式'].value_counts()

交易方式
协议        134091
公开市场交易        59
要约            30
集中竞价          17
其他            15
Name: count, dtype: int64

In [85]:
invest_all['支付方式'].value_counts()

支付方式
现金      154688
其他         207
混合支付       180
资产          97
股票          91
Name: count, dtype: int64

In [84]:
invest_all['交易类型'].value_counts()

交易类型
增资      148192
股权转让      4248
合资设立      1666
股份回购       858
其他         283
资产收购        16
Name: count, dtype: int64

In [83]:
invest_all['中介类型'].str.split(';').explode().str.strip().value_counts()

中介类型
投行      10620
会计所      5761
律所       4166
评估公司      370
Name: count, dtype: int64

In [81]:
invest_all['投资机构类型'].str.split(';').explode().str.strip().replace('',pd.NA).value_counts()

投资机构类型
VC、PE            107056
企业                81957
VC                68676
PE                37194
早期投资机构            12772
VC、早期投资机构          2925
VC、战略投资者           1717
PE、战略投资者           1689
PE、券商直投            1495
战略投资者               875
人物                  812
VC、PE、战略投资者         807
孵化器                 556
VC、PE、早期投资机构        462
早期投资机构、孵化器          304
券商直投                220
VC、孵化器              206
VC、早期投资机构、孵化器        58
VC、PE、券商直投           38
PE、早期投资机构            29
战略投资者、早期投资机构         28
PE、企业                21
VC、PE、孵化器            15
VC、券商直投               2
PE、战略投资者、企业           2
孵化器、企业                1
VC、PE、企业              1
Name: count, dtype: int64

In [79]:
invest_all['投资方交易所'].str.split(';|,',regex=True).explode().str.strip().replace('',pd.NA).value_counts()

投资方交易所
上海证券交易所            4876
深圳证券交易所            4398
香港交易所              3165
全国中小企业股份转让系统       1937
纽约证券交易所            1031
纳斯达克                573
东京证券交易所             177
泛欧交易所               102
台湾证券交易所              38
伦敦证券交易所              35
北京证券交易所              34
新加坡交易所                8
韩国证券期货交易所             7
澳大利亚证券交易所             5
法兰克福证券交易所             4
苏黎世证券交易所              4
多伦多证券交易所              3
美国证券交易所               3
纳斯达克OMX斯德哥尔摩交易所       2
Name: count, dtype: int64

In [8]:
import numpy as np
invest_all['北上广深']=np.where(invest_all['省'].isin(['北京','上海','北京市','上海市']) | invest_all['市'].isin(['深圳','广州']),1,0)

In [40]:
(invest_all['市']=='宁波').sum()

np.int64(1665)

In [43]:
invest_all.loc[invest_all['市']=='宁波','投资方名称_B'].explode().value_counts()

投资方名称_B
宁波市天使投资引导基金管理          174
宁波通商基金                  29
宁波市知识产权运营基金有限公司（管理）     25
海邦投资                    22
镇海产业发展基金                21
                      ... 
元禾厚望                     1
上海国投创投                   1
万向钱潮                     1
勇诚资产                     1
国新国同                     1
Name: count, Length: 1967, dtype: int64

In [41]:
invest_all[invest_all['市']=='宁波'].to_excel('data/invest_all_宁波.xlsx')

In [74]:
invest_all[(invest_all['市']=='宁波')&(invest_all['详细地址'].str.contains('奉化').fillna(False)|invest_all['注册地址'].str.contains('奉化').fillna(False))].to_excel('data/invest_宁波奉化.xlsx')

C:\Users\Admin\AppData\Local\Temp\ipykernel_4008\3605488784.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  invest_all[(invest_all['市']=='宁波')&(invest_all['详细地址'].str.contains('奉化').fillna(False)|invest_all['注册地址'].str.contains('奉化').fillna(False))].to_excel('data/invest_宁波奉化.xlsx')


In [12]:
invest_all=invest_all.sort_values(['企业全称','交易时间']).reset_index()
invest_all['Cum_invset']=invest_all.groupby(['企业全称'])['交易时间'].transform(lambda x: x.rank(method='min'))

In [13]:
invest_all['本次交易股权比例_A'].explode().dropna().astype(float).describe()

count    53501.000000
mean         7.276539
std         11.744171
min          0.000000
25%          1.240000
50%          3.120000
75%          8.000000
max        100.000000
Name: 本次交易股权比例_A, dtype: float64

In [15]:
a=invest_all['本次交易股权比例_B'].apply(sum)
a[a>0]

69         0.71
98        30.30
99        21.66
100       13.08
101       26.45
          ...  
155295    11.76
155296     7.32
155299     2.30
155300    33.08
155301     4.00
Name: 本次交易股权比例_B, Length: 24601, dtype: float64

In [61]:
a.max()

np.float64(100.01)

In [55]:
invest_all.iloc[99][['本次交易股权比例','投资方名称']]

本次交易股权比例    10.91% ;2.14% ;2.18% ;4.29% ;2.14%
投资方名称         纪源资本 ;顺为资本 ;海纳亚洲创投基金 ;京东科技 ;小米科技
Name: 99, dtype: object

In [16]:
invest_all[['投资方名称_B','本次交易股权比例_B','投资方名称_A','本次交易股权比例_A']]

,投资方名称_B,本次交易股权比例_B,投资方名称_A,本次交易股权比例_A
0,"[招银国际, HashKey Capital, GSR, 青松基金, SNZ, 纪源资本]",[],"[招银国际, HashKey Capital, 青松基金, 纪源资本, SNZ, GSR]","[nan, nan, nan, nan, nan, nan]"
1,"[Delphi Digital, Hack VC, Alliance VC, Animoca...",[],"[Hack VC, Alliance VC, Delphi Digital, Symboli...","[nan, nan, nan, nan, nan, nan, nan]"
2,"[Delphi Digital, Hack VC, Alliance VC, Animoca...",[],"[Hack VC, Alliance VC, Delphi Digital, Symboli...","[nan, nan, nan, nan, nan, nan, nan]"
3,"[Bonfire Union, ABCDE, 梁信军, Bodl Ventures, Has...",[],"[ABCDE, Hash Global, 梁信军, Bonfire Union, Bodl ...","[nan, nan, nan, nan, nan]"
4,"[HashKey Capital, XIN Family, GSR, 红点中国, 丹华资本,...",[],"[HashKey Capital, 红点中国, OKX Ventures, GSR, 丹华资...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...,...
155304,[老鹰基金],[],[老鹰基金],NaN
155305,[],[],NaN,NaN
155306,[True Ventures],[],[True Ventures],NaN
155307,"[弘博资本, 璟侑投资]",[],"[弘博资本, 璟侑投资]","[nan, nan]"


In [17]:
all(invest_all[['投资方名称_A','本次交易股权比例_A']].dropna().map(len).nunique(axis=1) == 1)

True

In [215]:
(invest_all['投资机构数量']==1).sum(),(invest_all['投资机构数量']>1).sum(),(invest_all['投资机构数量']==0).sum()

(np.int64(77048), np.int64(64942), np.int64(13319))

In [24]:
mask=invest_all['投资机构数量']==1
invest_all.loc[mask,'领投机构']=invest_all.loc[mask,'投资方名称'].str.strip()

In [35]:
invest_all['投资机构数量_A']=invest_all['投资方名称_A'].dropna().map(len)
mask=(invest_all['投资机构数量']==1)&(invest_all['投资机构数量_A']>1)
invest_all.loc[mask,'领投机构_1']=invest_all.loc[mask,'投资方名称_B'].map(lambda x:x[0])
invest_all.loc[mask,'领投机构_1']

98               DCM公司
145              IDG资本
146              IDG资本
594             鼎晖地产投资
3341              行健资本
              ...     
154889            博润投资
154930            万信投资
154943          深圳东方汇富
154949    黑河国众产业引导基金管理
155217            科力投资
Name: 领投机构_1, Length: 2307, dtype: object

In [36]:
mask=(invest_all['投资机构数量']>0)&(invest_all['投资机构数量_A']>invest_all['投资机构数量'])
mask.sum()

np.int64(9511)

In [30]:
invest_all[mask].to_excel('data/vc_重复_2.xlsx')

In [34]:
mask=(invest_all['投资机构数量']>1)&(invest_all['本次交易股权比例_B'].map(len)>0)
invest_all.loc[mask,['投资方名称_A','本次交易股权比例_A','投资方名称_B','投资方基金名称_B']]

,投资方名称_A,本次交易股权比例_A,投资方名称_B,投资方基金名称_B
99,"[纪源资本, 顺为资本, 海纳亚洲创投基金, 京东科技, 小米科技]","[10.91, 2.14, 2.18, 4.29, 2.14]","[顺为资本, 海纳亚洲创投基金, 小米科技, 京东科技, 纪源资本]",[SIG China Investments Master Fund III]
121,"[英联投资, 华平投资]","[16.6, 4.98]","[华平投资, 英联投资]","[英联中国投资基金二期, 华平房地产基金]"
147,"[祥峰投资中国基金, 智基创投, 德同资本]","[7.14, 3.57, 3.57]","[德同资本, 智基创投, 祥峰投资中国基金]","[智二基金, 德同资本中国成长基金, Vertex Asia Growth Ltd]"
149,"[Pacific Century Diversified Limited, Grandwin...","[4.14, 0.43]","[Grandwin Enterprises Limited, Pacific Century...",[]
231,"[一品红, Montesy Capital]","[3.11, 6.23]","[Montesy Capital, 一品红]",[]
...,...,...,...,...
155225,"[深圳力合清源创投, 嘉盛兴业]","[5.0, nan]","[深圳力合清源创投, 嘉盛兴业]",[力合清源创业投资基金]
155232,"[西藏金葵花资本, 无锡浚源资本, 浚源投资, 飞凡协立, 无锡金投, 无锡金投]","[4.51, 0.63, 1.63, 1.35, 2.71, 2.71]","[西藏金葵花资本, 无锡浚源资本, 浚源投资, 无锡金投, 飞凡协立]","[潍坊浚源基金, 飞凡协立物联网创投, 金投产业升级基金, 金投永赢产投基金]"
155233,"[上海金浦投资, 东证资本, 深圳力鼎]","[8.09, 5.01, 1.54]","[东证资本, 上海金浦投资, 深圳力鼎]","[嘉兴力鼎三号投资, 诸暨东证致臻基金, 金浦国调基金]"
155260,"[元禾控股, 上海乾瞻投资]","[6.49, 2.6]","[上海乾瞻投资, 元禾控股]","[元禾之星创业基金, 乾瞻广鑫基金]"


In [168]:
invest_all.loc[145,['投资方名称']]

投资方名称    IDG资本 ;IDG资本 ;IDG资本
Name: 145, dtype: object

In [135]:
invest_all.loc[mask,'领投机构_21']=invest_all[mask].apply(lambda row: row['投资方名称_A'][row['本次交易股权比例_A'].index(max(row['本次交易股权比例_A']))]
                                                   if row['本次交易股权比例_A'].count(max(row['本次交易股权比例_A'])) == 1 else None,axis=1)
invest_all.loc[mask,'领投机构_21']

98           DCM公司
99            纪源资本
121           英联投资
145          IDG资本
146          IDG资本
            ...   
155225    深圳力合清源创投
155232     西藏金葵花资本
155233      上海金浦投资
155260        元禾控股
155300        赛富盛元
Name: 领投机构_21, Length: 12333, dtype: object

In [136]:
invest_all.loc[mask,'领投机构_21'].isna().sum()

np.int64(2150)

如果没有值或者交易比例相同的话，首先根据每个投资机构在这家公司之前投过的事件判断，如果之前有投过，并且在前面轮次有股权交易比例，并且是最大的话，则判断这家投资机构为领投

In [139]:
mask=(invest_all['投资机构数量']>1)&(invest_all['领投机构_21'].isna())
mask.sum()

np.int64(57058)

In [151]:
def find_lead_investor(group):
    group_name = group['企业全称'].iloc[0]
    exploded = group.explode(['投资方名称_A', '本次交易股权比例_A']).reset_index(drop=True)
    exploded = exploded[exploded['本次交易股权比例_A'].notna()]
    if exploded.empty:
        return None
    max_index = exploded[exploded['本次交易股权比例_A'] == exploded['本次交易股权比例_A'].max()]#.idxmax()
    if max_index['投资方名称_A'].nunique() == 1:
        return max_index.iloc[0]['投资方名称_A']

    return None #exploded.loc[max_index, '投资方名称_A']

In [156]:
invest_all.loc[mask,'领投机构_22']=invest_all[mask].groupby('企业全称', group_keys=False).apply(find_lead_investor)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17940\2169620777.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  invest_all.loc[mask,'领投机构_22']=invest_all[mask].groupby('企业全称', group_keys=False).apply(find_lead_investor)


In [154]:
invest_all.loc[mask,'领投机构_22'].dropna()

Series([], Name: 领投机构_22, dtype: object)

In [49]:
invest_vc=invest_all.explode(['投资方名称_A','本次交易股权比例_A','投资金额_A']).rename(columns={
    '投资方名称_A':'投资机构','本次交易股权比例_A':'投资方股权比例'}).dropna(subset='投资机构').reset_index()#.explode('投资方名称_B')
invest_vc.shape#333462投资金额

(320143, 85)

In [46]:
(invest_vc['投资机构']=='').sum()

np.int64(0)

In [123]:
invest_vc['投资机构']=invest_vc['投资机构'].replace('',pd.NA)
invest_vc.dropna(subset='投资机构',inplace=True)
invest_vc.shape

(320143, 85)

In [50]:
(invest_vc['投资金额_A']=='未披露').sum()

np.int64(150720)

In [54]:
invest_vc.loc[invest_vc['投资金额_A']=='未披露','投资机构'].value_counts().head(20)

投资机构
深创投        512
紫金科创       492
达晨财智       424
真格基金       361
险峰长晴       301
光谷人才基金     272
IDG资本      262
君联资本       261
中科招商       249
梅花创投       228
前海方舟       221
创新工场       213
创东方投资      212
九鼎投资       206
啟赋资本       206
涌铧投资       206
广发信德       201
拉萨顺为       196
深圳高新投创投    193
老虎基金       190
Name: count, dtype: int64

In [88]:
(invest_vc['投资方股权比例'].isna()&invest_vc['投资金额_A'].notna()).sum()

np.int64(255706)

In [89]:
(invest_vc['投资方股权比例'].isna()&invest_vc['投资金额_A'].notna()&(invest_vc['投资金额_A']!='未披露')).sum()

np.int64(116035)

In [128]:
invest_all['投资方名称_B'].explode().dropna().count(),invest_all['投资方名称_B'].explode().dropna().nunique()

(np.int64(307789), 69270)

In [20]:
invest_vc_B=invest_all.explode(['投资方名称_B']).rename(columns={'投资方名称_B':'投资机构'})
invest_vc_B.投资机构.isna().sum()

np.int64(13319)

In [22]:
invest_vc_B.dropna(subset='投资机构',inplace=True)
invest_vc_B.shape

(307789, 85)

In [23]:
invest_vc_B.sort_values(['企业全称','企业简称', '行业', '子行业', '交易时间','投资机构'],inplace=True)
invest_vc_B.reset_index(drop=True,inplace=True)

In [26]:
invest_vc_B['企业被投资次数']=invest_vc_B.groupby(['企业全称', '投资机构'])['交易时间'].transform(lambda x: x.rank(method='min'))
invest_vc_B['子行业投资次数']=invest_vc_B.groupby(['子行业','投资机构'])['交易时间'].transform(lambda x: x.rank(method='min'))#, '行业'
invest_vc_B['机构投资次数']=invest_vc_B.groupby(['投资机构'])['交易时间'].transform(lambda x: x.rank(method='min'))
invest_vc_B['机构投资次数'].mean(),invest_vc_B['机构投资次数'].median(),invest_vc_B['企业被投资次数'].mean()

(np.float64(40.24770540857536),
 np.float64(8.0),
 np.float64(1.1389263423969018))

In [61]:
invest_vc_B.投资机构.nunique()

69270

In [293]:
invest_vc_B['企业被投资次数'].describe()

count    307789.000000
mean          1.138926
std           0.459744
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          12.000000
Name: 企业被投资次数, dtype: float64

In [166]:
invest_vc_B['子行业投资次数'].describe()

count    307785.000000
mean          3.946485
std           8.903325
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         174.000000
Name: 子行业投资次数, dtype: float64

In [27]:
invest_vc_B['机构投资次数'].describe()

count    307789.000000
mean         40.247705
std         100.676120
min           1.000000
25%           2.000000
50%           8.000000
75%          32.000000
max        1413.000000
Name: 机构投资次数, dtype: float64

In [294]:
invest_vc_B[invest_vc_B['企业被投资次数']==12].to_dict()

{'index': {12965: 71105},
 '企业简称': {12965: 'Flipkart'},
 '企业全称': {12965: 'Flipkart Oneline Services Pvt. Ltd.'},
 '业务描述': {12965: '综合电商平台'},
 '所在行业': {12965: '批发零售-电子商务'},
 '所在地': {12965: '印度'},
 '企业简介': {12965: 'Flipkart Oneline Services Pvt. Ltd.是印度一家综合电商平台，主要销售书籍、音乐和电影周边产品、3c产品等，同时也有自主品牌“DigiFlip”主要生产笔记本电脑和3c产品。用户可通过线上平台和手机APP进行自主购物。'},
 '详细地址': {12965: 'Outer Ring Road, Devarabeesanahalli Village,Bengaluru, 560103,Karnataka, India'},
 '联系电话': {12965: '1800 208 9898'},
 '企业网址': {12965: 'http://www.Flipkart.com'},
 '邮箱': {12965: nan},
 '成立时间': {12965: NaT},
 '统一信用代码': {12965: nan},
 '法人代表': {12965: nan},
 '注册资本': {12965: nan},
 '经营状态': {12965: nan},
 '经营范围': {12965: nan},
 '注册地址': {12965: nan},
 '上市状态': {12965: '未上市'},
 '股票代码': {12965: nan},
 '交易所': {12965: nan},
 '交易轮次': {12965: 'Pre-IPO'},
 '交易金额': {12965: 360000},
 '单位': {12965: '万美元'},
 '交易股权': {12965: nan},
 '投后估值': {12965: 3760000.0},
 '单位.1': {12965: '万美元'},
 '交易时间': {12965: Timestamp('2021-07-12 00:00:00')},
 '交易状态': {12965: 

In [161]:
invest_vc_B[invest_vc_B['子行业投资次数']==174]

,index,企业简称,企业全称,业务描述,所在行业,所在地,企业简介,详细地址,联系电话,企业网址,...,本次交易股权比例_A,Cum_invset,本次交易股权比例_B,领投机构_1,领投机构,投资机构数量_A,投资金额_A,领投机构_2,投资次数,子行业投资次数
16168,128357,Hivello,Hivello Inc.,去中心化物理基础设施网络提供商,电子信息-信息化服务,荷兰,Hivello Inc.成立于2022年1月，主要业务是为人们解锁一种参与web3项目并赚取...,NaN,NaN,https://www.crunchbase.com/organization/hivello,...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan]",3.0,[],NaN,NaN,9.0,"[数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万]",None,3.0,174.0
16169,147044,Hivello,Hivello Inc.,去中心化物理基础设施网络提供商,电子信息-信息化服务,荷兰,Hivello Inc.成立于2022年1月，主要业务是为人们解锁一种参与web3项目并赚取...,NaN,NaN,https://www.crunchbase.com/organization/hivello,...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan]",3.0,[],NaN,NaN,9.0,"[数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万, 数十万]",None,3.0,174.0


In [158]:
invest_vc_B.loc[(invest_vc_B.企业简称=='Flipkart')&(invest_vc_B.投资机构=='老虎基金'),['投资方名称','交易时间']]

,投资方名称,交易时间
12905,老虎基金,2010-06-16
12906,老虎基金,2011-06-16
12910,Accel ;ICONIQ Capital ;老虎基金 ;Naspers,2012-08-24
12914,ICONIQ Capital ;老虎基金 ;Accel ;Naspers,2013-07-10
12919,摩根士丹利 ;Sofina ;瓦肯资本 ;老虎基金 ;DIG,2013-10-08
12923,数字天空科技 ;老虎基金 ;ICONIQ Capital ;Naspers,2014-05-26
12931,老虎基金 ;ICONIQ Capital ;摩根士丹利 ;Accel ;新加坡政府投资 ;S...,2014-07-29
12941,Steadview Capital ;老虎基金 ;新加坡政府投资 ;ICONIQ Capit...,2014-12-21
12943,Steadview Capital ;老虎基金,2015-07-28
12950,微软创投 ;深圳腾讯产业投资 ;老虎基金 ;Accel ;数字天空科技 ;eBay ;Nas...,2017-04-10


In [167]:
invest_vc_B.loc[(invest_vc_B.投资机构=='IDG资本')&(invest_vc_B.子行业=='信息化服务'),['投资方名称','交易时间','投资机构','企业简称','子行业投资次数']]

,投资方名称,交易时间,投资机构,企业简称,子行业投资次数
8595,GSV Ventures ;IDG资本 ;SVA ;Maveron,2014-11-11,IDG资本,Course Hero,90.0
14728,高榕创投 ;IDG资本 ;海通开元 ;玖富 ;神州数字,2017-07-01,IDG资本,Goopal,139.0
15952,IDG资本,2017-11-04,IDG资本,Hevo Data,144.0
16146,NGC Ventures ;Blockchain Founders Fund ;Side D...,2023-12-14,IDG资本,Hivello,170.0
16147,NGC Ventures ;Blockchain Founders Fund ;Side D...,2023-12-14,IDG资本,Hivello,170.0
...,...,...,...,...,...
270192,IDG资本 ;五源资本 ;创新工场 ;58同城,2014-08-16,IDG资本,273二手车,81.0
274386,IDG资本,2014-09-24,IDG资本,美味书签,84.0
275414,IDG资本,2006-05-31,IDG资本,球探体育,25.0
288226,IDG资本,2015-02-14,IDG资本,云景智维,97.0


In [175]:
invest_vc_B.to_csv('data/invest_vc_all_2412.csv')

1.处理投资方名称，将同一个名称的投资方进行合并，再计算对应的投资机构数量
2.对于投资机构数量为1的部分，直接用这家机构作为领投机构
3.对于投资机构数量大于1的部分，首先看本次交易股权比例，如果本次交易股权比例中有值，选择本次交易股权比例中占比最大的（大多数股权交易比例加总不为1），但暂时不考虑这个问题，只要有存在值的，就先根据那些有值的进行判断
---------------
统计到这一步为止能够处理的投融资事件数量和不能处理的投融资事件数量

In [121]:
max_c1=0
def get_vc_first(row):
    global max_c1
    vc_name=row['投资方名称_B']
    if len(vc_name)==0:
        return None

    if len(vc_name)==1:
        return vc_name[0]

    

    if len(row['本次交易股权比例_B']) > 0:
        #row['投资方名称_A'][row['本次交易股权比例_A'].index(max(row['本次交易股权比例_A']))]
        merged = {}
        for name, ratio in zip(row["投资方名称_A"], row["本次交易股权比例_A"]):
            if pd.isna(ratio):
                continue
                
            if name in merged:
                merged[name] += ratio
            else:
                merged[name] = ratio

        if merged:
            max_ratio = max(merged.values())
            max_names = [name for name, ratio in merged.items() if ratio == max_ratio] # max_name = max(merged, key=merged.get)
            
            if len(max_names)==1:
                return max_names[0]
            else:
                max_c1+=1
                print(max_names,merged,row.投资机构数量,row.投资机构基金数量)

    return None

    # filter_value=invest_vc[(invest_vc['企业全称']==row['企业全称']) & invest_vc['投资机构'].isin(vc_name)&(invest_vc['交易时间']<row['交易时间'])]
                        
    # if not filter_value.empty:
    #     filter_value1=filter_value[filter_value['投资方股权比例'].notna()].groupby("投资机构", as_index=False)["投资方股权比例"].sum()
    #     if not filter_value1.empty:
    #         # 按有股权比例的记录
    #         max_values= filter_value1['投资方股权比例'].max()# max_equity
    #         max_index = filter_value1[filter_value1['投资方股权比例'] == max_values]#.idxmax()
    #         # 如果最大股权比例唯一，返回对应投资机构
    #         if max_index['投资机构'].nunique() == 1:
    #             return max_index.iloc[0]['投资机构']
    #         # 如果最大股权相同...
        
        # # 按出现次数筛选
        # vc_counts =filter_value['投资机构'].value_counts()
        # max_values=vc_counts.iloc[0]
        # max_index = vc_counts[vc_counts.values == max_values].index
        # # 如果出现次数最多的唯一
        # if len(max_index)==1:
        #     return vc_counts.index[0]
        # #如果最大投资次数相同，按交易时间排序并选择最早的    
        # filter_value2=filter_value[filter_value['投资机构'].isin(max_index)].sort_values('交易时间')
        # first_row=filter_value2.iloc[0]
        # if (filter_value2['交易时间']==first_row['交易时间']).sum()==1:
        #     return first_row['投资机构']

    return None

In [122]:
invest_all['领投机构_2']=invest_all.apply(get_vc_first,axis=1)

['泰珑投资', '泰福谨通', '建发新兴'] {'启明创投': 2.12, '上海启明维创': 0.02, '泰珑投资': 2.14, '泰煜投资': 1.28, '瑞华创业投资': 1.28, '深圳天择资本管理有限公司': 1.28, '泰福谨通': 2.14, '建发新兴': 2.14} 8 6
['阿里巴巴', '富士康'] {'阿里巴巴': 20.0, '富士康': 20.0} 2 0
['加达控股', '飞道旅游科技'] {'加达控股': 50.0, '飞道旅游科技': 50.0} 2 0
['中科招商', '迪策鸿通投资管理', '聚益投资'] {'中科招商': 4.17, '广发信德': 2.5, '迪策鸿通投资管理': 4.17, '盈峰资本': 2.5, '聚益投资': 4.17, '泰豪晟大股权投资': 1.67} 6 4
['国新国控', '长江电力', '云南能投资本'] {'国新国控': 10.0, '长江电力': 10.0, '云南能投资本': 10.0} 3 1
['广西锦蓝', '达风投资'] {'广西锦蓝': 3.53, '达风投资': 3.53} 2 2
['中山证券', '万和证券', '财通证券', '国联证券'] {'中山证券': 0.64, '万和证券': 0.64, '财通证券': 0.64, '国联证券': 0.64, '财达证券': 0.5, '西部证券': 0.32} 6 0
['一牙网', '通策医疗'] {'一牙网': 38.75, '通策医疗': 38.75} 2 0
['容亿投资', '新芯资产'] {'容亿投资': 1.88, '明裕创投': 0.94, '冠亚投资': 0.28, '厦门冠亚创新投资': 0.09, '新芯资产': 1.88} 5 6
['欣博杰益', '沄泷投资'] {'欣博杰益': 2.69, '沄泷投资': 2.69} 2 2
['临港科创投', '明势私募'] {'临港科创投': 2.48, '明势私募': 2.48, '为瀛企业管理': 0.71} 3 2
['沄涌投资', '上海钟鼎'] {'沄涌投资': 6.25, '上海钟鼎': 6.25} 3 3
['浦科投资', '上海科投'] {'浦科投资': 20.0, '上海科投': 20.0} 2 0
['上海自贸区基金

In [177]:
max_c1,invest_all['领投机构_2'].count(),invest_all['领投机构_2'].isna().sum()

(1280, np.int64(87538), np.int64(67771))

In [180]:
mask=invest_all['领投机构_2'].isna()
invest_all.loc[~mask,'企业全称'].nunique(),invest_all.loc[mask,'企业全称'].nunique()

(60176, 42222)


------
统计到这一步为止能够处理的投融资事件数量和不能处理的投融资事件数量，公司数量，以及公司的平均融资次数和时间分布  

4.如果有披露部分的股权交易比例相同，则将这部分和未披露的部分等同处理，但只需要保留股权交易比例占比最大的几家VC即可，删除剩下的VC，根据每家投资机构在该公司历史投融资事件中的投资次数进行判断，  
1）首先看这几家VC是否有出现在该公司历史的投融资事件中，如果都有的话，看历史投融资事件中是否有之前做过领投的VC，如果有的话，则选择这家VC作为领投，如果历史上也没有出现过领投，则选择出现次数最多的VC为领投 （先看一下到这步为止能够判断出多少投融资事件以及公司）  
2）如果出现次数相同的话，则从VC层面进行判断，看每一家VC，到这个投融资事件发生之前，在被投资公司所处的这个行业历史投资次数，选择历史投资次数最多的VC为领投  


In [203]:
counts=[0,0,0,0,0]
def get_vc_first2(row):

    vc_name=row['投资方名称_B']
    if len(vc_name)==0:
        return None

    if len(vc_name)==1:
        #return vc_name[0]
        pass


    if len(row['本次交易股权比例_B']) > 0:
        #row['投资方名称_A'][row['本次交易股权比例_A'].index(max(row['本次交易股权比例_A']))]
        merged = {}
        for name, ratio in zip(row["投资方名称_A"], row["本次交易股权比例_A"]):
            if pd.isna(ratio):
                continue
                
            if name in merged:
                merged[name] += ratio
            else:
                merged[name] = ratio

        if merged:
            max_ratio = max(merged.values())
            max_names = [name for name, ratio in merged.items() if ratio == max_ratio] # max_name = max(merged, key=merged.get)
            
            if len(max_names)==1:
                #return max_names[0]
                pass
            else:
                counts[0]+=1
                vc_name=max_names

    filter_value=invest_vc_B[(invest_vc_B['企业全称']==row['企业全称']) & invest_vc_B['投资机构'].isin(vc_name)&(invest_vc_B['交易时间']<row['交易时间'])]
    if not filter_value.empty:
        fist_index=filter_value.loc[filter_value['领投机构_2'].isin(vc_name)]
        if fist_index['领投机构_2'].nunique()==1:
            counts[1]+=1
            return fist_index.iloc[0]['领投机构_2']
        else:
            pass

        max_values=filter_value['企业被投资次数'].max()
        max_count_index=filter_value.loc[filter_value['企业被投资次数'] == max_values]
        if max_count_index['投资机构'].nunique() == 1:
            counts[2]+=1
            return max_count_index.iloc[0]['投资机构']
        else:
            pass

    filter_value=invest_vc_B[(invest_vc_B['子行业']==row['子行业']) & invest_vc_B['投资机构'].isin(vc_name)&(invest_vc_B['交易时间']<row['交易时间'])]
    if not filter_value.empty:
        max_values=filter_value['子行业投资次数'].max()
        max_count_index=filter_value.loc[filter_value['子行业投资次数'] == max_values]
        if max_count_index['投资机构'].nunique() == 1:
            counts[3]+=1
            return max_count_index.iloc[0]['投资机构']
        else:
            pass 

    counts[4]+=1
    return None

In [204]:
%%time
mask=invest_all['领投机构_2'].isna()
invest_all.loc[mask,'领投机构_3']=invest_all[mask].apply(get_vc_first2,axis=1)
counts

CPU times: total: 1h 1min 8s
Wall time: 15h 33min 25s


[1280, 5102, 5252, 25315, 18783]

In [205]:
invest_all.loc[mask,'领投机构_3'].count(),invest_all.loc[mask,'领投机构_3'].isna().sum()

(np.int64(35669), np.int64(32102))

In [206]:
invest_all.shape[0]-invest_all['领投机构_2'].count()-invest_all['领投机构_3'].count()

np.int64(32102)

In [217]:
32102-13319

18783

In [225]:
invest_all.省.unique()

array([<NA>, '未披露', '英国', '北京', '四川', '台湾', '辽宁', '印度', '挪威', '美国', '广东',
       '开曼群岛', '加拿大', '韩国', '日本', '上海', '新加坡', '香港特别行政区', '巴西', '内蒙古自治区',
       '法国', '瑞士', '江苏', '匈牙利', '德国', '马来西亚', '丹麦', '意大利', '澳大利亚', '爱尔兰',
       '巴基斯坦', '以色列', '坦桑尼亚', '卢森堡', '瑞典', '海南', '印度尼西亚', '葡萄牙', '荷兰',
       '泰国', '沙特阿拉伯', '哈萨克斯坦共和国', '欧洲', '河南', '重庆', '西班牙', '阿根廷', '新西兰',
       '英属维尔京群岛', '马绍尔群岛共和国', '墨西哥', '塞舌尔', '爱沙尼亚共和国', '越南', '芬兰', '阿联酋',
       '菲律宾', '比利时', '约旦', '乌克兰', '俄罗斯', '福建', '波兰', '尼日利亚', '马耳他共和国',
       '土耳其', '孟加拉', '奥地利', '克罗地亚', '缅甸', '哥伦比亚', '肯尼亚', '河北', '南非', '埃及',
       '天津', '冰岛', '浙江', '黑龙江', '智利', '刚果', '蒙古', '湖北', '云南', '山东', '吉林',
       '西藏自治区', '江西', '安徽', '贵州', '陕西', '湖南', '广西壮族自治区', '山西', '百慕大群岛',
       '甘肃', '新疆维吾尔自治区', '老挝', '宁夏回族自治区', '青海', '埃塞俄比亚', '加纳', '毛里求斯',
       '捷克', '希腊', '瓦努阿图'], dtype=object)

In [230]:
invest_all.省.value_counts().head(100).index

Index(['北京', '广东', '上海', '江苏', '浙江', '美国', '四川', '湖北', '山东', '安徽', '福建', '陕西',
       '湖南', '天津', '河南', '重庆', '印度', '辽宁', '河北', '江西', '开曼群岛', '贵州', '香港特别行政区',
       '海南', '新疆维吾尔自治区', '黑龙江', '云南', '吉林', '新加坡', '未披露', '广西壮族自治区', '英国',
       '内蒙古自治区', '山西', '以色列', '台湾', '甘肃', '法国', '宁夏回族自治区', '西藏自治区', '加拿大',
       '印度尼西亚', '德国', '韩国', '青海', '日本', '澳大利亚', '瑞典', '瑞士', '百慕大群岛', '马来西亚',
       '泰国', '巴西', '西班牙', '越南', '阿联酋', '荷兰', '芬兰', '菲律宾', '丹麦', '俄罗斯', '意大利',
       '爱尔兰', '墨西哥', '英属维尔京群岛', '新西兰', '奥地利', '尼日利亚', '南非', '阿根廷', '哥伦比亚',
       '冰岛', '比利时', '波兰', '土耳其', '孟加拉', '塞舌尔', '巴基斯坦', '葡萄牙', '沙特阿拉伯', '挪威',
       '爱沙尼亚共和国', '卢森堡', '匈牙利', '老挝', '克罗地亚', '缅甸', '欧洲', '肯尼亚', '埃及', '坦桑尼亚',
       '哈萨克斯坦共和国', '加纳', '毛里求斯', '乌克兰', '马耳他共和国', '马绍尔群岛共和国', '约旦', '刚果',
       '蒙古'],
      dtype='object', name='省')

In [232]:
china_s=['北京', '广东', '上海', '江苏', '浙江', '四川', '湖北', '山东', '安徽', '福建', '陕西',
       '湖南', '天津', '河南', '重庆', '辽宁', '河北', '江西','贵州', '香港特别行政区',
       '海南', '新疆维吾尔自治区', '黑龙江', '云南', '吉林','广西壮族自治区','内蒙古自治区', 
       '山西','台湾', '甘肃', '宁夏回族自治区', '西藏自治区','青海' ]

In [235]:
invest_all['中国']=invest_all.省.isin(china_s).astype(int)
invest_all['中国'].sum()

np.int64(141560)

In [239]:
141560/invest_all.省.count()

np.float64(0.9314444758815362)

In [238]:
invest_all.loc[mask,'中国'].sum()/18783

np.float64(0.8992173774157483)

In [241]:
invest_all[mask].to_excel('data/invest_18783.xlsx')

In [249]:
invest_vc_B.loc[invest_vc_B.投资机构.str.len()<=3,'投资机构'].value_counts().head(20)

投资机构
深创投    1413
淡马锡     267
高盛      225
GV      158
软银      157
信中利     155
微创投     127
B站      126
SVA     115
KKR     112
IVP     102
GSR      87
西高投      86
成创投      82
苏河汇      81
飞马旅      75
CRV      72
深投控      68
比亚迪      64
学而思      61
Name: count, dtype: int64

In [277]:
vc_count=invest_vc_B.投资机构.value_counts()
(vc_count>1).sum(),(vc_count==1).sum(),invest_vc_B.投资机构.nunique()

(np.int64(26574), np.int64(42696), 69270)

In [284]:
vc_count[vc_count==1].index

Index(['天枢企业管理', '新蔡金财投资', '贝嘉豪置业', '集睿科技投资', '卓越投控', '齐齐哈尔一院', '科力友融创业投资',
       '米道投资', '力明医疗项目', '你好贷',
       ...
       '兖州区惠民城建投', '华宇集团', '山东三庆置业', '济南城建投资', '五矿发展', '香江金控', '奇鑫财富管理',
       'Clinton Global', 'Cachet Asset Management', 'INCE 220 LIMITED'],
      dtype='object', name='投资机构', length=42696)

In [273]:
invest_vc_B[invest_vc_B.投资机构=='远致投资']

,index,企业简称,企业全称,业务描述,所在行业,所在地,企业简介,详细地址,联系电话,企业网址,...,北上广深,本次交易股权比例_A,Cum_invset,本次交易股权比例_B,领投机构,投资机构数量_A,投资金额_A,领投机构_2,投资次数,子行业投资次数
41520,70551,万邑通,万邑通（上海）信息科技股份有限公司,跨境电商解决方案服务商,批发零售-电子商务,上海市-虹口区,万邑通（上海）信息科技有限公司成立于2012年11月21日，是一家跨境电商解决方案服务商，提...,上海市虹口区吴淞路575号吉汇大厦2303 ;,021-55668390 ; 400-996-0766,http://www.winit.com.cn,...,1,"[1.92, 1.6, 0.64, 1.6, 7.99]",5.0,"[1.92, 1.6, 0.64, 1.6, 7.99]",NaN,5.0,"[未披露, 未披露, 未披露, 未披露, 未披露]",Mega Element (Hong Kong) Limited,1.0,1.0
76350,62121,中储智运,中储智运科技股份有限公司,智慧物流服务商,物流运输-物流运输其他,江苏省-南京市,中储智运科技股份有限公司成立于2014年07月02日，注册地址为南京市鼓楼区北祥路65号，法...,南京市鼓楼区中央北路河路道一号 ;南京市鼓楼区河路道1号,;02586901989,NaN,...,0,"[0.46, 0.46, 6.88, 4.58, 2.09, 2.98]",4.0,"[0.46, 0.46, 6.88, 4.58, 2.09, 2.98]",NaN,6.0,"[2000.00, 2000.00, 30000.00, 20000.00, 9100.00...",诚通混改私募基金管理,1.0,1.0
94738,104384,五色科技,北京五色科技有限公司,NaN,医疗健康-医疗服务,北京市-房山区,北京五色科技有限公司成立于2015年06月30日，注册地址为北京市房山区良乡政通路19号1号...,北京市西城区月坛南街59号4层401,68521236,NaN,...,1,"[9.85, 7.23, 6.82, 4.43, 3.61, 2.27, 2.19, 2.71]",4.0,"[9.85, 7.23, 6.82, 4.43, 3.61, 2.27, 2.19, 2.71]",NaN,8.0,"[未披露, 未披露, 未披露, 未披露, 未披露, 未披露, 未披露, 未披露]",普耀九州基金,1.0,1.0
133254,14171,华泰保险,华泰保险集团股份有限公司,综合性金融保险集团,金融-保险,北京市-西城区,华泰保险集团股份有限公司成立于1996年8月29日，是一家集财险、寿险、资产管理、基金管理于...,北京市西城区金融大街35号国际企业大厦 ;北京市西城区金融大街35号国际企业大厦A座18层,010-59371328 ;010-59371888,NaN,...,1,"[nan, nan]",4.0,[],NaN,2.0,"[未披露, 未披露]",None,1.0,1.0
165478,131106,橙仕汽车,山东豪驰智能汽车有限公司,新能源汽车整车设计智造服务商,汽车交通-汽车整车,山东省-济南市,山东豪驰智能汽车有限公司旗下橙仕汽车是一家新能源汽车整车设计智造服务商，致力于为智慧城市协同...,山东省济南市莱芜区杨庄镇新材料产业园,400-799-1212,NaN,...,0,"[nan, nan, nan, nan]",3.0,[],NaN,4.0,"[数亿, 数亿, 数亿, 数亿]",None,1.0,1.0
165479,149797,橙仕汽车,山东豪驰智能汽车有限公司,新能源汽车整车设计智造服务商,汽车交通-汽车整车,山东省-济南市,山东豪驰智能汽车有限公司旗下橙仕汽车是一家新能源汽车整车设计智造服务商，致力于为智慧城市协同...,山东省济南市莱芜区杨庄镇新材料产业园,400-799-1212,NaN,...,0,"[nan, nan, nan, nan]",3.0,[],NaN,4.0,"[数亿, 数亿, 数亿, 数亿]",None,1.0,1.0
219882,15098,沃得农机,江苏沃得农业机械股份有限公司,农业全程机械制造商,传统制造-机械设备,江苏省-镇江市,江苏沃得农业机械股份有限公司成立于2003年1月29日，是一家集研发、生产、销售和服务于一体...,丹阳市丹北镇埤城西沟荡,0511-86348008,http://www.worldnyjx.com,...,0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2.0,[],NaN,20.0,"[近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近亿, 近...",None,1.0,1.0
269339,11277,矽电半导体,矽电半导体设备（深圳）股份有限公司,半导体检测设备研发商,电子信息-半导体,广东省-深圳市,矽电半导体设备（深圳）股份有限公司成立于2003年，是一家半导体检测设备研发商，专注于研发...,深圳市龙岗区龙城街道黄阁坑社区龙城工业园3号厂房三楼东区，五楼中西区\n\n ;深圳市龙岗区...,0755-28957758 ;0755-28957958,http://www.sidea.com.cn,...,1,"[nan, nan, nan, nan]",1.0,[],NaN,4.0,"[未披露, 未披露, 未披露, 未披露]",None,1.0,1.0
307737,17844,龙焱能源科技,龙焱能源科技（杭州）有限公司,太阳能薄膜电池生产商,能源及矿业-电池与储能技术,浙江省-杭州市,龙焱能源科技（杭州）有限公司成立于2008年5月4日，是一家太阳能薄膜电池生产商，专注于碲化...,浙江省杭州市经济技术开发区8号大街1号中策园4栋2层 ;浙江省杭州市钱塘新区白杨街道凌云待801号,+86-571-86875511-203 ;86906915,NaN,...,0,"[nan, nan]",3.0,[],NaN,2.0,"[未披露, 未披露]",None,1.0,1.0


In [265]:
invest_vc_B[invest_vc_B.投资机构=='浙能创投']

,index,企业简称,企业全称,业务描述,所在行业,所在地,企业简介,详细地址,联系电话,企业网址,...,北上广深,本次交易股权比例_A,Cum_invset,本次交易股权比例_B,领投机构,投资机构数量_A,投资金额_A,领投机构_2,投资次数,子行业投资次数
41776,139932,三维通信,三维通信股份有限公司,NaN,通信通讯-通信设备,浙江省-杭州市,三维通信股份有限公司（002115.SZ）成立于1993年5月，是一家移动通信网络建设供应商...,浙江省杭州市滨江区火炬大道581号,+86-571-88866999,NaN,...,0,[20.0],1.0,[20.0],浙能创投,1.0,NaN,浙能创投,1.0,1.0
307736,17844,龙焱能源科技,龙焱能源科技（杭州）有限公司,太阳能薄膜电池生产商,能源及矿业-电池与储能技术,浙江省-杭州市,龙焱能源科技（杭州）有限公司成立于2008年5月4日，是一家太阳能薄膜电池生产商，专注于碲化...,浙江省杭州市经济技术开发区8号大街1号中策园4栋2层 ;浙江省杭州市钱塘新区白杨街道凌云待801号,+86-571-86875511-203 ;86906915,NaN,...,0,"[nan, nan]",3.0,[],NaN,2.0,"[未披露, 未披露]",None,1.0,1.0


In [334]:
%%time
def get_vc_count_rate(row):
    vc_name=row['投资方名称_B']
    if len(vc_name)>1:
        filter_value=invest_vc_B[invest_vc_B['投资机构'].isin(vc_name)&(invest_vc_B['交易时间']<row['交易时间'])]
        if not filter_value.empty:
            vc_counts =filter_value.groupby('投资机构')['投资机构投资次数'].max().sort_values(ascending=False)#filter_value.投资机构.value_counts()
            if len(vc_counts) >= 2:
                return vc_counts.index[0],vc_counts.iloc[0]/ vc_counts.iloc[1]
    return None,None

invest_all['vc_count_max'],invest_all['vc_count_rate']=zip(*invest_all.apply(get_vc_count_rate,axis=1))

CPU times: total: 26min 27s
Wall time: 26min 53s


In [338]:
invest_all['vc_count_rate'].describe()

count    49372.000000
mean        11.447779
std         35.681758
min          1.000000
25%          1.666667
50%          3.071429
75%          8.000000
max       1096.000000
Name: vc_count_rate, dtype: float64

In [360]:
(invest_all['vc_count_rate']==1).sum()

np.int64(1351)

In [342]:
(invest_all['vc_count_rate']>=5).sum(),(invest_all['vc_count_rate']>=5).sum()/invest_all['vc_count_max'].count()#>=2

(np.int64(17943), np.float64(0.3634246131410516))

In [361]:
mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&(invest_all['投资机构数量']>0))
(invest_all.loc[mask,'vc_count_rate']==1).sum(),(invest_all.loc[mask,'vc_count_rate']>=3).sum(),(invest_all.loc[mask,'vc_count_rate']>=3).sum()/mask.sum()

(np.int64(620), np.int64(5056), np.float64(0.2691795772773252))

In [376]:
invest_all.loc[mask&(invest_all.loc[mask,'vc_count_rate']==1),['投资方名称','投资机构类型','投资金额',"本次交易股权比例",'投资方名称_B','交易时间','企业简称','子行业','行业','省','中国','vc_count_max','vc_count_rate']]

,投资方名称,投资机构类型,投资金额,本次交易股权比例,投资方名称_B,交易时间,企业简称,子行业,行业,省,中国,vc_count_max,vc_count_rate
128,Velocity Capital Fintech Ventures ;Leitmotiv P...,VC ;PE,数百万; 数百万,;,"[Leitmotiv Private Equity, Velocity Capital Fi...",2012-06-30,8 Securities,资本市场服务,金融,香港特别行政区,1,Leitmotiv Private Equity,1.0
285,富士康 ;Silver 8 Capital ;RRE Ventures ;Lerer Hip...,企业 ;VC、PE ;VC ;VC ;VC、早期投资机构 ;VC ;VC ;VC,近千万; 百万（级）; 百万（级）; 百万（级）; 百万（级）; 百万（级）; 百万（级）;...,; ; ; ; ; ; ;,"[Silver 8 Capital, 富士康, Jungle Ventures, RRE V...",2017-10-24,Abra,科技金融,金融,美国,0,RRE Ventures,1.0
406,Playground Global ;Sony Innovation Fund ;The R...,VC、PE ;PE ;VC,数百万; 数百万; 数百万,; ;,"[The Robotics Hub, Sony Innovation Fund, Playg...",2018-03-22,Agility Robotics,机器人,先进制造,<NA>,0,Playground Global,1.0
414,天士力资本 ;奥佳华 ;Brain Robotics Capital,VC、战略投资者 ;VC ;VC,未披露; 未披露; 未披露,; ;,"[Brain Robotics Capital, 奥佳华, 天士力资本]",2017-01-01,AiTreat,人工智能,电子信息,新加坡,0,天士力资本,1.0
422,Incubate ;网龙网络,VC ;企业,未披露; 未披露,;,"[网龙网络, Incubate]",2011-09-01,Aiming,动漫游戏,文娱传媒,日本,0,Incubate,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154228,首汽集团 ;首旅集团 ;首旅酒店,企业 ;企业 ;企业,数千万; 数千万; 未披露,; ;,"[首汽集团, 首旅集团, 首旅酒店]",2015-01-01,首汽租车,汽车交通其他,汽车交通,北京,1,首旅酒店,1.0
154253,首钢基金 ;国寿股权,VC、PE、战略投资者 ;VC、PE、战略投资者,未披露; 未披露,;,"[国寿股权, 首钢基金]",2017-07-10,首颐医疗,资本市场服务,金融,北京,1,国寿股权,1.0
155008,北京长安德瑞威 ;宝聿基金,PE ;VC、PE,未披露; 未披露,;,"[宝聿基金, 北京长安德瑞威]",2016-03-01,天宏药业,生物医药,医疗健康,黑龙江,1,北京长安德瑞威,1.0
155074,富德恒利 ;北京同泰盛源 ;科力创投 ;国开证券,VC、PE ;VC ;VC、PE ;企业,未披露; 未披露; 未披露; 未披露,; ; ;,"[北京同泰盛源, 科力创投, 富德恒利, 国开证券]",2017-08-14,立高科技,企业服务其他,企业服务,黑龙江,1,国开证券,1.0


In [382]:
invest_all['投资方名称_first']=invest_all.投资方名称_A.dropna().map(lambda x:x[0])

In [385]:
(invest_all['投资方名称_first']==invest_all['vc_count_max']).sum(),invest_all['vc_count_max'].count(),invest_all['投资方名称_first'].count()

(np.int64(21593), np.int64(49372), np.int64(141985))

In [388]:
mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&(invest_all['投资机构数量']>0))
(invest_all.loc[mask,'投资方名称_first']==invest_all.loc[mask,'vc_count_max']).sum(),mask.sum()

(np.int64(4812), np.int64(18783))

In [363]:
invest_all.loc[mask&(invest_all.loc[mask,'vc_count_rate']==1),'企业全称'].value_counts().head(20)

企业全称
Ambrx Inc.                     4
EndoGastric Solutions, Inc.    4
Yotpo，Inc.                     2
上海微宇天导技术有限责任公司                 2
北京小马智行科技有限公司                   2
上海慷梭电子科技有限公司                   2
云工工业科技（深圳）有限公司                 2
易宝支付有限公司                       2
Stan                           2
PANDORA MEDIA, INC.            2
山东益大新材料股份有限公司                  2
易登创新（北京）网络技术有限公司               2
杭州华安生物技术有限公司                   2
江苏亨睿碳纤维科技有限公司                  2
江苏科维仪表控制工程有限公司                 2
浙江赛思电子科技有限公司                   2
深圳市未蓝新能源科技有限公司                 2
矽电半导体设备（深圳）股份有限公司              2
花垣县十八洞紫霞湖美综合开发有限公司             2
长芯盛（武汉）科技股份有限公司                2
Name: count, dtype: int64

In [371]:
(mask&(invest_all['vc_count_rate']==1)&(invest_all.中国==0)).sum()

np.int64(93)

In [372]:
invest_all.loc[mask&(invest_all['vc_count_rate']==1),'交易时间'].dt.year.value_counts()

交易时间
2016    76
2015    74
2018    67
2017    65
2019    47
2020    45
2022    38
2023    29
2014    25
2021    22
2012    20
2024    20
2013    18
2011    17
2010    15
2007    11
2006    11
2008     7
2009     6
2005     2
2004     2
2001     2
2003     1
Name: count, dtype: int64

In [378]:
%%time
def get_vc_first3(row):

    vc_name=row['投资方名称_B']
    if len(vc_name)==0:
        return None

    if len(vc_name)==1:
        #return vc_name[0]
        pass

    if len(row['本次交易股权比例_B']) > 0:
        #row['投资方名称_A'][row['本次交易股权比例_A'].index(max(row['本次交易股权比例_A']))]
        merged = {}
        for name, ratio in zip(row["投资方名称_A"], row["本次交易股权比例_A"]):
            if pd.isna(ratio):
                continue
                
            if name in merged:
                merged[name] += ratio
            else:
                merged[name] = ratio

        if merged:
            max_ratio = max(merged.values())
            max_names = [name for name, ratio in merged.items() if ratio == max_ratio] # max_name = max(merged, key=merged.get)
            
            if len(max_names)==1:
                #return max_names[0]
                pass
            else:
                vc_name=max_names

    filter_value=invest_vc_B[invest_vc_B['投资机构'].isin(vc_name)&(invest_vc_B['交易时间']<row['交易时间'])]
    if not filter_value.empty:
        vc_counts =filter_value.投资机构.value_counts()#filter_value.groupby('投资机构')['机构投资次数'].max().sort_values(ascending=False)
        vc_one=vc_counts[vc_counts==1].index
        res=set(vc_name)-set(vc_one)
        if len(res)==1:
            return list(res)[0]
        # else:
        #     # max_index = vc_counts[vc_counts.values == vc_counts.iloc[0]].index
        #     if vc_counts.iloc[0] >= 3 * vc_counts.iloc[1]:
        #         counts[2]+=1
        #         return vc_counts.index[0]
                        
    return None

mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&(invest_all['投资机构数量']>0))
invest_all.loc[mask,'领投机构_4']=invest_all[mask].apply(get_vc_first3,axis=1)
invest_all.loc[mask,'领投机构_4'].count()

np.int64(2070)

In [389]:
mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&invest_all['领投机构_4'].isna()&(invest_all['投资机构数量']>0)&(invest_all['vc_count_rate']>=3))
invest_all.loc[mask,'领投机构_5']=invest_all.loc[mask,'vc_count_max']
invest_all.loc[mask,'领投机构_5'].count()

np.int64(4086)

In [390]:
mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&invest_all['领投机构_4'].isna()&invest_all['领投机构_5'].isna()&(invest_all['投资机构数量']>0))
invest_all.loc[mask,'领投机构_6']=invest_all.loc[mask,'投资方名称_first']
invest_all.loc[mask,'领投机构_6'].count()

np.int64(12625)

In [394]:
mask=(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&invest_all['领投机构_4'].isna()&invest_all['领投机构_5'].isna()&invest_all['领投机构_6'].isna()&(invest_all['投资机构数量']>0))
invest_all.loc[mask,['投资方名称','投资机构类型','投资金额','投资方名称_B','交易时间','企业简称','子行业','行业','省','中国',"投资方名称_A","本次交易股权比例_A",'vc_count_max','vc_count_rate']]

,投资方名称,投资机构类型,投资金额,投资方名称_B,交易时间,企业简称,子行业,行业,省,中国,投资方名称_A,本次交易股权比例_A,vc_count_max,vc_count_rate
87248,;银泰资本 ;招商和腾控股有限公司CMHJ Holdings Company Limited,;早期投资机构 ;企业,未披露; 未披露; 未披露,"[招商和腾控股有限公司CMHJ Holdings Company Limited, 银泰资本]",2008-02-01,广西开元机器,机械设备,传统制造,广西壮族自治区,1,"[<NA>, 银泰资本, 招商和腾控股有限公司CMHJ Holdings Company L...","[nan, nan, nan]",None,NaN
96036,;Norwest ;Focus Ventures,;VC ;VC,千万（级）; 数百万; 数百万,"[Norwest, Focus Ventures]",2008-09-23,PCH普诚华,高端装备制造,先进制造,广东,1,"[<NA>, Norwest, Focus Ventures]","[nan, nan, nan]",Norwest,1.5


In [402]:
('招商和腾控股有限公司'==invest_vc_B.投资机构).sum(),('银泰资本'== invest_vc_B.投资机构).sum(),('Focus Ventures'==invest_vc_B.投资机构).sum(),('Norwest'== invest_vc_B.投资机构).sum()

(np.int64(0), np.int64(74), np.int64(7), np.int64(94))

In [403]:
invest_all.loc[mask,'领投机构_6']=['银泰资本','Norwest']

In [272]:
invest_all.loc[invest_all.企业简称=='龙焱能源科技',['投资方名称','投资方名称_B','交易时间','企业简称','子行业','行业','省','中国',"本次交易股权比例_A",'领投机构_2','领投机构_3','领投机构_4']]

,投资方名称,投资方名称_B,交易时间,企业简称,子行业,行业,省,中国,本次交易股权比例_A,领投机构_2,领投机构_3,领投机构_4
155275,五源资本 ;君联资本,"[君联资本, 五源资本]",2008-05-04,龙焱能源科技,电池与储能技术,能源及矿业,浙江,1,"[nan, nan]",None,None,None
155276,君联资本 ;越超,"[越超, 君联资本]",2008-07-15,龙焱能源科技,电池与储能技术,能源及矿业,浙江,1,"[nan, nan]",None,君联资本,NaN
155277,远致投资 ;浙能创投,"[远致投资, 浙能创投]",2018-04-09,龙焱能源科技,电池与储能技术,能源及矿业,浙江,1,"[nan, nan]",None,None,None
155278,深圳资本集团,[深圳资本集团],2020-07-31,龙焱能源科技,电池与储能技术,能源及矿业,浙江,1,NaN,深圳资本集团,NaN,NaN
155279,光控浦益 ;浙江创新投 ;水木春锦资本 ;立元创投 ;博海泽华 ;东远投资管理,"[立元创投, 博海泽华, 东远投资管理, 光控浦益, 水木春锦资本, 浙江创新投]",2022-11-18,龙焱能源科技,电池与储能技术,能源及矿业,浙江,1,"[nan, nan, nan, nan, nan, nan]",None,水木春锦资本,NaN


In [275]:
invest_all.loc[mask,'投资方名称_B'].explode()

26        EQT Ventures
26            Skagerak
26              Nistad
26        Samsung NEXT
26           Sandwater
              ...     
155283          山东滨海投资
155303            青云创投
155303          芦荟私募基金
155307            弘博资本
155307            璟侑投资
Name: 投资方名称_B, Length: 46196, dtype: object

In [263]:
invest_all.loc[mask,"本次交易股权比例_B"].explode().count()

np.int64(2022)

In [281]:
invest_all.loc[mask,'省'].value_counts()

省
北京       3127
广东       2103
上海       2076
江苏       1421
浙江       1356
         ... 
埃及          1
爱尔兰         1
墨西哥         1
沙特阿拉伯       1
欧洲          1
Name: count, Length: 82, dtype: int64

In [279]:
invest_all.loc[mask,'行业'].value_counts()

行业
电子信息     2983
医疗健康     1521
先进制造     1124
文娱传媒     1069
金融       1004
传统制造      941
企业服务      935
消费        788
汽车交通      630
批发零售      581
能源及矿业     534
教育培训      474
化学工业      349
物流运输      271
通信通讯      237
建筑建材      220
综合        212
旅游        211
房地产       207
公用事业      191
体育        188
农林牧渔      185
Name: count, dtype: int64

In [404]:
invest_all['vc_first'] = (invest_all['领投机构_2'].combine_first(invest_all['领投机构_3']).combine_first(invest_all['领投机构_4']).combine_first(invest_all['领投机构_5']).combine_first(invest_all['领投机构_6']))
invest_all['vc_first'].isna().sum(),(invest_all['领投机构_2'].isna()&invest_all['领投机构_3'].isna()&invest_all['领投机构_4'].isna()&invest_all['领投机构_5'].isna()&invest_all['领投机构_6'].isna()).sum()

(np.int64(13319), np.int64(13319))

In [421]:
def find_source(row):
    for col in ['领投机构_2', '领投机构_3', '领投机构_4', '领投机构_5', '领投机构_6']:
        if pd.notna(row[col]):
            return col
    return None
invest_all['vc_source'] = invest_all.apply(find_source, axis=1).str[-1].fillna(0).astype(int)

In [422]:
(invest_all['投资机构数量']==0).sum(),invest_all['vc_source'].isna().sum()

(np.int64(13319), np.int64(13319))

In [407]:
invest_all.groupby('企业全称')['vc_first'].apply(lambda x:x.dropna().nunique()).sort_values()

企业全称
上海依行机器人科技有限公司        0
上海俏贝丽宠物用品有限公司        0
上海俏动智能化科技有限公司        0
上海信桦普企业征信服务有限公司      0
上海修真网络科技有限公司         0
                    ..
四川易冲科技有限公司          13
上海泽生科技开发股份有限公司      15
优客工场（北京）创业投资有限公司    16
Byju’s              17
北京小桔科技有限公司          17
Name: vc_first, Length: 83793, dtype: int64

In [408]:
invest_all[invest_all.企业全称=='上海泽生科技开发股份有限公司']

,index,企业简称,企业全称,业务描述,所在行业,所在地,企业简介,详细地址,联系电话,企业网址,...,领投机构_2,领投机构_3,中国,领投机构_4,vc_first,vc_count_max,vc_count_rate,投资方名称_first,领投机构_5,领投机构_6
22854,140726,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,上海创投,NaN,1,NaN,上海创投,None,NaN,上海创投,NaN,NaN
22855,140534,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,上海创投,NaN,1,NaN,上海创投,None,NaN,上海创投,NaN,NaN
22856,140085,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,上海创投,NaN,1,NaN,上海创投,None,NaN,上海创投,NaN,NaN
22857,153323,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,晨山投资,NaN,1,NaN,晨山投资,None,NaN,晨山投资,NaN,NaN
22858,151116,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,张江科投,NaN,1,NaN,张江科投,None,NaN,张江科投,NaN,NaN
22859,126452,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,张江科投,NaN,1,NaN,张江科投,浦科投资,16.000000,张江科投,NaN,NaN
22860,123145,泽生科技,上海泽生科技开发股份有限公司,NaN,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,金石投资,NaN,1,NaN,金石投资,金石投资,15.000000,金石投资,NaN,NaN
22861,52570,泽生科技,上海泽生科技开发股份有限公司,原创新药研发商,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,久有基金,NaN,1,NaN,久有基金,None,NaN,久有基金,NaN,NaN
22862,143340,泽生科技,上海泽生科技开发股份有限公司,原创新药研发商,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,东证创新,NaN,1,NaN,东证创新,东证创新,2.222222,东证创新,NaN,NaN
22863,12689,泽生科技,上海泽生科技开发股份有限公司,原创新药研发商,医疗健康-生物医药,上海市-浦东新区,上海泽生科技开发股份有限公司（871392）是一家原创新药研发商，产品研发管线专注于心血管、...,上海市浦东新区居里路68号 ;上海市浦东新区张江高科技园区居里路68号,021-50802627 ;+86-21-50802627,NaN,...,天地资本,NaN,1,NaN,天地资本,None,NaN,天地资本,NaN,NaN


In [411]:
invest_all['投资机构名称']=invest_all['投资方名称_B'].map(';'.join)

In [417]:
invest_all.drop(columns=['投资机构数量_A', '本次交易股权比例_A',  '投资金额_A', '投资方名称_B', '投资方名称_A', '投资方基金名称_A','本次交易股权比例_B','投资方基金名称_B', '领投机构'], inplace=True)

In [423]:
invest_all.drop(columns=['领投机构_2', '领投机构_3', '领投机构_4', '领投机构_5', '领投机构_6'], inplace=True)

In [429]:
del invest_all['政府引导基金的管理机构_机构全称']

In [442]:
invest_all.vc_source.value_counts()

vc_source
2    87538
3    35669
0    13319
6    12627
5     4086
4     2070
Name: count, dtype: int64

In [435]:
invest_all.columns

Index(['index', '企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话',
       '企业网址', '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址',
       '上市状态', '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1',
       '交易时间', '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式',
       '是否关联交易', '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型',
       '投资金额', '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称',
       '退出方基金名称', '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称',
       '中介类型', '政府引导基金的管理机构_机构简称', '行业', '子行业', 'ChinaVC_alias', 'VC_fullname',
       'GVC', 'GVC_DUMMY', 'GVC_Continuous', '投资机构基金数量', '投资机构数量', 'Age',
       '交易年月', '省', '市', '北上广深', 'Cum_invset', '领投机构_1', '中国', 'vc_first',
       'vc_count_max', 'vc_count_rate', '投资方名称_first', '投资机构名称', 'vc_source'],
      dtype='object')

In [443]:
invest_all[['ChinaVC_alias', 'VC_fullname', 'GVC', 'GVC_DUMMY', 'GVC_Continuous','政府引导基金的管理机构_机构简称','投资机构名称', 'vc_first', 'vc_source']]

,ChinaVC_alias,VC_fullname,GVC,GVC_DUMMY,GVC_Continuous,政府引导基金的管理机构_机构简称,投资机构名称,vc_first,vc_source
0,招银国际;GSR;青松基金;SNZ;纪源资本,招银国际金融有限公司;GSR Ventures;SNZ Holding;深圳市前海青松创业投...,1;0;0;0;0;0,1.0,0.166667,NaN,招银国际;HashKey Capital;GSR;青松基金;SNZ;纪源资本,GSR,3
1,NaN,NaN,NaN,NaN,NaN,NaN,Delphi Digital;Hack VC;Alliance VC;Animoca Bra...,OKX Ventures,3
2,NaN,NaN,NaN,NaN,NaN,NaN,Delphi Digital;Hack VC;Alliance VC;Animoca Bra...,OKX Ventures,3
3,梁信军,信宜市思贺镇梁信军化肥店,0,0.0,0.000000,NaN,Bonfire Union;ABCDE;梁信军;Bodl Ventures;Hash Global,梁信军,3
4,丹华资本;Amber;GSR;红点中国,DHVC;锐普投资咨询(上海)有限公司;GSR Ventures;AMBER CAPITAL...,0;0;0;0;0,0.0,0.000000,NaN,HashKey Capital;XIN Family;GSR;红点中国;丹华资本;Antal...,Hash Global,3
...,...,...,...,...,...,...,...,...,...
155304,老鹰基金,北京老鹰投资基金管理有限公司,0,0.0,0.000000,NaN,老鹰基金,老鹰基金,2
155305,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0
155306,NaN,NaN,NaN,NaN,NaN,NaN,True Ventures,True Ventures,2
155307,璟侑投资,杭州璟侑投资管理合伙企业(有限合伙),0,0.0,0.000000,NaN,弘博资本;璟侑投资,弘博资本,6


In [6]:
invest_all['政府引导基金参与']=invest_all['vc_first'].isin(invest_all['政府引导基金的管理机构_机构简称'].str.split(';').explode().dropna()).astype(int)
invest_all['政府引导基金参与'].sum()

np.int64(7111)

In [10]:
invest_all['ChinaVC']=invest_all['vc_first'].isin(invest_all['ChinaVC_alias'].str.split(';').explode().dropna()).astype(int)
invest_all['ChinaVC'].sum()

np.int64(107749)

In [472]:
ChinaVCtype['Alias'].nunique(),ChinaVCtype['Alias'].str.split(';').explode().nunique()

(40753, 42869)

In [8]:
ChinaVCtype['Alias']=ChinaVCtype['Alias'].str.split(';')
ChinaVCtype1=ChinaVCtype.explode('Alias').dropna(subset='Alias')
ChinaVCtype1.shape

(42894, 30)

In [9]:
ChinaVCtype1['Alias'].nunique(),ChinaVCtype1.duplicated().sum()

(42869, np.int64(0))

In [14]:
invest_all['ChinaVC']=invest_all['vc_first'].isin(ChinaVCtype1['Alias'].dropna().unique()).astype(int)
invest_all['ChinaVC'].sum()

np.int64(107533)

In [486]:
ChinaVCtype1[ChinaVCtype1[['Alias','GVC_excl']].duplicated()]

,VC_fullname,Unified_social_credit_ID,GVCagency_ne,GVCagency_equity,GVCsoe_ne,GVCsoe_equity,IVC_ne,CVC_ne,BVC_ne,FVC_ne,...,PenVC_excl,Unknown,Direct_judge,Corporate_parent,Parent_unified_social_credit_ID,Parent_industry_qcc,Parent_listed,Parent_listedmarket,Parent_stockcode,Alias
939,上海创源科技发展有限公司,91310114301430346U,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,创源
2882,上海汉韬投资管理有限公司,91310114779755318P,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,汉理资本
4466,上海道杰股权投资管理有限公司,91310000698807155M,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,道杰资本
4583,上海铂进投资管理有限公司,91310109342417201W,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,铂泉资本
6411,乌鲁木齐长石股权投资管理有限公司,916501000919425378,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,长石资本
8521,北京励石投资管理有限公司,91110108348382118N,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,励石投资
10161,北京松源资本管理有限公司,91110108306379922Q,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,松源资本
13715,和君集团有限公司,91110000575156081L,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,和君资本
15411,天津真格天地投资管理有限公司,911201163286869745,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,真格基金
30802,点亮投资管理（上海）有限公司,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN,点亮资本


In [488]:
invest_all.shape

(155309, 83)

In [56]:
GVC_excl_Alias=ChinaVCtype1.loc[ChinaVCtype1['GVC_excl']==1,'Alias'].unique()
len(GVC_excl_Alias)

4360

In [57]:
invest_all['first_GVC']=invest_all['vc_first'].isin(GVC_excl_Alias).astype(int)
invest_all['first_GVC'].sum()

np.int64(15482)

In [60]:
invest_all.to_excel('data/invest_all_2412.xlsx')

In [236]:
invest_all=pd.read_excel('data/invest_all_2412.xlsx',index_col=0)
invest_all.shape

(155309, 83)

In [4]:
invest_all.rename(columns={'投资机构基金数量':'投资方基金数量','投资机构名称':'投资方名称_unique'},inplace=True)

In [16]:
invest_all['vc_first'].nunique(),invest_all['vc_first'].nunique()/invest_all['投资方名称_unique'].str.split(';').explode().dropna().nunique()

(31359, 0.4527067994802945)

In [ ]:
def get_vc_first3(row):

    vc_name=row['投资方名称_B']
    if len(vc_name)==0:
        return None

    if len(vc_name)==1:
        #return vc_name[0]
        return None

    # if len(row['本次交易股权比例_B']) > 0:
    #     #row['投资方名称_A'][row['本次交易股权比例_A'].index(max(row['本次交易股权比例_A']))]
    #     merged = {}
    #     for name, ratio in zip(row["投资方名称_A"], row["本次交易股权比例_A"]):
    #         if pd.isna(ratio):
    #             continue
                
    #         if name in merged:
    #             merged[name] += ratio
    #         else:
    #             merged[name] = ratio

    #     if merged:
    #         max_ratio = max(merged.values())
    #         max_names = [name for name, ratio in merged.items() if ratio == max_ratio] # max_name = max(merged, key=merged.get)
            
    #         if len(max_names)==1:
    #             #return max_names[0]
    #             pass
    #         else:
    #             vc_name=max_names

    filter_value=invest_vc_B[invest_vc_B['投资机构'].isin(vc_name)&(invest_vc_B['交易时间']<row['交易时间'])]
    if not filter_value.empty:
        vc_counts =filter_value.投资机构.value_counts()#filter_value.groupby('投资机构')['投资机构投资次数'].max().sort_values(ascending=False)
        vc_one=vc_counts[vc_counts==1].index
        res=set(vc_name)-set(vc_one)
        if len(res)==1:
            return list(res)[0]
        # else:
        #     # max_index = vc_counts[vc_counts.values == vc_counts.iloc[0]].index
        #     if vc_counts.iloc[0] >= 3 * vc_counts.iloc[1]:
        #         counts[2]+=1
        #         return vc_counts.index[0]
                        
    return None

invest_all['领投机构_4']=invest_all.apply(get_vc_first3,axis=1)

加权要不按照这个领投机构_4：50；领投机构_5：30、领投机构_6：20

In [30]:
invest_all['领投机构_4'].count()

np.int64(4268)

In [37]:
Mask=(~invest_all['vc_source'].isin([0,2,3]))
invest_all.loc[Mask,'领投机构_4'].count(),Mask.sum()

(np.int64(2030), np.int64(18783))

In [36]:
mask=(Mask&(invest_all['vc_count_rate']>=3))
invest_all.loc[mask,'领投机构_5']=invest_all.loc[mask,'vc_count_max']
invest_all.loc[mask,'领投机构_5'].count(),invest_all.loc[(Mask&(invest_all['vc_count_rate']>=2)),'vc_count_max'].count()

(np.int64(5056), np.int64(6922))

In [34]:
invest_all.loc[Mask,'投资方名称_first'].count()

np.int64(18781)

In [41]:
weights = {
    "领投机构_4": 50,
    "领投机构_5": 30,
    "投资方名称_first": 20,
}

def calculate_match(row):
    values = row[["领投机构_4", "领投机构_5", "投资方名称_first"]]
    unique_values = values.dropna().unique()
    
    if len(unique_values) == 1:    # 如果有唯一的值
        return unique_values[0]
    
    # 如果有多个不同值，根据权重计算
    scores = {}
    for col, weight in weights.items():
        if pd.notna(row[col]):
            scores[row[col]] = scores.get(row[col], 0) + weight
    
    if scores:
        return max(scores, key=scores.get)
    return np.nan 
    
invest_all.loc[Mask,'领投机构_match']=invest_all[Mask].apply(calculate_match, axis=1)
invest_all.loc[Mask,'领投机构_match'].count()

np.int64(18781)

In [43]:
(invest_all.loc[Mask,'领投机构_match']!=invest_all.loc[Mask,'vc_first']).sum()

np.int64(28)

In [51]:
invest_all.loc[Mask&(invest_all.loc[Mask,'领投机构_match']!=invest_all.loc[Mask,'vc_first']),['投资方名称','投资方名称_B','本次交易股权比例','交易时间','企业简称','子行业','行业','省','中国',"vc_first",'领投机构_match','领投机构_4','领投机构_5','投资方名称_first','vc_source']]

,投资方名称,投资方名称_B,本次交易股权比例,交易时间,企业简称,子行业,行业,省,中国,vc_first,领投机构_match,领投机构_4,领投机构_5,投资方名称_first,vc_source
25427,大众资本 ;诚柏投资 ;联新资本 ;深圳天正投资,"[大众资本, 联新资本, 深圳天正投资, 诚柏投资]",5.02% ;1.67% ;2.09% ;5.02%,2010-12-27,第一财经,传媒出版,文娱传媒,上海,1,深圳天正投资,联新资本,None,联新资本,大众资本,4
29714,平安德成 ;上海招银投资 ;中钰贤齐 ;东方富海 ;云晖资本,"[中钰贤齐, 平安德成, 上海招银投资, 云晖资本, 东方富海]",4.84% ;4.84% ;0.60% ;1.37% ;0.46%,2016-01-26,韵达快递,仓储物流,物流运输,上海,1,上海招银投资,东方富海,None,东方富海,平安德成,4
33286,太原高新投资 ;山证投资 ;国泰创投,"[国泰创投, 太原高新投资, 山证投资]",1.82% ;1.82% ;0.91%,2016-02-24,中绿环保,公用事业其他,公用事业,山西,1,山证投资,国泰创投,None,国泰创投,太原高新投资,4
33760,银华资本 ;建银国际资本 ;天星资本 ;景涵财务 ;信业宏益投资 ;惠博投资 ;天任投资 ;...,"[景涵财务, 天任投资, 湖州昂业, 河北聚焦网络, 虢盛资本, 信於资产, 武汉诚盛电子,...",0.62% ;0.42% ;0.29% ;0.54% ;1.63% ;1.63% ;1.57...,2015-09-25,中际联合,机械设备,传统制造,北京,1,信业宏益投资,天星资本,None,天星资本,银华资本,4
35223,鼎泰海富 ;慧德融合产业基金 ;武汉苹湖股权投资 ;金控浙商资本管理,"[金控浙商资本管理, 鼎泰海富, 武汉苹湖股权投资, 慧德融合产业基金]",1.91% ;5.75% ;3.83% ;5.75%,2020-12-25,赤诚生物,化学工业其他,化学工业,湖北,1,金控浙商资本管理,鼎泰海富,None,鼎泰海富,鼎泰海富,4
35663,VNTR IX HK Holdings Limited ;合肥科讯创投 ;MAUT IV H...,"[MAUT IV HK Holdings, 合肥科讯创投, VNTR IX HK Holdi...",4.43% ;2.63% ;4.43%,2022-08-09,仁洁智能,机器人,先进制造,安徽,1,VNTR IX HK Holdings Limited,合肥科讯创投,None,合肥科讯创投,VNTR IX HK Holdings Limited,4
36901,充澄投资 ;知点投资 ;嘉鸿投资管理 ;同安投资,"[充澄投资, 知点投资, 同安投资, 嘉鸿投资管理]",3.33% ;3.33% ;1.67% ;0.44%,2016-02-04,傲基电商,电子商务,批发零售,广东,1,知点投资,同安投资,None,同安投资,充澄投资,4
37028,Coyote Investment ;Bio Success ;Hillhouse COFL...,"[TPG ASIA VII SF PTE.LTD., 奥博资本, Hillhouse COF...",1.06% ;1.06% ;7.44% ;4.04% ;1.10% ;1.60% ;1.50...,2020-11-17,兆科眼科,生物医药,医疗健康,开曼群岛,0,Hillhouse COFL,奥博资本,None,奥博资本,Coyote Investment,4
55008,中投建华投资 ;静远投资 ;同创伟业,"[中投建华投资, 同创伟业, 静远投资]",4.50% ;4.50% ;,2013-03-19,盛磁科技,传统制造其他,传统制造,北京,1,中投建华投资,同创伟业,None,同创伟业,中投建华投资,4
61777,恒健创投 ;国富基金 ;高林资本 ;赣州招商致远壹号投资 ;中文创投 ;中广文影基金 ;微资...,"[新华网, 微资方来, 赣州招商致远壹号投资, 东源汇信, 国富基金, 中国文化产业投资基金...",0.45% ;0.45% ;0.34% ;1.36% ;1.13% ;1.01% ;0.70...,2016-03-13,华强方特,休闲娱乐,文娱传媒,广东,1,池州中安招商投资,恒健创投,None,NaN,恒健创投,4


In [17]:
invest_all.columns

Index(['index', '企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话',
       '企业网址', '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址',
       '上市状态', '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1',
       '交易时间', '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式',
       '是否关联交易', '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型',
       '投资金额', '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称',
       '退出方基金名称', '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称',
       '中介类型', '政府引导基金的管理机构_机构简称', '行业', '子行业', 'ChinaVC_alias', 'VC_fullname',
       'GVC', 'GVC_DUMMY', 'GVC_Continuous', '投资方基金数量', '投资机构数量', 'Age',
       '交易年月', '省', '市', '北上广深', 'Cum_invset', '领投机构_1', '中国', 'vc_first',
       'vc_count_max', 'vc_count_rate', '投资方名称_first', '投资方名称_unique',
       'vc_source', '政府引导基金参与', 'ChinaVC'],
      dtype='object')

In [6]:
from collections import defaultdict,Counter
import itertools
import igraph as ig

In [51]:
filter_df_invest.columns

Index(['序号', '标题 (中文)', '摘要 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日',
       '专利类型', '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量',
       '简单同族', '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人',
       '被引证申请人', '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人',
       '工商统一社会信用代码', 'table_name', '企业全称'],
      dtype='object')

In [38]:
filter_df_invest['申请日']

0          1993-10-11 00:00:00
1          1994-01-27 00:00:00
2          1994-03-04 00:00:00
3          1994-04-14 00:00:00
4          1994-04-29 00:00:00
                  ...         
3261600    2020-12-31 00:00:00
3261601    2020-12-31 00:00:00
3261602    2020-12-31 00:00:00
3261603    2020-12-31 00:00:00
3261604    2020-12-31 00:00:00
Name: 申请日, Length: 3261605, dtype: object

In [67]:
filter_df_invest['申请年']=filter_df_invest['申请日'].dt.year
filter_df_invest['企业名称']=filter_df_invest['企业全称'].str.split(';').map(lambda x: list(map(str.strip, x)))
(filter_df_invest['企业名称'].map(len)==0).sum()

np.int64(0)

In [73]:
# filter_df_invest['企业名称']=filter_df_invest['企业全称'].str.strip(';')
patent_invest_1=filter_df_invest.loc[(filter_df_invest['申请年']<=2020)&(filter_df_invest['企业名称'].map(len)>0),
['申请日', '申请人','公开（公告）日','申请号','公开（公告）号','专利类型','引证申请人', '被引证申请人','申请年','企业名称']].sort_values(
    ['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号','专利类型'],keep='first').reset_index(drop=True)
patent_invest_1.shape

(1836363, 10)

In [58]:
[(pair[0], pair[1]) for pair in itertools.combinations([1,2], 2)]

[(1, 2)]

In [74]:
%%time
snapshots = {}
yearly_centrality = []
for year, group in patent_invest_1.groupby('申请年'):
    if year<2005:
        continue
    g = ig.Graph() 
    applicants=group['企业名称'].explode().dropna().unique()#'企业名称'
    g.add_vertices(applicants.tolist())

    added_edges = set() 
    for _, row in group.iterrows():
        application_number = row['申请号']
        edges = [(pair[0], pair[1]) for pair in itertools.combinations(row['企业名称'], 2)]
        for edge in edges:
            if (application_number,edge) not in added_edges: #如果是不同的申请后的话 添加边 同一个申请号就去重
                g.add_edge(edge[0], edge[1]) #无向边
                added_edges.add((application_number,edge))
            
    snapshots[year] = g.copy()

    centralities = {
    'degree_centrality':  g.degree(),
    'betweenness_centrality': g.betweenness(),
    'closeness_centrality': g.closeness(),
    'eigenvector_centrality':  g.eigenvector_centrality()
    }
    
    df = pd.DataFrame(centralities)
    df['year'] = year
    df['company'] = applicants  
    yearly_centrality.append(df)

    print(year,len(added_edges), f'igraph Nodes: {g.vcount()}, Edges: {g.ecount()}')
    
centrality_df = pd.concat(yearly_centrality, ignore_index=True)
centrality_df

2005 2 igraph Nodes: 444, Edges: 2
2006 4 igraph Nodes: 629, Edges: 4
2007 52 igraph Nodes: 864, Edges: 52
2008 74 igraph Nodes: 1229, Edges: 74
2009 91 igraph Nodes: 1712, Edges: 91
2010 186 igraph Nodes: 2226, Edges: 186
2011 356 igraph Nodes: 3008, Edges: 356
2012 770 igraph Nodes: 3761, Edges: 770
2013 1077 igraph Nodes: 4399, Edges: 1077
2014 1725 igraph Nodes: 5299, Edges: 1725
2015 2013 igraph Nodes: 7051, Edges: 2013
2016 2283 igraph Nodes: 9218, Edges: 2283
2017 2643 igraph Nodes: 10704, Edges: 2643
2018 3038 igraph Nodes: 11806, Edges: 3038
2019 3816 igraph Nodes: 12743, Edges: 3816
2020 4726 igraph Nodes: 13808, Edges: 4726
CPU times: total: 42.3 s
Wall time: 42.9 s


,degree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,year,company
0,0,0.0,NaN,0.000000e+00,2005,中兴通讯股份有限公司
1,0,0.0,NaN,0.000000e+00,2005,深圳迈瑞生物医疗电子股份有限公司
2,0,0.0,NaN,0.000000e+00,2005,上海复旦张江生物医药股份有限公司
3,0,0.0,NaN,0.000000e+00,2005,威海广泰空港设备股份有限公司
4,0,0.0,NaN,0.000000e+00,2005,深圳市力合材料有限公司
...,...,...,...,...,...,...
88896,0,0.0,NaN,6.950132e-18,2020,武汉滴水智能科技有限公司
88897,1,0.0,1.0,6.948119e-16,2020,优品汽车服务集团有限公司
88898,0,0.0,NaN,6.950132e-18,2020,广州晶优电子科技有限公司
88899,0,0.0,NaN,6.950132e-18,2020,科来网络技术股份有限公司


In [76]:
centrality_df.to_csv('data//patent_invest_all_applicant_centrality_2020.csv',  encoding="utf_8_sig")

In [79]:
patent_invest_companie=pd.read_csv('data\\融资与专利2020b_202411.csv', usecols=['观察年','企业全称']).drop_duplicates()
invest_companies=set(patent_invest_companie['企业全称'])
len(invest_companies)

20704

In [88]:
patent_invest_1['企业名称'].explode().nunique()

23890

In [89]:
invest_companies-set(patent_invest_1['企业名称'].explode())

set()

In [90]:
invest_companies=set(patent_invest_1['企业名称'].explode())

In [91]:
%%time
#patent_invest_1['applicants'] = patent_invest_1['企业名称'].apply(lambda x:[i for i in x if clean_short_name(clean_fullname(i)) in tax_gx_companies])
patent_invest_1['cited_applicants'] =  patent_invest_1['被引证申请人'].str.split(';').apply(lambda x:   [i.strip() for i in x if i.strip() and i.strip() in invest_companies] if isinstance(x, list) else []) # pd.isna(x)
patent_invest_1['citing_applicants'] =   patent_invest_1['引证申请人'].str.split(';').apply(lambda x:  [i.strip() for i in x if i.strip() and i.strip() in invest_companies] if isinstance(x, list) else []) 

CPU times: total: 8.17 s
Wall time: 8.32 s


In [93]:
patent_invest_1['cited_applicants'].explode().nunique(),patent_invest_1['citing_applicants'].explode().nunique()

(19453, 17912)

In [95]:
import numpy as np

In [97]:
%%time
snapshots = {}
yearly_centrality = []
G = ig.Graph(directed=True)
for year, group in patent_invest_1.groupby('申请年'):
    g = ig.Graph(directed=True) 
    all_applicants=pd.concat([group['企业名称'].explode().dropna(),group['cited_applicants'].explode().dropna(),group['citing_applicants'].explode().dropna()]).value_counts()
    g.add_vertices(list(all_applicants.index))
    
    frequency = max(1, len(group) // 100)    
    all_edges = []
    for index, row in group.iterrows():
        edges = set()
        for applicant in row['企业名称']:
            # 申请人 -> 被引证申请人
            edges.update((applicant, cited) for cited in row['cited_applicants']) #frozenset(
            # 引证申请人 -> 申请人
            edges.update((citing, applicant) for citing in row['citing_applicants'])
                
        all_edges += list(edges)

        if index % frequency == 0:
            print('.', end='', flush=True)

   
    if not all_edges:
        continue
#-------------------------------------
    edge_weights = Counter(all_edges)
    edges, weights = zip(*edge_weights.items())# for (source, target), weight in edge_weights.items(): #edge, weight 
    g.add_edges(edges) 
    g.es['weight'] = weights

    snapshots[year] = g.copy()
#---------------------------------------
    new_vertices = set(g.vs['name']) - set(G.vs['name'] if G.vcount() > 0 else [])
    G.add_vertices(list(new_vertices))
    # vertex_index = {v['name']: v.index for v in G.vs}
    for (src_idx, dst_idx), weight in zip(iter(g.get_edgelist()), iter(g.es["weight"])):# for edge in g.es:
        src = g.vs[src_idx]["name"] #edge.source
        dst = g.vs[dst_idx]["name"] 

        if G.are_adjacent(src, dst):
            existing_edge = G.es.select(_source=G.vs.find(name=src).index, _target=G.vs.find(name=dst).index)[0] # vertex_index[src], vertex_index[dst]
            existing_edge['weight'] += weight    # g.es['weight'] 如果边已经存在，则仅更新其权重
        else:
            G.add_edge(src, dst, weight=weight)   # 如果边不存在，则添加边并设置权重

    print('\n',year,f'unique Edges: {len(edge_weights)} total Edges: {sum(edge_weights.values())}',
          f'data: {len(group),len(all_applicants),len(all_edges)}', 
          f'igraph Nodes: {g.vcount()}, Edges: {g.ecount()}',f', Graph Nodes: {G.vcount()}, Edges: {G.ecount()}',end=' ')
#---------------------------------------    
    centralities = {
    'degree_centrality': G.degree(mode="all"),
    'log_degree_centrality': [np.log(out_deg + in_deg + 1) for out_deg, in_deg in zip(G.outdegree(), G.indegree())],#lnDC
    'pagerank_scores' : G.pagerank(),
    'betweenness_centrality': G.betweenness(weights=G.es['weight']),
    'closeness_centrality': G.closeness(weights=G.es['weight']),
    # 'katz_centrality':  katz_centrality(G.get_adjacency(attribute="weight").data,alpha=0.1, beta=1),    # 获取图的邻接矩阵
    }
    if not G.is_dag(): #是否为有向无环图,对于DAG，特征向量中心性通常为零，因为这种类型的图结构没有回路，而特征向量中心性依赖于节点相互影响的循环结构，因此会导致所有的特征值为零。
        centralities['eigenvector_centrality'] = G.eigenvector_centrality(weights=G.es['weight'])
    # else:
    #     g.to_undirected() 
#---------------------------------------    
    df = pd.DataFrame(centralities)
    df['year'] = year
    df['company'] = G.vs['name']
    df['count']= all_applicants.reindex(G.vs['name']).values #.fillna(0).astype(int) all_applicants.loc[g.vs['name']]
    yearly_centrality.append(df)

    print(f'DF: {df.shape}')

.
 1987 unique Edges: 2 total Edges: 2 data: (1, 2, 2) igraph Nodes: 2, Edges: 2 , Graph Nodes: 2, Edges: 2 DF: (2, 9)
...
 1989 unique Edges: 3 total Edges: 3 data: (3, 3, 3) igraph Nodes: 3, Edges: 3 , Graph Nodes: 3, Edges: 3 DF: (3, 9)
...
 1991 unique Edges: 3 total Edges: 3 data: (2, 3, 3) igraph Nodes: 3, Edges: 3 , Graph Nodes: 5, Edges: 5 DF: (5, 9)
............................
 1994 unique Edges: 5 total Edges: 5 data: (25, 11, 5) igraph Nodes: 11, Edges: 5 , Graph Nodes: 14, Edges: 8 DF: (14, 9)
.................
 1995 unique Edges: 5 total Edges: 5 data: (17, 11, 5) igraph Nodes: 11, Edges: 5 , Graph Nodes: 20, Edges: 11 DF: (20, 9)
.............................................
 1996 unique Edges: 2 total Edges: 3 data: (45, 7, 3) igraph Nodes: 7, Edges: 2 , Graph Nodes: 23, Edges: 11 DF: (23, 9)
.............................................................................
 1997 unique Edges: 10 total Edges: 10 data: (77, 18, 10) igraph Nodes: 18, Edges: 10 , Graph Nodes: 3

In [98]:
yearly_centrality_df = pd.concat(yearly_centrality, ignore_index=True)
yearly_centrality_df.to_csv('data//patent_invest_all_applicant_cite_directed_centrality_2020.csv',  encoding="utf_8_sig")
yearly_centrality_df

,degree_centrality,log_degree_centrality,pagerank_scores,betweenness_centrality,closeness_centrality,eigenvector_centrality,year,company,count
0,3,1.386294,0.500000,0.0,1.000000,1.0,1987,中国科学院微生物研究所,2.0
1,1,0.693147,0.500000,0.0,1.000000,1.0,1987,中国石油化工股份有限公司,1.0
2,4,1.609438,0.333333,0.0,0.666667,1.0,1989,中国科学院微生物研究所,5.0
3,1,0.693147,0.333333,0.0,0.400000,1.0,1989,中国石油化工股份有限公司,1.0
4,1,0.693147,0.333333,0.0,0.500000,0.5,1989,北京恩泽康泰生物科技有限公司,1.0
...,...,...,...,...,...,...,...,...,...
222537,3,1.386294,0.000015,0.0,0.241149,0.0,2020,苏州市协诚微波技术有限公司,9.0
222538,0,0.000000,0.000008,0.0,NaN,0.0,2020,江苏先科半导体新材料有限公司,1.0
222539,0,0.000000,0.000008,0.0,NaN,0.0,2020,上海启见科技信息技术有限公司,1.0
222540,2,1.098612,0.000056,0.0,NaN,0.0,2020,北京途灵微电子科技有限公司,6.0


In [99]:
import pickle
with open("data//patent_invest_all_applicant_cite_directed_2020_graph_snapshots.pkl", "wb") as f:
    pickle.dump(snapshots, f)

G.write_pickle("data//patent_invest_all_applicant_cite_directed_2020_graph.pickle")

In [35]:
unmatched_sample_listed=pd.read_csv('data/unmatched_sample_listed_1215.csv',index_col=0)
unmatched_sample_listed.columns

Index(['Unnamed: 0', '投资方名称', '投资机构类型_x', '投资方全称', '投资方省份', 'VC_fullname',
       'Alias', 'Unified_social_credit_ID', '投资方类型', '投资方所在地', '投资方详细地址',
       '投资方法人代表', 'GPLP类型', '投资方区县', '投资方城市', '投资方统一社会信用代码', '投资方经营范围',
       '投资方行业', '匹配数', '交易时间', '投资方名称_1', '投资方名称_2', 'Symbol', 'ShortName',
       'SecurityID', 'PROVINCECODE', 'PROVINCE', 'CITYCODE', 'RegisterAddress',
       'CITY', 'IndustryName', 'IndustryCode', 'IndustryNameC',
       'IndustryCodeC', 'IndustryNameD', 'IndustryCodeD', 'SocialCreditCode'],
      dtype='object')

In [37]:
unmatched_sample_listed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21906 entries, 0 to 21905
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                21906 non-null  int64  
 1   投资方名称                     21906 non-null  object 
 2   投资机构类型_x                  21846 non-null  object 
 3   投资方全称                     1024 non-null   object 
 4   投资方省份                     16 non-null     object 
 5   VC_fullname               1474 non-null   object 
 6   Alias                     1474 non-null   object 
 7   Unified_social_credit_ID  71 non-null     object 
 8   投资方类型                     1024 non-null   object 
 9   投资方所在地                    1020 non-null   object 
 10  投资方详细地址                   120 non-null    object 
 11  投资方法人代表                   16 non-null     object 
 12  GPLP类型                    1024 non-null   object 
 13  投资方区县                     0 non-null      float64
 14  投资方城市      

In [39]:
unmatched_sample_listed.投资机构类型_x.unique()

array(['企业', 'VC、PE', 'VC', 'PE', '战略投资者', 'VC、早期投资机构', '早期投资机构', '人物',
       'VC、PE、早期投资机构', 'VC、战略投资者', '孵化器', 'VC、孵化器', nan, '券商直投',
       'VC、PE、战略投资者', 'PE、战略投资者', 'VC、PE、券商直投', 'VC、PE、孵化器', 'VC、券商直投',
       'VC、早期投资机构、孵化器', '早期投资机构、孵化器', 'PE、券商直投'], dtype=object)

In [41]:
unmatched_sample_listed.投资方名称.nunique()

21906

In [42]:
unmatched_sample_listed.投资方名称

0           *ST京蓝
1           *ST保千
2           *ST升达
3           *ST宜康
4           *ST宜生
           ...   
21901    科力友融创业投资
21902      星辰信通投资
21903      广州碧蓝环保
21904      阿基米德金融
21905      芦荟私募基金
Name: 投资方名称, Length: 21906, dtype: object

In [46]:
unmatched_sample_listed.投资方名称_1

0            ST京蓝
1            ST保千
2            ST升达
3            ST宜康
4            ST宜生
           ...   
21901    科力友融创业投资
21902      星辰信通投资
21903      广州碧蓝环保
21904      阿基米德金融
21905      芦荟私募基金
Name: 投资方名称_1, Length: 21906, dtype: object

In [47]:
unmatched_sample_listed.交易时间

0        2002-08-23
1        2016-12-14
2        2016-02-03
3        2018-07-11
4        2015-06-25
            ...    
21901    2018-02-11
21902    2022-01-20
21903    2020-08-28
21904    2020-08-28
21905    2008-09-01
Name: 交易时间, Length: 21906, dtype: object

In [64]:
unmatched_sample_listed['交易时间']=pd.to_datetime(unmatched_sample_listed['交易时间'])

In [57]:
china_co_ids=pd.read_csv('data/source/中国大陆企业注册数据_201905.csv',usecols=['企业名称','统一社会信用代码','注册日期','企业类型','法人代表','所在省份','地区'])
china_co_ids.shape

(5888431, 7)

In [58]:
china_co_ids.duplicated().sum()

np.int64(130)

In [59]:
china_co_ids.drop_duplicates(inplace=True)

In [60]:
china_co_ids['注册日期']=pd.to_datetime(china_co_ids.注册日期)

In [67]:
china_co_ids.企业名称.nunique()

5888180

In [79]:
from fuzzywuzzy import fuzz, process

In [80]:
%%time
process.extractOne('星辰信通投资', china_co_ids['企业名称'].tolist(),scorer=fuzzywuzzy.fuzz.token_sort_ratio)

KeyboardInterrupt: 

In [81]:
from rapidfuzz import fuzz, process

In [82]:
%%time
process.extractOne('星辰信通投资', china_co_ids['企业名称'].tolist())

CPU times: total: 7.83 s
Wall time: 7.99 s


('睿信通投资集团有限公司', 65.45454545454547, 1650993)

In [85]:
process.extractOne('星辰信通投资有限公司', china_co_ids['企业名称'].tolist())

('厦门信通投资有限公司', 80.0, 650745)

In [86]:
process.extractOne('广东星辰信通科技有限公司', china_co_ids['企业名称'].tolist())

('通用科技有限公司', 84.0, 349437)

In [88]:
china_co_ids.iloc[650745]

企业名称                 厦门信通投资有限公司
统一社会信用代码     91350203705459479C
注册日期        2000-12-28 00:00:00
企业类型                        NaN
法人代表                        NaN
所在省份                         福建
地区                           厦门
Name: 650875, dtype: object

In [83]:
def fuzzy_match_and_filter(investor_name, transaction_date, company_df):
    mask=company_df.注册日期<=transaction_date
    if not mask.sum():
        return None,0,None

    company_names = company_df.loc[mask,'企业名称'].tolist()#.str.lower
    best_match, best_score, idx = process.extractOne(investor_name, company_names)
    if best_score > 80:  
        return best_match, best_score,company_df[mask].index[idx]
    # for _, row in company_df[mask].iterrows():
    #     company_name = row['企业名称']
        
    #     score = fuzz.token_sort_ratio(investor_name.lower(), company_name.lower())
        
    #     if score > best_score:
    #         best_match = full_name
    #         best_score = score
    return None, best_score,None
    # return best_match,best_score

In [84]:
%%time
unmatched_sample_listed['投资方名称_3'] ,unmatched_sample_listed['match_score'],unmatched_sample_listed['match_index']  =  zip(*unmatched_sample_listed.apply(lambda row: fuzzy_match_and_filter(row['投资方名称'], row['交易时间'], china_co_ids), axis=1))

KeyboardInterrupt: 

In [ ]:
from multiprocessing import Process
def start_task():
    p = Process(target=long_running_task)
    p.start()
    
start_task()

In [107]:
map_alias_dict

{'(Ad)Ventures': ['Revolutionary(Ad)Ventures'],
 '108度集团': ['一百零捌度（上海）企业管理中心'],
 '10T': ['10T Holdings'],
 '11.2 Capital': ['Eleven Two Capital'],
 '123RF': ['123RF LIMITED'],
 '1898 创投': ['一八九八创投基金(管理)'],
 '1898创投': ['北京一八九八创业投资基金管理中心(有限合伙)'],
 '1982 Ventures': ['1982 Ventures Pte.Ltd.'],
 '1号生活': ['黄山市屯溪区1号生活超市'],
 '20/20 HealthCare': ['20/20 HealthCare Partners'],
 '21st Century Fox': ['21st Century Fox Printing Limited'],
 '21世纪不动产': ['市南区21世纪不动产经纪中心'],
 '21世纪投资控股（中国）有限公司': ['二十一世纪天使创业投资有限公司'],
 '21世纪资本': ['21世纪投资控股(中国)有限公司'],
 '33 Capital': ['GLOBAL VCAP INVESTMENTS GROUP LIMITED'],
 '360 Capital': ['360 Capital Partners'],
 '360金融': ['360金融線上有限公司'],
 '36氪基金': ['嘉兴部道投资管理有限公司'],
 '3H Health Investment GP I': ['3H Health Investment GP I Ltd.'],
 '3K': ['Third Kind Venture Capital'],
 '3M': ['3M New Ventures'],
 '3W Fund Management': ['3W Fund Management Limited'],
 '3W Partners GP II': ['3W Partners GP II Limited'],
 '3W咖啡天使投资': ['北京3W科技有限公司'],
 '3i APTech': ['3i APTech Nominees Lim

In [309]:
unmatched_sample_listed['VC_fullname_1'] =unmatched_sample_listed['投资方名称'].map(map_alias_dict)
unmatched_sample_listed['VC_fullname_1'].count()

np.int64(4702)

In [217]:
(unmatched_sample_listed['VC_fullname_1'].dropna().map(len)>1).sum()

np.int64(13)

In [168]:
len(set(unmatched_sample_listed['投资方名称'])&set(ChinaVCtype.VC_fullname))

3022

In [300]:
ChinaVCtype['VC_fullname_1']=ChinaVCtype.VC_fullname.str.replace(' ','').str.strip()
len(set(unmatched_sample_listed['投资方名称'])&set(ChinaVCtype['VC_fullname_1']))

6217

In [310]:
mask=unmatched_sample_listed['VC_fullname_1'].isna()&unmatched_sample_listed['投资方名称'].isin(ChinaVCtype.VC_fullname)
mask.sum()

np.int64(3001)

In [219]:
unmatched_sample_listed.loc[mask,'投资方名称']

1            *ST保千
2            *ST升达
4            *ST宜生
7            *ST绿庭
8            *ST赛为
           ...    
21849    香港科学园风投基金
21851         天达银行
21853         倚格咨询
21866      宁波梅山甲鼎投
21892         火猫直播
Name: 投资方名称, Length: 3001, dtype: object

In [311]:
unmatched_sample_listed.loc[mask,'VC_fullname_1']=unmatched_sample_listed.loc[mask,'投资方名称'].apply(lambda x:[x])
unmatched_sample_listed['VC_fullname_1'].count()

np.int64(7703)

In [312]:
mask=unmatched_sample_listed['VC_fullname_1'].isna()&unmatched_sample_listed['投资方名称'].isin(ChinaVCtype['VC_fullname_1'])
mask.sum()

np.int64(3193)

In [302]:
unmatched_sample_listed.loc[mask,'投资方名称']

9                        01Advisors
10                       021Capital
17                   3RiversCapital
21                      406Ventures
22                      408Ventures
                    ...            
21799               GeneralOriental
21812    GaorongAromaHoldingLimited
21813    SCCVentureVIIIHoldcoW,Ltd.
21867           BlockchainDreamFund
21888                BiomarkCapital
Name: 投资方名称, Length: 3193, dtype: object

In [313]:
union_fullname=ChinaVCtype.set_index('VC_fullname_1')['VC_fullname'].to_dict()
unmatched_sample_listed.loc[mask,'VC_fullname_1']=unmatched_sample_listed.loc[mask,'投资方名称'].map(union_fullname).apply(lambda x:[x])
unmatched_sample_listed['VC_fullname_1'].count()

np.int64(10896)

In [315]:
Alias['Alias_1']=Alias['Alias'].str.replace(' ','').str.strip()
map_alias_dict_1=Alias.groupby('Alias_1')['VC_fullname'].apply(lambda x:list(set(x))).to_dict()

In [317]:
mask=unmatched_sample_listed['VC_fullname_1'].isna()
unmatched_sample_listed.loc[mask,'VC_fullname_1'] =unmatched_sample_listed.loc[mask,'投资方名称'].map(map_alias_dict_1)
unmatched_sample_listed['VC_fullname_1'].count()

np.int64(12089)

In [335]:
(unmatched_sample_listed['VC_fullname_1']=='').sum()

np.int64(0)

In [319]:
fullname_id = ChinaVCtype.set_index('VC_fullname')['Unified_social_credit_ID'].to_dict()
unmatched_sample_listed['Unified_social_credit_ID_1'] = unmatched_sample_listed['VC_fullname_1'].dropna().map(lambda vc_list: [fullname_id.get(vc, pd.NA) for vc in vc_list])#if pd.notna(vc_list) else pd.NA
unmatched_sample_listed['Unified_social_credit_ID_1'].count()

np.int64(12089)

In [320]:
(unmatched_sample_listed['Unified_social_credit_ID_1'].dropna().map(len)>1).sum()

np.int64(13)

In [321]:
mask=unmatched_sample_listed['Unified_social_credit_ID_1'].isna()
unmatched_sample_listed.loc[(~mask)&(unmatched_sample_listed.loc[~mask,'Unified_social_credit_ID_1'].map(len)>1),['投资方名称', '投资机构类型_x', '投资方全称','VC_fullname_1','Unified_social_credit_ID_1']]

,投资方名称,投资机构类型_x,投资方全称,VC_fullname_1,Unified_social_credit_ID_1
2318,悦榕资本,PE,NaN,"[悦榕资本有限公司, Banyan Tree Capital]","[91120116556520300K, <NA>]"
2362,捷鸿资本,VC、PE,NaN,"[捷鸿资本有限公司, 捷鸿资本（NPEA Capital）]","[nan, <NA>]"
3284,里昂证券亚太恒富资本,VC、PE,NaN,"[里昂证券亚太恒富资本(香港)有限公司, CLSA Capital Partners]","[nan, nan]"
3370,阿尔法公社,企业,NaN,"[阿尔法创业科技（北京）有限公司, 宁波梅山保税港区阿尔法天使创投投资管理有限公司]","[<NA>, 91330206MA2820MD5W]"
3573,丹华资本,VC、PE,NaN,"[DHVC, 丹晟创业投资管理(北京)有限公司]","[<NA>, 91110114MA0089Y52L]"
3967,中国加速,早期投资机构,NaN,"[Chinaccelerator, 佳速投资咨询(上海)有限公司]","[<NA>, 91310000321683631K]"
6838,凯门资本,VC、PE,NaN,"[Commonfund, The Commonfund Capital]","[nan, <NA>]"
11276,春晓资本,PE,NaN,"[北京春晓汇商股权投资管理有限公司, 石河子春晓股权投资管理有限公司]","[91110108339801843X, 91659001328755503Q]"
13400,大有资本,VC,NaN,"[青岛大有私募基金管理有限公司, 北京大有世纪投资管理有限公司]","[91370220MA3D9UQF3Y, 91110105344228471E]"
14205,正和磁系大天使基金,VC,NaN,"[正和磁系大天使基金(管理), 北京正和岛投资管理有限责任公司]","[91110108339745968E, 91110108082879708G]"


In [142]:
len(set(china_co_ids['企业名称'])&set(unmatched_sample_listed['投资方名称']))

2

In [143]:
len(set(china_co_ids['企业名称'])&set(unmatched_sample_listed['VC_fullname_1'].explode().dropna()))

329

In [458]:
fullname_id_1=china_co_ids[china_co_ids.企业名称.isin(set(china_co_ids['企业名称'])&set(unmatched_sample_listed['VC_fullname_1'].explode().dropna()))].set_index('企业名称')['统一社会信用代码'].to_dict()
len(fullname_id_1)

330

In [459]:
unmatched_sample_listed['Unified_social_credit_ID_2'] = unmatched_sample_listed['VC_fullname_1'].dropna().map(lambda vc_list: [fullname_id_1.get(vc, pd.NA) for vc in vc_list])#if pd.notna(vc_list) else pd.NA
unmatched_sample_listed['Unified_social_credit_ID_2']= unmatched_sample_listed['Unified_social_credit_ID_2'].dropna().map(lambda x: pd.NA if all(pd.isna(i) for i in x) else x)#pd.NA if pd.isna(x) or all(pd.isna(i) for i in x) else x
unmatched_sample_listed['Unified_social_credit_ID_2'].count()

np.int64(0)

In [323]:
mask=(~unmatched_sample_listed['VC_fullname_1'].isna())&(unmatched_sample_listed['Unified_social_credit_ID_1'].isna())&(~unmatched_sample_listed['Unified_social_credit_ID_2'].isna())
mask.sum()

np.int64(0)

In [324]:
# unmatched_sample_listed.loc[mask,'Unified_social_credit_ID_1']=unmatched_sample_listed.loc[mask,'Unified_social_credit_ID_2']
unmatched_sample_listed['Unified_social_credit_ID_1']= unmatched_sample_listed['Unified_social_credit_ID_1'].dropna().map(lambda x: pd.NA if all(pd.isna(i) for i in x) else ['' if pd.isna(i) else i for i in x])#pd.NA if pd.isna(x) or all(pd.isna(i) for i in x) else x
unmatched_sample_listed['Unified_social_credit_ID_1'].count()

np.int64(3827)

In [158]:
ChinaVCtype.index

RangeIndex(start=0, stop=49187, step=1)

In [160]:
def fuzzy_match_and_filter(investor_name, company_names):
    best_match, best_score, idx = process.extractOne(investor_name, company_names)
    if best_score > 80:  
        return best_match, best_score, idx
    return None, best_score,None

In [161]:
%%time
company_names=ChinaVCtype.VC_fullname.tolist()
unmatched_sample_listed['投资方名称_3'] ,unmatched_sample_listed['match_score'],unmatched_sample_listed['match_index'] =  zip(*unmatched_sample_listed.apply(lambda row: fuzzy_match_and_filter(row['投资方名称'], company_names), axis=1))

CPU times: total: 27min 19s
Wall time: 27min 49s


In [162]:
unmatched_sample_listed['投资方名称_3'].count()

np.int64(11487)

In [165]:
unmatched_sample_listed['match_score'].describe()

count    21906.000000
mean        81.831768
std         13.437951
min         30.000000
25%         72.000000
50%         81.818182
75%         94.736842
max        100.000000
Name: match_score, dtype: float64

In [197]:
(unmatched_sample_listed['match_score']==100).sum(),(unmatched_sample_listed['match_score']>90).sum(),(unmatched_sample_listed['match_score']>80).sum()

(np.int64(3022), np.int64(6298), np.int64(11487))

In [326]:
unmatched_sample_listed.loc[(unmatched_sample_listed['match_score']>80)&(unmatched_sample_listed['match_score']<85),['投资方名称','投资方名称_3','match_score','VC_fullname_1','Unified_social_credit_ID_1']]

,投资方名称,投资方名称_3,match_score,VC_fullname_1,Unified_social_credit_ID_1
67,ActionCapital,DCapital,84.000000,[Action Capital Corporation],<NA>
70,AdidasVentures,Adit Ventures,81.481481,NaN,NaN
72,AdventVenture,Advent Venture Partners,83.076923,[Advent Venture Partners],<NA>
74,AetosCapital,Aetius Capital,84.615385,"[Aetos Capital,LLC]",<NA>
84,AlphaTokenCapital,DCapital,84.000000,NaN,NaN
...,...,...,...,...,...
21848,HighlightCapital,DCapital,84.000000,NaN,NaN
21852,贵州经开创投,贵州经开创业投资管理有限公司,81.818182,[贵州经开创业投资管理有限公司],[91520114598377290X]
21878,宁波凯盟投资,宁波卓盟投资,83.333333,NaN,NaN
21883,CapitalTENII，Inc.,DCapital,84.000000,NaN,NaN


In [327]:
unmatched_sample_listed.loc[(unmatched_sample_listed['match_score']>85)&(unmatched_sample_listed['match_score']<90),['投资方名称','投资方名称_3','match_score','VC_fullname_1','Unified_social_credit_ID_1']]

,投资方名称,投资方名称_3,match_score,VC_fullname_1,Unified_social_credit_ID_1
12,10XCapital,100xCapital,85.714286,NaN,NaN
33,A-GradeInvestments,"A-Grade Investments,LLC",87.804878,"[A-Grade Investments,LLC]",<NA>
45,ADDCapital,DCapital,88.888889,[ADD Capital Corp.],<NA>
51,AMEXVentures,APEX Ventures,88.000000,[American Express Ventures],<NA>
58,AXVentures,APEX Ventures,86.956522,[Amatil X Ventures],<NA>
...,...,...,...,...,...
21503,StellarSCapital,Stellar Kapital,86.666667,[StellarS Capital Limited],<NA>
21624,GSRIIIAT,GSR II IAT,88.888889,[GSR II IAT],<NA>
21690,XAdventureManagement,Advent Management,86.486486,NaN,NaN
21890,QM108Limited,QM8 Limited,86.956522,[QM108 LIMITED],<NA>


In [328]:
unmatched_sample_listed.loc[(unmatched_sample_listed['match_score']>90)&(unmatched_sample_listed['match_score']<95),['投资方名称','投资方名称_3','match_score','VC_fullname_1','Unified_social_credit_ID_1']]

,投资方名称,投资方名称_3,match_score,VC_fullname_1,Unified_social_credit_ID_1
17,3RiversCapital,3 Rivers Capital,93.333333,[3 Rivers Capital],<NA>
31,9+Program,9+ Program,94.736842,[9+ Program],[91120116MAC20F0T9K]
37,ABCapital,AB Capital,94.736842,[AB Capital],<NA>
41,ACERedpointChinaII(HK)Limited,ACE Redpoint China II (HK) Limited,92.063492,[ACE Redpoint China II (HK) Limited],<NA>
71,AdvanceStepHoldingsLimited,Advance Step Holdings Limited,92.857143,[Advance Step Holdings Limited],<NA>
...,...,...,...,...,...
21617,上海錾义创业投资有限公司,上海创业投资有限公司,90.909091,NaN,NaN
21775,CUMedical,CU Medical,94.736842,[CU Medical],<NA>
21812,GaorongAromaHoldingLimited,Gaorong Aroma Holding Limited,94.545455,[Gaorong Aroma Holding Limited],<NA>
21813,"SCCVentureVIIIHoldcoW,Ltd.","SCC Venture VIII Holdco W,Ltd.",92.857143,"[SCC Venture VIII Holdco W,Ltd.]",<NA>


In [329]:
unmatched_sample_listed['VC_fullname_1']=unmatched_sample_listed['VC_fullname_1'].dropna().map(';'.join)
unmatched_sample_listed['Unified_social_credit_ID_1']=unmatched_sample_listed['Unified_social_credit_ID_1'].dropna().map(';'.join)

In [331]:
unmatched_sample_listed.to_excel('data\\unmatched_sample_listed_1219.xlsx')

In [340]:
unmatched_sample_listed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21906 entries, 0 to 21905
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  21906 non-null  int64         
 1   投资方名称                       21906 non-null  object        
 2   投资机构类型_x                    21846 non-null  object        
 3   投资方全称                       1024 non-null   object        
 4   投资方省份                       16 non-null     object        
 5   VC_fullname                 1474 non-null   object        
 6   Alias                       1474 non-null   object        
 7   Unified_social_credit_ID    71 non-null     object        
 8   投资方类型                       1024 non-null   object        
 9   投资方所在地                      1020 non-null   object        
 10  投资方详细地址                     120 non-null    object        
 11  投资方法人代表                     16 non-null     object        


In [341]:
(unmatched_sample_listed.VC_fullname_1==unmatched_sample_listed.VC_fullname).sum()

np.int64(693)

In [347]:
invest_all.投资方名称_unique.str.split(';').explode().dropna().nunique()

69270

In [351]:
unmatched_sample_listed.Symbol.dropna()

1434          2.0
1505     601888.0
1511     601601.0
1517     600176.0
1562     833873.0
           ...   
21647    688102.0
21713    600060.0
21729    688128.0
21780    605287.0
21877    300563.0
Name: Symbol, Length: 120, dtype: float64

In [356]:
VC_fullname_2=invest_all.投资方名称_unique.str.split(';').explode().dropna().unique()
len(set(unmatched_sample_listed['投资方名称'])&set(VC_fullname_2))

14428

In [357]:
VC_fullname_2=pd.DataFrame(VC_fullname_2,columns=['VC_fullname'])
VC_fullname_2['VC_fullname_1']=VC_fullname_2.VC_fullname.str.replace(' ','').str.strip()
mask=unmatched_sample_listed['VC_fullname_1'].isna()&unmatched_sample_listed['投资方名称'].isin(VC_fullname_2['VC_fullname_1'])
mask.sum()#6720

np.int64(9817)

In [358]:
union_fullname=VC_fullname_2.set_index('VC_fullname_1')['VC_fullname'].to_dict()
unmatched_sample_listed.loc[mask,'VC_fullname_1']=unmatched_sample_listed.loc[mask,'投资方名称'].map(union_fullname)#.apply(lambda x:[x])
unmatched_sample_listed['VC_fullname_1'].count()

np.int64(21906)

In [386]:
patent_all['申请人'].isna().sum(),patent_all['当前权利人'].isna().sum()

(np.int64(763), np.int64(1180))

In [38]:
patent_all_1=patent_all[['申请号','申请人','当前权利人','工商统一社会信用代码']].dropna()#how='all'
patent_all_1

,申请号,申请人,当前权利人,工商统一社会信用代码
462,CN85100032.0,天津大学; 交通部第一航务工程勘察设计院天津市橡胶工业研究所,天津大学; 交通部第一航务工程勘察设计院天津市橡胶工业研究所,12100000401359321Q;
463,CN85100037.1,清华大学,清华大学,12100000400000624D
464,CN85100042.8,清华大学,清华大学,12100000400000624D
465,CN85100043.6,清华大学,清华大学,12100000400000624D
466,CN85100044.4,清华大学,清华大学,12100000400000624D
...,...,...,...,...
22254927,CN202410910686.1,铁流股份有限公司,铁流股份有限公司,91330100697072273U
22254928,CN202410943641.4,广州白云山医药集团股份有限公司白云山何济公制药厂,广州白云山医药集团股份有限公司白云山何济公制药厂,914401017955021269
22254929,CN202410977428.5,天津港股份有限公司,天津港股份有限公司,911200001030643818
22254930,CN202410988446.3,宁波远洋运输股份有限公司; 海泓网络(宁波)有限公司,宁波远洋运输股份有限公司; 海泓网络(宁波)有限公司,91330200144071334Q; 91330108MA27W0KP34


In [39]:
patent_all_1['申请人']=patent_all_1['申请人'].str.split(';')
patent_all_1['当前权利人']=patent_all_1['当前权利人'].str.split(';')
patent_all_1['工商统一社会信用代码']=patent_all_1['工商统一社会信用代码'].str.split(';')
patent_all_1['申请人_C']=patent_all_1['申请人'].map(len)
patent_all_1['当前权利人_C']=patent_all_1['当前权利人'].map(len)
patent_all_1['工商统一社会信用代码_C']=patent_all_1['工商统一社会信用代码'].map(len)
patent_all_1#.explode(['申请人','当前权利人','工商统一社会信用代码'])

,申请号,申请人,当前权利人,工商统一社会信用代码,申请人_C,当前权利人_C,工商统一社会信用代码_C
462,CN85100032.0,"[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]","[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]","[12100000401359321Q, ]",2,2,2
463,CN85100037.1,[清华大学],[清华大学],[12100000400000624D],1,1,1
464,CN85100042.8,[清华大学],[清华大学],[12100000400000624D],1,1,1
465,CN85100043.6,[清华大学],[清华大学],[12100000400000624D],1,1,1
466,CN85100044.4,[清华大学],[清华大学],[12100000400000624D],1,1,1
...,...,...,...,...,...,...,...
22254927,CN202410910686.1,[铁流股份有限公司],[铁流股份有限公司],[91330100697072273U],1,1,1
22254928,CN202410943641.4,[广州白云山医药集团股份有限公司白云山何济公制药厂],[广州白云山医药集团股份有限公司白云山何济公制药厂],[914401017955021269],1,1,1
22254929,CN202410977428.5,[天津港股份有限公司],[天津港股份有限公司],[911200001030643818],1,1,1
22254930,CN202410988446.3,"[宁波远洋运输股份有限公司, 海泓网络(宁波)有限公司]","[宁波远洋运输股份有限公司, 海泓网络(宁波)有限公司]","[91330200144071334Q, 91330108MA27W0KP34]",2,2,2


In [377]:
(patent_all_1['申请人_C']!=patent_all_1['当前权利人_C']).sum()

np.int64(178449)

In [380]:
patent_all_1[patent_all_1['申请人_C']!=patent_all_1['当前权利人_C']]#北京交通大学1210000040088209X1中国国家铁路集团有限公司 91100000000013477B,山东艾琳智能科技有限公司 91370700MA3M200W5G

,申请人,当前权利人,工商统一社会信用代码,申请人_C,当前权利人_C,工商统一社会信用代码_C
638,"[鞍山钢铁公司, 鞍山钢铁公司立山医院]",[鞍山钢铁公司],[912103002414200141],2,1,1
673,"[鞍山钢铁公司, 鞍山钢铁公司无缝钢管厂]",[鞍山钢铁公司],[912103002414200141],2,1,1
781,"[哈尔滨工业大学, 黑龙江省鹤岗市半导体材料厂]",[哈尔滨工业大学],[12100000400000456B],2,1,1
1150,[中国科学院声学研究所],"[林仲茂 范国良, 中国科学院声学研究所电子工业部第十一研究所]",[12100000400883447M],1,2,1
1201,[中国科学院上海有机化学研究所],"[中国科学院上海有机化学研究所, 中国石油化工总公司上海石油化工总厂]",[12100000425005907W],1,2,1
...,...,...,...,...,...,...
22199140,[湖南国科通导时空科技有限公司],"[湖南国科通导时空科技有限公司, 长沙量子测量产业技术研究院有限公司]",[91430100MABLXCP59M],1,2,1
22215872,[沈阳航空工业发展有限公司],"[沈阳航空工业发展有限公司, 刘海涛]",[91210112MA0P5JK30W],1,2,1
22218443,[东莞市华灏技术有限公司],"[东莞市华灏技术有限公司, 深圳市华灏机电有限公司]",[91441900MA56P4AE77],1,2,1
22231821,[山东艾琳智能科技有限公司],"[山东科技职业学院, 山东艾琳智能科技有限公司]",[91370700MA3M200W5G],1,2,1


In [376]:
patent_all_1[patent_all_1['申请人_C']!=patent_all_1['工商统一社会信用代码_C']]

,申请人,当前权利人,工商统一社会信用代码,申请人_C,当前权利人_C,工商统一社会信用代码_C
462,"[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]","[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]",[12100000401359321Q],2,2,1
473,"[武汉大学, 湖北省农科院, 湖北省林科所]","[武汉大学, 湖北省农科院, 湖北省林科所]","[12100000707137123P, ]",3,3,2
474,"[北京大学, 潍坊电子计算机公司]","[北京大学, 潍坊电子计算机公司]",[12100000400002259P],2,2,1
476,"[北京大学, 潍坊电子计算机公司]","[北京大学, 潍坊电子计算机公司]",[12100000400002259P],2,2,1
602,"[鞍山钢铁公司, 鞍钢无缝钢管厂]","[鞍山钢铁公司, 鞍钢无缝钢管厂]",[912103002414200141],2,2,1
...,...,...,...,...,...,...
22254810,"[中集渔业科技有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(集团)有...","[中集渔业科技有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(集团)有...","[91440300MA5HH37L9X, 91440300618869509J]",3,3,2
22254812,"[南通中集特种运输设备制造有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装...","[南通中集特种运输设备制造有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装...","[91320600703728006Y, 91440300618869509J]",3,3,2
22254847,"[中集渔业科技有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(集团)有...","[中集渔业科技有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(集团)有...","[91440300MA5HH37L9X, 91440300618869509J]",3,3,2
22254917,"[大连中集特种物流装备有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(...","[大连中集特种物流装备有限公司, 中国国际海运集装箱(集团)股份有限公司, 中集集装箱(...","[91210242751584887D, 91440300618869509J]",3,3,2


In [387]:
patent_all_1[patent_all_1['当前权利人_C']<patent_all_1['工商统一社会信用代码_C']]

,申请人,当前权利人,工商统一社会信用代码,申请人_C,当前权利人_C,工商统一社会信用代码_C
48513,"[西北工业大学, 西北工业大学, 西安市中心医院]","[西北工业大学, 西安市中心医院]","[12100000435231705W, 12100000435231705W, 126...",3,2,3
52228,"[西安电子科技大学, 刘青, 蒲国胜]",[西安电子科技大学],"[121000004352307294, ]",3,1,2
121003,"[天津市南开太阳高技术发展有限公司, 天津市自动化仪表十厂]",[天津市南开太阳高技术发展有限公司],"[91120116238993065L, 91120105103474970M]",2,1,2
122377,"[兰州铝厂, 东北大学]",[兰州铝业股份有限公司],"[91620104224490723R, 91210102MA10YYY110]",2,1,2
139744,"[中国农业科学院柑桔研究所, 中国国际经济技术交流中心]",[中国农业科学院柑桔研究所],"[121000004504027523, 12100000400000480U]",2,1,2
...,...,...,...,...,...,...
22148907,"[北京微点科技有限公司, 孔雀表业(集团)有限公司]",[孔雀表业(集团)有限公司],"[91110112MA01QXAX31, 912106005909453539]",2,1,2
22160586,"[泉州迪特工业产品设计有限公司, 湖南科技学院]",[湖南科技学院],"[91350503798368196N, 12430000447744663G]",2,1,2
22161435,"[山西海诚智能制造有限公司, 西安重装蒲白煤矿机械有限公司, 陕西理工大学]","[西安重装蒲白煤矿机械有限公司, 陕西理工大学]","[91140122MA0JRAXK2M, 916105005521867124, 126...",3,2,3
22177772,"[深圳市志和兴实业有限公司, 深圳市智享视觉科技有限公司]",[深圳市智享视觉科技有限公司],"[91440300MA5D9XPE0A, 91440300MA5EQ48E7W]",2,1,2


In [388]:
(patent_all_1['申请人_C']<patent_all_1['工商统一社会信用代码_C']).sum()

np.int64(0)

In [440]:
mask=patent_all_1.申请人_C==patent_all_1.工商统一社会信用代码_C
co_ids=patent_all_1.loc[mask,['申请人','工商统一社会信用代码']].explode(['申请人','工商统一社会信用代码']).rename(columns={'申请人':'fullname'})
co_ids.fullname=co_ids.fullname.str.strip()
co_ids.工商统一社会信用代码=co_ids.工商统一社会信用代码.str.strip().replace('',pd.NA)
co_ids=co_ids.dropna(subset='工商统一社会信用代码').drop_duplicates()
len(co_ids)#1071916

1070701

In [423]:
mask=(patent_all_1.申请人_C!=patent_all_1.工商统一社会信用代码_C)&(patent_all_1.当前权利人_C==patent_all_1.工商统一社会信用代码_C)
mask.sum()

np.int64(18197)

In [424]:
co_ids_1=patent_all_1.loc[mask,['当前权利人','工商统一社会信用代码']].explode(['当前权利人','工商统一社会信用代码']).rename(columns={'当前权利人':'fullname'})#.drop_duplicates()
co_ids_1.fullname=co_ids_1.fullname.str.strip()
co_ids_1.工商统一社会信用代码=co_ids_1.工商统一社会信用代码.str.strip()
co_ids_1.drop_duplicates(inplace=True)
len(co_ids_1)

6312

In [425]:
len(set(co_ids_1.fullname)-set(co_ids.fullname))#2142

2142

In [433]:
co_ids_1[(~co_ids_1.fullname.isin(co_ids.fullname))&(~co_ids_1.工商统一社会信用代码.isin(co_ids.工商统一社会信用代码))]#硅谷数模(上海)半导体有限公司 &91310000MACBM2JT2X,广西净水先生智能科技有限公司,91450100MA5KDUGQ1R

,fullname,工商统一社会信用代码
29483,首都图书馆,121100004006859345
62438,交通部公路科学研究所 河北省交通厅,11130000000218173U
67255,冶金工业部建筑研究总院,9111000010000093XR
107542,桑加考姆公开有限公司,91320583753205940X
112284,华夏海南开发建设经营公司,91460000201277946U
...,...,...
20260965,硅谷数模(上海)半导体有限公司,91310115MA1K3H4211
20679212,周广驰,91130125MA0D4TFQ89
20725194,姜学锋,91370112MA3R11BJ0C
20796558,广西净水先生智能科技有限公司,91440101MA9Y9E837U


In [476]:
co_ids.groupby(['工商统一社会信用代码']).fullname.nunique().sort_values()

工商统一社会信用代码
N2340822MF3447400F    1
111000000022781538    1
11100000002403081A    1
111000000024032689    1
11100000002408253W    1
                     ..
916100002942569160    5
91320000134758212F    6
91440300192176077R    6
91430000776799294P    6
91430000796858896G    6
Name: fullname, Length: 1019902, dtype: int64

In [477]:
co_ids[co_ids.工商统一社会信用代码=='91430000796858896G']

,fullname,工商统一社会信用代码
1518949,湖南多喜爱保健科技有限公司,91430000796858896G
1747619,湖南多喜爱纺织科技有限公司,91430000796858896G
3248340,湖南多喜爱家纺股份有限公司,91430000796858896G
4236701,多喜爱家纺股份有限公司,91430000796858896G
9246008,多喜爱集团股份有限公司,91430000796858896G
9423787,浙江省建设投资集团股份有限公司,91430000796858896G


In [480]:
co_ids[co_ids.工商统一社会信用代码=='91320000134758212F']#中石化南京工程有限公司

,fullname,工商统一社会信用代码
14443,南京化学工业公司设计院,91320000134758212F
57501,南京化学工业(集团)公司设计院,91320000134758212F
287909,南化集团设计院,91320000134758212F
1021208,中国石化集团南京设计院,91320000134758212F
3231045,中国石化集团南京工程有限公司,91320000134758212F
3627949,中石化南京工程有限公司,91320000134758212F


In [479]:
co_ids[co_ids.工商统一社会信用代码=='91430000776799294P']

,fullname,工商统一社会信用代码
2235473,湖南省浏阳河生态农业科技开发有限公司,91430000776799294P
2850684,湖南省浏阳河生态农业科技开发集团有限公司,91430000776799294P
2959239,浏阳河农业集团有限公司,91430000776799294P
3474035,浏阳河农业产业集团有限公司,91430000776799294P
5557539,浏阳河农业产业集团股份有限公司,91430000776799294P
8682415,浏阳河集团股份有限公司,91430000776799294P


In [478]:
co_ids[co_ids.fullname=='浙江省建设投资集团股份有限公司']

,fullname,工商统一社会信用代码
9423787,浙江省建设投资集团股份有限公司,91430000796858896G


In [444]:
xxxx=co_ids.groupby(['fullname']).工商统一社会信用代码.nunique().sort_values()
(xxxx>1).sum(),xxxx

(np.int64(659),
 fullname
 EDF(中国)投资有限公司        1
 龙陵县畜牧工作站             1
 龙陵县石斛协会              1
 龙陵县石斛研究所             1
 龙陵县碧寨金戈土特产品加工厂       1
                     ..
 青岛日日顺物流有限公司          3
 江苏金通灵流体机械科技股份有限公司    3
 无锡华光锅炉股份有限公司         3
 杭州涛度电子商务商行           3
 扬州天富龙汽车内饰纤维有限公司      3
 Name: 工商统一社会信用代码, Length: 1070029, dtype: int64)

In [443]:
co_ids[co_ids.fullname.isin(['大连中集特种物流装备有限公司','中国国际海运集装箱(集团)股份有限公司','扬州天富龙汽车内饰纤维有限公司'])]

,fullname,工商统一社会信用代码
242468,中国国际海运集装箱(集团)股份有限公司,91440300618869509J
2072476,扬州天富龙汽车内饰纤维有限公司,913210817546452787
2357481,扬州天富龙汽车内饰纤维有限公司,913210816891862979
4994437,大连中集特种物流装备有限公司,91210242751584887D
11465341,扬州天富龙汽车内饰纤维有限公司,913210815668589460


In [427]:
co_ids.loc[242468].to_dict()

{'fullname': '中国国际海运集装箱(集团)股份有限公司', '工商统一社会信用代码': '91440300618869509J'}

In [453]:
fullname_id_2=co_ids[~co_ids.fullname.isin(xxxx[xxxx>1].index)].set_index('fullname')['工商统一社会信用代码'].to_dict()
len(fullname_id_2)

1069370

In [462]:
fullname_id = ChinaVCtype.dropna(subset='Unified_social_credit_ID').drop_duplicates().set_index('VC_fullname')['Unified_social_credit_ID'].to_dict()
len(fullname_id)

31871

In [463]:
fullname_id_1=china_co_ids.dropna(subset='统一社会信用代码').set_index('企业名称')['统一社会信用代码'].to_dict()
len(fullname_id_1)

5888180

In [ ]:
all_fullname_id=fullname_id_1.copy()
all_fullname_id.update(fullname_id_2)
all_fullname_id.update(fullname_id)
len(all_fullname_id)

In [471]:
mask=unmatched_sample_listed['Unified_social_credit_ID_1'].isna()
unmatched_sample_listed.loc[mask,'Unified_social_credit_ID_1'] = unmatched_sample_listed.loc[mask,'VC_fullname_1'].dropna().map(lambda vc: all_fullname_id.get(vc, pd.NA))#if pd.notna(vc_list) else pd.NA
unmatched_sample_listed['Unified_social_credit_ID_1'].count()

np.int64(3835)

In [473]:
unmatched_sample_listed.to_excel('data\\unmatched_sample_listed_1220.xlsx')

In [475]:
import json
with open('data\\fullname_id_all.json', 'w') as file:
    json.dump(all_fullname_id, file, ensure_ascii=ascii)

In [37]:
patent_all_1[mask]

,申请人,当前权利人,工商统一社会信用代码,申请人_C,当前权利人_C,工商统一社会信用代码_C
462,"[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]","[天津大学, 交通部第一航务工程勘察设计院天津市橡胶工业研究所]","[12100000401359321Q, ]",2,2,2
463,[清华大学],[清华大学],[12100000400000624D],1,1,1
464,[清华大学],[清华大学],[12100000400000624D],1,1,1
465,[清华大学],[清华大学],[12100000400000624D],1,1,1
466,[清华大学],[清华大学],[12100000400000624D],1,1,1
...,...,...,...,...,...,...
22254927,[铁流股份有限公司],[铁流股份有限公司],[91330100697072273U],1,1,1
22254928,[广州白云山医药集团股份有限公司白云山何济公制药厂],[广州白云山医药集团股份有限公司白云山何济公制药厂],[914401017955021269],1,1,1
22254929,[天津港股份有限公司],[天津港股份有限公司],[911200001030643818],1,1,1
22254930,"[宁波远洋运输股份有限公司, 海泓网络(宁波)有限公司]","[宁波远洋运输股份有限公司, 海泓网络(宁波)有限公司]","[91330200144071334Q, 91330108MA27W0KP34]",2,2,2


In [43]:
mask=patent_all_1.申请人_C==patent_all_1.工商统一社会信用代码_C
co_ids=patent_all_1[mask].dropna(subset=['申请人','工商统一社会信用代码']).explode(['申请人','工商统一社会信用代码']).rename(columns={'申请人':'fullname'})#drop_duplicates(['申请号'])
co_ids.fullname=co_ids.fullname.str.strip()
co_ids.工商统一社会信用代码=co_ids.工商统一社会信用代码.str.strip().replace('',pd.NA).replace('-',pd.NA)
co_ids.dropna(subset='工商统一社会信用代码',inplace=True)
##co_ids.dropna(subset='工商统一社会信用代码').drop_duplicates()

In [21]:
#一个社会信用代码对应多个公司名称
co_ids.groupby('工商统一社会信用代码')['fullname'].apply(set).reset_index(name='公司名称集')

,工商统一社会信用代码,公司名称集
0,05034320435010211A,{福建中医药大学附属康复医院}
1,11100000000018497R,{中华人民共和国北京出入境检验检疫局}
2,11100000000019414F,{中华人民共和国北京出入境边防检查总站}
3,11100000000019692W,{中华人民共和国工业和信息化部}
4,11100000000053719K,{首都机场公安局}
...,...,...
1025533,N2320612MF2068265X,{南通市通州区东社镇五马路村股份经济合作社}
1025534,N2320623MF28093851,{如东县曹埠镇甜水村股份经济合作社}
1025535,N2340223MF2175650A,{南陵县何湾镇龙山村股份经济合作社}
1025536,N2340822MF3447400F,{怀宁县小市镇平坦社区经济联合社}


In [485]:
co_ids.groupby('工商统一社会信用代码').fullname.value_counts().reset_index(name='次数') 

,工商统一社会信用代码,fullname,次数
0,05034320435010211A,福建中医药大学附属康复医院,10
1,11100000000018497R,中华人民共和国北京出入境检验检疫局,96
2,11100000000019414F,中华人民共和国北京出入境边防检查总站,2
3,11100000000019692W,中华人民共和国工业和信息化部,1
4,11100000000053719K,首都机场公安局,1
...,...,...,...
1070694,N2320612MF2068265X,南通市通州区东社镇五马路村股份经济合作社,1
1070695,N2320623MF28093851,如东县曹埠镇甜水村股份经济合作社,1
1070696,N2340223MF2175650A,南陵县何湾镇龙山村股份经济合作社,1
1070697,N2340822MF3447400F,怀宁县小市镇平坦社区经济联合社,1


In [45]:
edge_weights = co_ids.groupby(['工商统一社会信用代码', 'fullname']).申请号.nunique().reset_index(name='数量') #.tolist()
edge_weights 

,工商统一社会信用代码,fullname,数量
0,05034320435010211A,福建中医药大学附属康复医院,10
1,11100000000018497R,中华人民共和国北京出入境检验检疫局,107
2,11100000000019414F,中华人民共和国北京出入境边防检查总站,3
3,11100000000019692W,中华人民共和国工业和信息化部,1
4,11100000000053719K,首都机场公安局,1
...,...,...,...
1076964,N2320612MF2068265X,南通市通州区东社镇五马路村股份经济合作社,1
1076965,N2320623MF28093851,如东县曹埠镇甜水村股份经济合作社,1
1076966,N2340223MF2175650A,南陵县何湾镇龙山村股份经济合作社,1
1076967,N2340822MF3447400F,怀宁县小市镇平坦社区经济联合社,1


In [46]:
#公司名称对应社会信用代码
most_freq_co2social = edge_weights.loc[edge_weights.groupby('fullname')['数量'].idxmax()]
most_freq_co2social#most_freq_co2social.groupby(['fullname']).工商统一社会信用代码.nunique()==1

,工商统一社会信用代码,fullname,数量
926020,914419007578977532,(文莱)希米亚股份有限公司东莞代表处,2
141760,912101002434019177,(沈阳)中国印钞造币总公司沈阳造币技术研究所,2
33853,91110105MA00783G76,360(北京)软件有限公司,2
870005,914404003038964142,360企业安全技术(珠海)有限公司,65
176939,913100006072204056,3M中国有限公司,49
...,...,...,...
216207,91310115MA1K4KB626,龙麻(上海)医药研发有限责任公司,6
141092,911522003184924124,龙鼎(内蒙古)农业股份有限公司,7
397587,913301007154585680,龚老汉控股集团有限公司,5
74879,91110302MA018MHMX6,龟芯网(北京)科技有限公司,2


In [47]:
most_freq_co2social.工商统一社会信用代码.nunique()#1025112

1025113

In [48]:
most_freq_co2social[most_freq_co2social.工商统一社会信用代码.str.len()<16]#91441900MA4UNULL4B

,工商统一社会信用代码,fullname,数量
293719,91320411MA2360,千代众享(常州)科贸有限公司,3
20573,220000000001235,吉化集团公司,43
20581,2201091420021,吉林农大珍源参业高科技有限公司,1
20575,2200001008742,吉林多维科技股份有限公司,1
20577,2200002006058,吉林威威药业股份有限公司,4
20582,2206811250310,吉林森工健今药业有限责任公司,1
20576,2200001030155,吉林省华洋药业股份有限公司,2
20574,220000000131058,吉林省华软技术股份有限公司,1
20578,2200002006552,吉林省巨力电池有限公司,1
927590,91441900MA4U4B,广东示润科技有限公司,4


In [49]:
edge_weights[edge_weights.fullname=='江门职业技术学院']

,工商统一社会信用代码,fullname,数量
15419,124407004561749819,江门职业技术学院,71
20566,144070000290,江门职业技术学院,203


In [50]:
most_freq_co2social.set_index('fullname').to_csv('data\\patent_fullname_social_id_2024.csv')

In [19]:
fullname_id_2=most_freq_co2social.set_index('fullname')['工商统一社会信用代码'].to_dict()
len(fullname_id_2)

1076281

In [489]:
from igraph import Graph
g = Graph()
g.add_vertices(edge_weights.工商统一社会信用代码.unique().tolist())
g.add_vertices(edge_weights.fullname.unique().tolist())

In [490]:
edges = [(row['工商统一社会信用代码'], row['fullname']) for _, row in edge_weights.iterrows()]
g.add_edges(edges)
g.es['weight'] = edge_weights['次数'].tolist()

In [ ]:
try:
    clusters = g.community_walktrap(weights="weight").as_clustering()
except KeyboardInterrupt:
    print("任务已中止")

In [59]:
vc_match=pd.read_excel('data/领投投资方名称匹配1217.xlsx')
vc_match.info()#69270

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127237 entries, 0 to 127236
Data columns (total 45 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id1                       116660 non-null  float64       
 1   投资方名称                     127237 non-null  object        
 2   交易时间                      116660 non-null  datetime64[ns]
 3   id2                       127237 non-null  int64         
 4   投资机构类型_x                  127187 non-null  object        
 5   投资方全称                     107426 non-null  object        
 6   投资方省份                     115804 non-null  object        
 7   vc_fullname               106208 non-null  object        
 8   alias                     106208 non-null  object        
 9   unified_social_credit_id  98175 non-null   object        
 10  投资方类型                     104763 non-null  object        
 11  投资方所在地                    104761 non-null  object        
 12  投资

In [67]:
vc_match.投资方名称.nunique()

29620

In [71]:
china_region=pd.read_excel('data/source/(完整版)全国各省份城市明细表.xls')
china_region

,省份,城市,县，区
0,安徽,安庆,迎江区，大观区，宜秀区，怀宁县，桐城市，枞阳县，潜山县，太湖县，宿松县，望江县，岳西县
1,安徽,蚌埠,龙子湖区，蚌山区，禹会区，淮上区，五河县，固镇县，怀远县
2,安徽,亳州,谯城区，蒙城县，涡阳县，利辛县
3,安徽,池州,贵池区，青阳县，石台县，东至县
4,安徽,滁州,琅琊区，南谯区，天长市，明光市，全椒县，来安县，凤阳县，定远县
...,...,...,...
348,浙江,绍兴,越城区、柯桥区、上虞区、诸暨市、嵊州市、新昌县
349,浙江,台州,椒江区、黄岩区、路桥区、温岭市、临海市、玉环县、三门县、天台县、仙居县
350,浙江,温州,鹿城区、龙湾区、瓯海区、瑞安市、乐清市、洞头县、永嘉县、平阳县、苍南县、文成县、泰顺县
351,浙江,舟山,定海区、普陀区、岱山县、嵊泗县


In [137]:
china_region["区县"]=china_region["县，区"].str.split('，|、| |\u3000')
china_region["区县"].dropna().map(len)

0      11
1       7
2       4
3       4
4       8
       ..
347     6
348     6
349     9
350    11
351     4
Name: 区县, Length: 346, dtype: int64

In [138]:
china_region["区县"]=china_region["区县"].map(lambda x: [i.strip() for i in x if pd.notna(i) and i.strip() and len(i.strip()) > 1 and not i.strip().isdigit()] if isinstance(x,list) else [])
china_region["区县"].tolist()

[['迎江区', '大观区', '宜秀区', '怀宁县', '桐城市', '枞阳县', '潜山县', '太湖县', '宿松县', '望江县', '岳西县'],
 ['龙子湖区', '蚌山区', '禹会区', '淮上区', '五河县', '固镇县', '怀远县'],
 ['谯城区', '蒙城县', '涡阳县', '利辛县'],
 ['贵池区', '青阳县', '石台县', '东至县'],
 ['琅琊区', '南谯区', '天长市', '明光市', '全椒县', '来安县', '凤阳县', '定远县'],
 ['颍州区', '颍泉区', '颍东区', '颍上县', '界首市', '临泉县', '阜南县', '太和县'],
 ['瑶海区', '庐阳区', '蜀山区', '包河区', '肥东县', '肥西县', '长丰县', '庐江县', '巢湖市'],
 ['相山区', '杜集区', '烈山区', '濉溪县'],
 ['大通区', '田家庵区', '谢家集区', '八公山区', '潘集区', '凤台县'],
 ['屯溪区', '黄山区', '徽州区', '歙县', '休宁县', '黟县', '祁门县'],
 ['金安区', '裕安区', '寿县', '霍邱县', '霍山县', '金寨县', '舒城县'],
 ['花山区', '雨山区', '博望区', '含山县', '和县', '当涂县'],
 ['铜官山区', '狮子山区', '铜陵县'],
 ['镜湖区', '弋江区', '鸠江区', '三山区', '无为县', '芜湖县', '繁昌县', '南陵县'],
 ['埇桥区', '萧县', '砀山县', '灵璧县', '泗县'],
 ['宣州区', '郎溪县', '广德县', '宁国市', '泾县', '绩溪县', '旌德县'],
 [],
 ['鼓楼区',
  '台江区',
  '仓山区',
  '马尾区',
  '晋安区',
  '福清市',
  '长乐市',
  '闽侯县',
  '连江县',
  '罗源县',
  '闽清县',
  '永泰县,平潭'],
 ['新罗区', '漳平市', '长汀县', '永定县', '上杭县', '武平县', '连城县'],
 ['建阳区', '延平区', '邵武市', '武夷山市', '建瓯市', '顺昌县', '浦城县', '光泽县

In [ ]:
    # if text.count('（') > text.count('）'):
    #     text += '）'  # 如果左括号多，加右括号
    # elif text.count('）') > text.count('（'):
    #     text = '（' + text  

In [189]:
def split_and_clean_counties(county_str):
    if not isinstance(county_str, str) or not county_str.strip():
        return []
    county_str = re.sub(r"\d{4}年\d{1,2}月\d{1,2}日正式挂牌", "", county_str) 
    # 清理括号及其内内容，保留括号外的内容
    while '(' in county_str or '（' in county_str:
        match = re.search(r"[（(](.*?)[)）]", county_str)
        if match:
            inner_text = match.group(1).strip()
            # 如果只有括号内内容，保留括号内的内容并去掉括号
            if county_str.strip() == f"({inner_text})" or county_str.strip() == f"（{inner_text}）":
                county_str = inner_text
                break
        county_str = re.sub(r"[（(].*?[)）]", "", county_str).strip()
        
    county_list = re.split(r'，|、| |\u3000', county_str)
    cleaned_counties = []
    for county in county_list:
        if not county or pd.isna(county) or len(county.strip()) <= 1 or county.strip().isdigit():
            continue
        if county in  ['高新区','开发区', '城区', '矿区', '郊区']: #exclude_list
            continue
        # 排除形如 "数字+个" 的字符串
        if re.match(r".*\d+个.*", county):#'辖7个街道', '10个镇', '6个管理区',，'3个街道', '20个镇'，
            continue
 
        if county:
            cleaned_counties.append(county)
    return cleaned_counties
    
china_region["区县"] = china_region["县，区"].map(split_and_clean_counties)
china_region["区县"].tolist()

[['迎江区', '大观区', '宜秀区', '怀宁县', '桐城市', '枞阳县', '潜山县', '太湖县', '宿松县', '望江县', '岳西县'],
 ['龙子湖区', '蚌山区', '禹会区', '淮上区', '五河县', '固镇县', '怀远县'],
 ['谯城区', '蒙城县', '涡阳县', '利辛县'],
 ['贵池区', '青阳县', '石台县', '东至县'],
 ['琅琊区', '南谯区', '天长市', '明光市', '全椒县', '来安县', '凤阳县', '定远县'],
 ['颍州区', '颍泉区', '颍东区', '颍上县', '界首市', '临泉县', '阜南县', '太和县'],
 ['瑶海区', '庐阳区', '蜀山区', '包河区', '肥东县', '肥西县', '长丰县', '庐江县', '巢湖市'],
 ['相山区', '杜集区', '烈山区', '濉溪县'],
 ['大通区', '田家庵区', '谢家集区', '八公山区', '潘集区', '凤台县'],
 ['屯溪区', '黄山区', '徽州区', '歙县', '休宁县', '黟县', '祁门县'],
 ['金安区', '裕安区', '寿县', '霍邱县', '霍山县', '金寨县', '舒城县'],
 ['花山区', '雨山区', '博望区', '含山县', '和县', '当涂县'],
 ['铜官山区', '狮子山区', '铜陵县'],
 ['镜湖区', '弋江区', '鸠江区', '三山区', '无为县', '芜湖县', '繁昌县', '南陵县'],
 ['埇桥区', '萧县', '砀山县', '灵璧县', '泗县'],
 ['宣州区', '郎溪县', '广德县', '宁国市', '泾县', '绩溪县', '旌德县'],
 [],
 ['鼓楼区',
  '台江区',
  '仓山区',
  '马尾区',
  '晋安区',
  '福清市',
  '长乐市',
  '闽侯县',
  '连江县',
  '罗源县',
  '闽清县',
  '永泰县,平潭'],
 ['新罗区', '漳平市', '长汀县', '永定县', '上杭县', '武平县', '连城县'],
 ['建阳区', '延平区', '邵武市', '武夷山市', '建瓯市', '顺昌县', '浦城县', '光泽县

In [190]:
county_ct=china_region["区县"].explode().value_counts()
(county_ct>1).sum()

np.int64(23)

In [191]:
county_ct[county_ct>1]

区县
鼓楼区     4
市中区     4
新城区     3
新华区     3
铁西区     3
桥西区     3
甘肃矿区    2
铁东区     2
白云区     2
桥东区     2
西安区     2
南山区     2
新市区     2
城中区     2
城关区     2
向阳区     2
新和县     2
武宁县     2
青山区     2
长安区     2
清河区     2
海州区     2
西湖区     2
Name: count, dtype: int64

In [194]:
china_region["区县"]=china_region["区县"].map(';'.join)
china_region.to_excel('data/全国各省份城市明细表.xlsx')

In [61]:
vc_match.columns

Index(['id1', '投资方名称', '交易时间', 'id2', '投资机构类型_x', '投资方全称', '投资方省份',
       'vc_fullname', 'alias', 'unified_social_credit_id', '投资方类型', '投资方所在地',
       '投资方详细地址', '投资方法人代表', 'GPLP类型', '投资方区县', '投资方城市', '投资方统一社会信用代码',
       '投资方经营范围', '投资方行业', 'qcc_原文件导入名称', 'qcc_统一社会信用代码', 'qcc_系统匹配企业名称',
       'qcc_登记状态', 'qcc_法定代表人', 'qcc_所属省份', 'qcc_所属城市', 'qcc_所属区县', 'qcc_曾用名',
       'qcc_成立日期', 'qcc_企业地址', 'qcc_国标行业门类', 'qcc_国标行业大类', 'qcc_国标行业中类',
       'qcc_国标行业小类', 'qcc_企查查行业门类', 'qcc_企查查行业大类', 'qcc_企查查行业中类',
       'qcc_企查查行业小类', 'qcc_经营范围', 'op_jc_id1', 'op_jc_id2', 'op_jc_机构名称',
       'op_jc_similscore', '_merge'],
      dtype='object')

In [70]:
vc_match[['投资方名称',  'vc_fullname', '投资方省份','投资方城市','投资方区县','投资方所在地','投资方详细地址' , 'qcc_所属省份', 'qcc_所属城市', 'qcc_所属区县',  'qcc_企业地址']]

,投资方名称,vc_fullname,投资方省份,投资方城市,投资方区县,投资方所在地,投资方详细地址,qcc_所属省份,qcc_所属城市,qcc_所属区县,qcc_企业地址
0,*ST必康,NaN,陕西省,NaN,NaN,陕西省-延安市,陕西省延安市宝塔区创新创业小镇E区,NaN,NaN,NaN,NaN
1,*ST慧辰,NaN,北京市,NaN,NaN,北京市-海淀区,北京市朝阳区酒仙桥中路18号南楼1层;北京市朝阳区酒仙桥中路18号国投创意信息产业园南楼1层,NaN,NaN,NaN,NaN
2,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,NaN,NaN,NaN
3,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,NaN,NaN,NaN
4,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
127232,龙虎网,江苏龙虎网信息科技股份有限公司,江苏省,南京市,秦淮区,NaN,南京市秦淮区永智路6号,天津市,天津市,滨海新区,天津生态城动漫中路482号创智大厦204（TG第567号）
127233,龙裕慧投股权投资,舟山龙裕慧投股权投资合伙企业（有限合伙）,湖北省,襄阳市,襄城区,NaN,襄南大道77号富春山居A2地块3幢1层2室,NaN,NaN,NaN,NaN
127234,龙财盘实,黑龙江龙财盘实高新技术创业股权投资基金管理有限公司,黑龙江省,哈尔滨市,南岗区,黑龙江省-哈尔滨市,哈尔滨市道里区友谊路428号交银大厦23层;黑龙江省哈尔滨市松北区创新路1616号15#,山西省,太原市,迎泽区,山西省太原市迎泽区迎泽街道新建南路156号
127235,龙金（北京）投资管理,龙金（北京）投资管理有限公司,北京市,北京市,海淀区,NaN,北京市海淀区太月园1号楼6层6007号,四川省,成都市,金牛区,成都市金牛区二环路西三段181号15层16号


#投资方省份和城市，基于投资方详细地址和省份、城市的明细，再补全一下nan值

In [192]:
province_list=china_region["省份"].unique().tolist()
other_county=county_ct[county_ct>1].index.tolist()
city_county_map=china_region.set_index('城市')["区县"].to_dict()
# province_city_map=china_region.set_index('省份')["城市"].to_dict()
county_to_city = {}
city_to_province = {}
for _, row in china_region.iterrows():
    province, city, counties = row["省份"], row["城市"], row["区县"]
    city_to_province[city] = province
    for county in counties:
        if county in other_county:
            continue
        county_to_city[county] = city

In [177]:
import re
def clean_suffix(name):
    return name  if (len(name) <= 2) else re.sub(r"(县|区|市)$", "", name) # 去掉“县”、“区”、“市”结尾的后缀,or name in['高新区','城区', '矿区', '郊区']
    
def complete_address(address):
    province, city, county = None, None, None
    
    for c in county_to_city:
        if c in address or clean_suffix(c) in address :
            county = c
            city = county_to_city[c]
            province = city_to_province[city]
            break
    
    #if not city:
    for c in city_to_province:
        if c in address or c.rstrip("市") in address:
            if county and city!=c:
                county=None
                
            city = c
            province = city_to_province[city]
            break

    # if not province:
    for p in province_list:
        if p in address:
            if city and province!=p:
                city=None
                county=None
                
            province = p
            break
            
    if city and not county:
        for c in set(city_county_map[city])&set(other_county):
            if c in address or clean_suffix(c) in address:
                county = c
                break

    return province, city, county #{'province':province, 'city':city, 'county':county}

In [171]:
xxx=vc_match["投资方详细地址"].str.split(';').map(len)
(xxx>1).sum()

np.int64(69557)

In [172]:
vc_match.loc[xxx>1,"投资方详细地址"]

1            北京市朝阳区酒仙桥中路18号南楼1层;北京市朝阳区酒仙桥中路18号国投创意信息产业园南楼1层
7             北京市朝阳区光华路15号院2号楼铜牛国际大厦6F;北京市朝阳区光华路15号院1号楼1401
8             北京市朝阳区光华路15号院2号楼铜牛国际大厦6F;北京市朝阳区光华路15号院1号楼1401
10        北京市朝阳区酒仙桥十街坊一号院3003-3006;北京市朝阳区姚家园路105号1号楼15层1805
11        北京市朝阳区酒仙桥十街坊一号院3003-3006;北京市朝阳区姚家园路105号1号楼15层1805
                                ...                        
127180               上海市静安区江场路1377弄1号楼22F;上海市闸北区共和新路4666弄1号
127192     北京市朝阳区三间房西路1号院3号楼1层3-1号-36室;北京市海淀区尚峰尚水尚居园5-1-301
127197    深圳市南山区招商街道商海路太子湾商务大厦4栋1202;广东省深圳市南山区科技中三路海王银河科...
127199    湖北省武汉市洪山区光谷中心花园A座15层06号;武汉市东湖新技术开发区珞喻路889号武汉光谷...
127234         哈尔滨市道里区友谊路428号交银大厦23层;黑龙江省哈尔滨市松北区创新路1616号15#
Name: 投资方详细地址, Length: 69557, dtype: object

In [176]:
vc_match.loc[xxx>1,"投资方详细地址"].str.split(';').str[0]

1                  北京市朝阳区酒仙桥中路18号南楼1层
7            北京市朝阳区光华路15号院2号楼铜牛国际大厦6F
8            北京市朝阳区光华路15号院2号楼铜牛国际大厦6F
10           北京市朝阳区酒仙桥十街坊一号院3003-3006
11           北京市朝阳区酒仙桥十街坊一号院3003-3006
                     ...             
127180           上海市静安区江场路1377弄1号楼22F
127192    北京市朝阳区三间房西路1号院3号楼1层3-1号-36室
127197     深圳市南山区招商街道商海路太子湾商务大厦4栋1202
127199        湖北省武汉市洪山区光谷中心花园A座15层06号
127234          哈尔滨市道里区友谊路428号交银大厦23层
Name: 投资方详细地址, Length: 69557, dtype: object

In [159]:
%%time
vc_match["province"], vc_match["city"],  vc_match["county"] = zip(*vc_match["投资方详细地址"].apply(complete_address))

CPU times: total: 2min 21s
Wall time: 2min 24s


In [193]:
%%time
vc_match["province"], vc_match["city"],  vc_match["county"] = zip(*vc_match["投资方详细地址"].str.split(';').str[0].apply(complete_address))

CPU times: total: 2min 26s
Wall time: 2min 28s


In [179]:
vc_match[['投资方名称',  'vc_fullname', '投资方省份','投资方城市','投资方区县','投资方所在地','投资方详细地址' ,  'qcc_企业地址', 'province', 'city', 'county']]

,投资方名称,vc_fullname,投资方省份,投资方城市,投资方区县,投资方所在地,投资方详细地址,qcc_企业地址,province,city,county
0,*ST必康,NaN,陕西省,NaN,NaN,陕西省-延安市,陕西省延安市宝塔区创新创业小镇E区,NaN,陕西,延安,宝塔区
1,*ST慧辰,NaN,北京市,NaN,NaN,北京市-海淀区,北京市朝阳区酒仙桥中路18号南楼1层;北京市朝阳区酒仙桥中路18号国投创意信息产业园南楼1层,NaN,北京,北京,None
2,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,浙江,绍兴,新昌县
3,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,浙江,绍兴,新昌县
4,*ST美盛,NaN,浙江省,NaN,NaN,浙江省-绍兴市,浙江省新昌省级高新技术园区内（南岩）,NaN,浙江,绍兴,新昌县
...,...,...,...,...,...,...,...,...,...,...,...
127232,龙虎网,江苏龙虎网信息科技股份有限公司,江苏省,南京市,秦淮区,NaN,南京市秦淮区永智路6号,天津生态城动漫中路482号创智大厦204（TG第567号）,江苏,南京,秦淮区
127233,龙裕慧投股权投资,舟山龙裕慧投股权投资合伙企业（有限合伙）,湖北省,襄阳市,襄城区,NaN,襄南大道77号富春山居A2地块3幢1层2室,NaN,None,None,None
127234,龙财盘实,黑龙江龙财盘实高新技术创业股权投资基金管理有限公司,黑龙江省,哈尔滨市,南岗区,黑龙江省-哈尔滨市,哈尔滨市道里区友谊路428号交银大厦23层;黑龙江省哈尔滨市松北区创新路1616号15#,山西省太原市迎泽区迎泽街道新建南路156号,黑龙江,哈尔滨,道里区
127235,龙金（北京）投资管理,龙金（北京）投资管理有限公司,北京市,北京市,海淀区,NaN,北京市海淀区太月园1号楼6层6007号,成都市金牛区二环路西三段181号15层16号,北京,北京,None


In [180]:
province, city, county = None, None, None
address="浙江省新昌省级高新技术园区内（南岩）"
for c in county_to_city:
    if c in address or clean_suffix(c) in address :
        county = c
        city = county_to_city[c]
        province = city_to_province[city]
        break
province, city, county

('浙江', '绍兴', '新昌县')

In [196]:
vc_match[['投资方名称',  'vc_fullname', '投资方省份','投资方城市','投资方区县','投资方所在地','投资方详细地址' ,  'qcc_企业地址', 'province', 'city', 'county']].count()#117317,24440/117198,33307

投资方名称          127237
vc_fullname    106208
投资方省份          115804
投资方城市           84158
投资方区县           84158
投资方所在地         104761
投资方详细地址        127237
qcc_企业地址        93957
province       123023
city           116969
county          36991
dtype: int64

In [195]:
vc_match.to_excel('data/领投投资方名称匹配1221.xlsx')